# Seq2seq translation in Flux
In this notebook, I share the code I wrote to make a seq2seq nmt-model (neural machine translation model) to translate simple english sentences to french.
The code is written in Julia using Flux.

### note:
For some reason, when I train the model for some epochs, I get gibberish results.

|Input (Eng)|Prediction (*Fr*)|Expected (Fr)|
| - | - | - |
|"You are too skinny"|"Vous êtes ' que . . . . . . . ."| "Vous êtes trop maigre"  |
|"He is painting a picture"|"Il est est de . . . . . . . ."|"Il est en train de peindre un tableau"|
| ... | ... | ... |
If you know what I'm doing wrong, please do let me know!

# The data
The english-french sentence pairs dataset is found on this website: http://www.manythings.org/anki/fra-eng.zip.  
For the data preparation, I mainly follow the official Pytorch tutorial on seq2seq models: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html.

We create a `Lang` struct which holds two dictionaries to convert words to indices and back. Every `Lang` instance gets instantiated with a SOS-(start of sentence), EOS(end of sentence)-, UNK(unknown word) and PAD(padding)-token.  
Padding is necessary because we will be training in batches of differently sized sentences.


*Since the data is relatively small (a lot of sentences get filtered out), we keep all words instead of discarding scarcely used words.  
This means the `UNK` token will not be used.*

The function `readLangs` takes the text file, splits up the sentences (which are tab-delimited) and intantiates two new languages (lang1 and lang2). and assigns them to two newly created languages.

In [1]:
using CuArrays, Flux, Statistics, Random

FILE = "D:/downloads/fra-eng/fra.txt"

mutable struct Lang
    name
    word2index
    word2count
    index2word
    n_words
end

Lang(name) = Lang(
    name,
    Dict{String, Int}(),
    Dict{String, Int}(),
    Dict{Int, String}(1=>"SOS", 2=>"EOS", 3=>"UNK", 4=>"PAD"),
    4)

function (l::Lang)(sentence::String)
    for word in split(sentence, " ")
            if word ∉ keys(l.word2index)
                l.word2index[word] = l.n_words + 1
                l.word2count[word] = 1
                l.index2word[l.n_words + 1] = word
                l.n_words += 1
            else
                l.word2count[word] += 1
            end
    end
end

function normalizeString(s)
    s = strip(lowercase(s))
    s = replace(s, r"([.!?,])"=>s" \1")
    s = replace(s, "'"=>" ' ")
    return s
end

function readLangs(lang1, lang2; rev=false)
    println("Reading lines...")
    lines = readlines(FILE)
    pairs = [normalizeString.(pair) for pair in split.(lines, "\t")]
    if rev
        pairs = reverse.(pairs)
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    end
    return(input_lang, output_lang, pairs)
end

readLangs (generic function with 1 method)

As suggested in the Pytorch tutorial, we create a function to filter out sentences that don't start with `english_prefixes` ("i am", "i'm", "you are"...), as well as sentences that exceed the `MAX_LENGTH` (which is set to 10).

The function `prepareData` takes the names of two languages and creates these language instances as well as the sentence pairs by calling `readLangs`.  
After the sentence pairs get filtered (with `filterPair`), every unique word in a sentence get's added to the corresponding language's vocabulary (`word2index`, `index2word`, `n_words`) while every additional instance of a word increments `n_words` by 1.

Sentences from the input language are added to `xs`, target sentences are added to `ys`. Finally, inputs (`xs`) and targets (`ys`) are shuffled.

In [2]:
MAX_LENGTH = 10

eng_prefixes = [
    "i am ", "i ' m ",
    "he is ", "he ' s ",
    "she is ", "she ' s ",
    "you are ", "you ' re ",
    "we are ", "we ' re ",
    "they are ", "they ' re "]
        
function filterPair(pair)
    return(false ∉ (length.(split.(pair, " ")) .<= MAX_LENGTH) && true ∈ (startswith.(pair[1], eng_prefixes)))
end

function prepareData(lang1, lang2; rev=false)
    input_lang, output_lang, pairs = readLangs(lang1, lang2; rev=rev)
    println("Read $(length(pairs)) sentence pairs.")
    pairs = [pair for pair in pairs if filterPair(pair)]
    println("Trimmed to $(length(pairs)) sentence pairs.\n")
    xs = []
    ys = []
    for pair in pairs
        push!(xs, pair[1])
        push!(ys, pair[2])
    end
    println("Counting words...")
    for pair in pairs
        input_lang(pair[2])
        output_lang(pair[1])
    end
    println("Counted words:")
    println("• ", input_lang.name, ": ", input_lang.n_words)
    println("• ", output_lang.name, ": ", output_lang.n_words)
    return(input_lang, output_lang, xs, ys)
end

fr, eng, xs, ys = prepareData("fr", "eng")
indices = shuffle([1:length(xs)...])
xs = xs[indices]
ys = ys[indices];

Reading lines...
Read 154883 sentence pairs.
Trimmed to 11645 sentence pairs.

Counting words...
Counted words:
• fr: 4831
• eng: 3047


The function `indexesFromSentence` takes a language's `word2index` and maps all the words in a sentence to a index, later this index will get used to get the word's embedding. Note that, at the end of every sentence, the `EOS`-index (2) gets added, this is for the model to know when to stop predicting during inference. 

To make batches for mini-batch training, the data (`[indexesFromSentence.([eng], xs), indexesFromSentence.([fr], ys)]`) gets split in chunks of `BATCH_SIZE`. Since sentences in a chunk often have different lengths, the `PAD`-index (4), gets added to the end of sentences to make them as long as the longest sentence of the chunk.

To be able to easily pass a chunk to an RNN, the n<sup>th</sup> word of every sentence in the chunk get placed next to each other in an array. Also, all the words get OneHot encoded.

![batching](data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz4KPHN2ZyB4bWxucz0iaHR0cDov%0D%0AL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5%0D%0AL3hsaW5rIiB3aWR0aD0iODQxLjY4cHQiIGhlaWdodD0iNTk1LjJwdCIgdmlld0JveD0iMCAwIDg0%0D%0AMS42OCA1OTUuMiIgdmVyc2lvbj0iMS4xIj4KPGRlZnM+CjxnPgo8c3ltYm9sIG92ZXJmbG93PSJ2%0D%0AaXNpYmxlIiBpZD0iZ2x5cGgwLTAiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDQu%0D%0AMjk2ODc1IC01LjY4NzUgTCA0LjI5Njg3NSAwIEwgMC4xODc1IDAgTCAwLjE4NzUgLTUuNjg3NSBa%0D%0AIE0gMy45MjE4NzUgLTAuMzU5Mzc1IEwgMy45MjE4NzUgLTUuMzI4MTI1IEwgMC41NjI1IC01LjMy%0D%0AODEyNSBMIDAuNTYyNSAtMC4zNTkzNzUgWiBNIDMuMzc1IC0zLjc1IEMgMy4zNzUgLTMuNTgyMDMx%0D%0AIDMuMzQ3NjU2IC0zLjQyOTY4OCAzLjI5Njg3NSAtMy4yOTY4NzUgQyAzLjI0MjE4OCAtMy4xNjAx%0D%0ANTYgMy4xNzE4NzUgLTMuMDQ2ODc1IDMuMDc4MTI1IC0yLjk1MzEyNSBDIDIuOTg0Mzc1IC0yLjg1%0D%0AOTM3NSAyLjg3NSAtMi43ODUxNTYgMi43NSAtMi43MzQzNzUgQyAyLjYyNSAtMi42Nzk2ODggMi40%0D%0AODgyODEgLTIuNjQ4NDM4IDIuMzQzNzUgLTIuNjQwNjI1IEwgMi4zMTI1IC0xLjk4NDM3NSBDIDIu%0D%0AMzEyNSAtMS45NTMxMjUgMi4yODkwNjIgLTEuOTI5Njg4IDIuMjUgLTEuOTIxODc1IEMgMi4yMDcw%0D%0AMzEgLTEuOTEwMTU2IDIuMTQ0NTMxIC0xLjkwNjI1IDIuMDYyNSAtMS45MDYyNSBDIDIuMDE5NTMx%0D%0AIC0xLjkwNjI1IDEuOTg0Mzc1IC0xLjkwNjI1IDEuOTUzMTI1IC0xLjkwNjI1IEMgMS45Mjk2ODgg%0D%0ALTEuOTA2MjUgMS45MTAxNTYgLTEuOTA2MjUgMS44OTA2MjUgLTEuOTA2MjUgQyAxLjg2NzE4OCAt%0D%0AMS45MTQwNjIgMS44NTE1NjIgLTEuOTI1NzgxIDEuODQzNzUgLTEuOTM3NSBDIDEuODMyMDMxIC0x%0D%0ALjk0NTMxMiAxLjgyODEyNSAtMS45NjA5MzggMS44MjgxMjUgLTEuOTg0Mzc1IEwgMS44MTI1IC0y%0D%0ALjc4MTI1IEMgMS44MDA3ODEgLTIuODc1IDEuODE2NDA2IC0yLjkzNzUgMS44NTkzNzUgLTIuOTY4%0D%0ANzUgQyAxLjg5ODQzOCAtMy4wMDc4MTIgMS45NTcwMzEgLTMuMDMxMjUgMi4wMzEyNSAtMy4wMzEy%0D%0ANSBMIDIuMTA5Mzc1IC0zLjAzMTI1IEMgMi4yNDIxODggLTMuMDMxMjUgMi4zNTE1NjIgLTMuMDQ2%0D%0AODc1IDIuNDM3NSAtMy4wNzgxMjUgQyAyLjUzMTI1IC0zLjExNzE4OCAyLjYwMTU2MiAtMy4xNjQw%0D%0ANjIgMi42NTYyNSAtMy4yMTg3NSBDIDIuNzA3MDMxIC0zLjI4MTI1IDIuNzQyMTg4IC0zLjM1MTU2%0D%0AMiAyLjc2NTYyNSAtMy40Mzc1IEMgMi43ODUxNTYgLTMuNTMxMjUgMi43OTY4NzUgLTMuNjI1IDIu%0D%0ANzk2ODc1IC0zLjcxODc1IEMgMi43OTY4NzUgLTMuOTM3NSAyLjczNDM3NSAtNC4xMDkzNzUgMi42%0D%0AMDkzNzUgLTQuMjM0Mzc1IEMgMi40OTIxODggLTQuMzU5Mzc1IDIuMzA0Njg4IC00LjQyMTg3NSAy%0D%0ALjA0Njg3NSAtNC40MjE4NzUgQyAxLjkyOTY4OCAtNC40MjE4NzUgMS44MjgxMjUgLTQuNDEwMTU2%0D%0AIDEuNzM0Mzc1IC00LjM5MDYyNSBDIDEuNjQ4NDM4IC00LjM2NzE4OCAxLjU3MDMxMiAtNC4zNDM3%0D%0ANSAxLjUgLTQuMzEyNSBDIDEuNDM3NSAtNC4yODEyNSAxLjM4MjgxMiAtNC4yNTM5MDYgMS4zNDM3%0D%0ANSAtNC4yMzQzNzUgQyAxLjMwMDc4MSAtNC4yMTA5MzggMS4yNjk1MzEgLTQuMjAzMTI1IDEuMjUg%0D%0ALTQuMjAzMTI1IEMgMS4yMzgyODEgLTQuMjAzMTI1IDEuMjI2NTYyIC00LjIwMzEyNSAxLjIxODc1%0D%0AIC00LjIwMzEyNSBDIDEuMjA3MDMxIC00LjIxMDkzOCAxLjE5NTMxMiAtNC4yMjI2NTYgMS4xODc1%0D%0AIC00LjIzNDM3NSBDIDEuMTg3NSAtNC4yNTM5MDYgMS4xNzk2ODggLTQuMjczNDM4IDEuMTcxODc1%0D%0AIC00LjI5Njg3NSBDIDEuMTcxODc1IC00LjMyODEyNSAxLjE3MTg3NSAtNC4zNjcxODggMS4xNzE4%0D%0ANzUgLTQuNDIxODc1IEMgMS4xNzE4NzUgLTQuNDcyNjU2IDEuMTcxODc1IC00LjUxNTYyNSAxLjE3%0D%0AMTg3NSAtNC41NDY4NzUgQyAxLjE3OTY4OCAtNC41NzgxMjUgMS4xOTUzMTIgLTQuNjAxNTYyIDEu%0D%0AMjE4NzUgLTQuNjI1IEMgMS4yMzgyODEgLTQuNjQ0NTMxIDEuMjgxMjUgLTQuNjY0MDYyIDEuMzQz%0D%0ANzUgLTQuNjg3NSBDIDEuNDA2MjUgLTQuNzE4NzUgMS40NzI2NTYgLTQuNzQyMTg4IDEuNTQ2ODc1%0D%0AIC00Ljc2NTYyNSBDIDEuNjI4OTA2IC00Ljc5Njg3NSAxLjcxODc1IC00LjgxNjQwNiAxLjgxMjUg%0D%0ALTQuODI4MTI1IEMgMS45MTQwNjIgLTQuODQ3NjU2IDIuMDE1NjI1IC00Ljg1OTM3NSAyLjEwOTM3%0D%0ANSAtNC44NTkzNzUgQyAyLjMyODEyNSAtNC44NTkzNzUgMi41MTU2MjUgLTQuODI4MTI1IDIuNjcx%0D%0AODc1IC00Ljc2NTYyNSBDIDIuODM1OTM4IC00LjcxMDkzOCAyLjk2ODc1IC00LjYzMjgxMiAzLjA2%0D%0AMjUgLTQuNTMxMjUgQyAzLjE2NDA2MiAtNC40Mzc1IDMuMjQyMTg4IC00LjMyMDMxMiAzLjI5Njg3%0D%0ANSAtNC4xODc1IEMgMy4zNDc2NTYgLTQuMDUwNzgxIDMuMzc1IC0zLjkwNjI1IDMuMzc1IC0zLjc1%0D%0AIFogTSAyLjQwNjI1IC0xLjIwMzEyNSBDIDIuNDA2MjUgLTEuMTQwNjI1IDIuMzk4NDM4IC0xLjA4%0D%0ANTkzOCAyLjM5MDYyNSAtMS4wNDY4NzUgQyAyLjM3ODkwNiAtMS4wMDM5MDYgMi4zNjMyODEgLTAu%0D%0AOTY4NzUgMi4zNDM3NSAtMC45Mzc1IEMgMi4zMjAzMTIgLTAuOTE0MDYyIDIuMjg5MDYyIC0wLjg5%0D%0AODQzOCAyLjI1IC0wLjg5MDYyNSBDIDIuMjA3MDMxIC0wLjg3ODkwNiAyLjE1NjI1IC0wLjg3NSAy%0D%0ALjA5Mzc1IC0wLjg3NSBDIDIuMDE5NTMxIC0wLjg3NSAxLjk2MDkzOCAtMC44Nzg5MDYgMS45MjE4%0D%0ANzUgLTAuODkwNjI1IEMgMS44Nzg5MDYgLTAuODk4NDM4IDEuODQzNzUgLTAuOTE0MDYyIDEuODEy%0D%0ANSAtMC45Mzc1IEMgMS43ODkwNjIgLTAuOTY4NzUgMS43NzM0MzggLTEuMDAzOTA2IDEuNzY1NjI1%0D%0AIC0xLjA0Njg3NSBDIDEuNzUzOTA2IC0xLjA4NTkzOCAxLjc1IC0xLjE0MDYyNSAxLjc1IC0xLjIw%0D%0AMzEyNSBDIDEuNzUgLTEuMjY1NjI1IDEuNzUzOTA2IC0xLjMxNjQwNiAxLjc2NTYyNSAtMS4zNTkz%0D%0ANzUgQyAxLjc3MzQzOCAtMS4zOTg0MzggMS43ODkwNjIgLTEuNDI5Njg4IDEuODEyNSAtMS40NTMx%0D%0AMjUgQyAxLjg0Mzc1IC0xLjQ4NDM3NSAxLjg3ODkwNiAtMS41MDM5MDYgMS45MjE4NzUgLTEuNTE1%0D%0ANjI1IEMgMS45NjA5MzggLTEuNTIzNDM4IDIuMDE5NTMxIC0xLjUzMTI1IDIuMDkzNzUgLTEuNTMx%0D%0AMjUgQyAyLjE1NjI1IC0xLjUzMTI1IDIuMjA3MDMxIC0xLjUyMzQzOCAyLjI1IC0xLjUxNTYyNSBD%0D%0AIDIuMjg5MDYyIC0xLjUwMzkwNiAyLjMyMDMxMiAtMS40ODQzNzUgMi4zNDM3NSAtMS40NTMxMjUg%0D%0AQyAyLjM2MzI4MSAtMS40Mjk2ODggMi4zNzg5MDYgLTEuMzk4NDM4IDIuMzkwNjI1IC0xLjM1OTM3%0D%0ANSBDIDIuMzk4NDM4IC0xLjMxNjQwNiAyLjQwNjI1IC0xLjI2NTYyNSAyLjQwNjI1IC0xLjIwMzEy%0D%0ANSBaIE0gMCAxLjc1IFogTSAwIDEuNzUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2%0D%0AaXNpYmxlIiBpZD0iZ2x5cGgwLTEiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDQu%0D%0AMjUgLTIuODU5Mzc1IEMgNC4yNSAtMi40MjE4NzUgNC4yMTA5MzggLTIuMDE5NTMxIDQuMTQwNjI1%0D%0AIC0xLjY1NjI1IEMgNC4wNzgxMjUgLTEuMzAwNzgxIDMuOTYwOTM4IC0wLjk5MjE4OCAzLjc5Njg3%0D%0ANSAtMC43MzQzNzUgQyAzLjY0MDYyNSAtMC40NzI2NTYgMy40Mjk2ODggLTAuMjY5NTMxIDMuMTcx%0D%0AODc1IC0wLjEyNSBDIDIuOTIxODc1IDAuMDA3ODEyNSAyLjYwMTU2MiAwLjA3ODEyNSAyLjIxODc1%0D%0AIDAuMDc4MTI1IEMgMS44NjMyODEgMC4wNzgxMjUgMS41NjI1IDAuMDE1NjI1IDEuMzEyNSAtMC4x%0D%0AMDkzNzUgQyAxLjA3MDMxMiAtMC4yMzQzNzUgMC44NzUgLTAuNDIxODc1IDAuNzE4NzUgLTAuNjcx%0D%0AODc1IEMgMC41NzAzMTIgLTAuOTIxODc1IDAuNDY4NzUgLTEuMjIyNjU2IDAuNDA2MjUgLTEuNTc4%0D%0AMTI1IEMgMC4zNDM3NSAtMS45NDE0MDYgMC4zMTI1IC0yLjM1OTM3NSAwLjMxMjUgLTIuODI4MTI1%0D%0AIEMgMC4zMTI1IC0zLjI2NTYyNSAwLjM0NzY1NiAtMy42NjAxNTYgMC40MjE4NzUgLTQuMDE1NjI1%0D%0AIEMgMC40OTIxODggLTQuMzc4OTA2IDAuNjA5Mzc1IC00LjY5MTQwNiAwLjc2NTYyNSAtNC45NTMx%0D%0AMjUgQyAwLjkyMTg3NSAtNS4yMTA5MzggMS4xMjg5MDYgLTUuNDEwMTU2IDEuMzkwNjI1IC01LjU0%0D%0ANjg3NSBDIDEuNjQ4NDM4IC01LjY5MTQwNiAxLjk2ODc1IC01Ljc2NTYyNSAyLjM0Mzc1IC01Ljc2%0D%0ANTYyNSBDIDIuNjk1MzEyIC01Ljc2NTYyNSAyLjk5MjE4OCAtNS43MDMxMjUgMy4yMzQzNzUgLTUu%0D%0ANTc4MTI1IEMgMy40ODQzNzUgLTUuNDUzMTI1IDMuNjc5Njg4IC01LjI2NTYyNSAzLjgyODEyNSAt%0D%0ANS4wMTU2MjUgQyAzLjk4NDM3NSAtNC43NjU2MjUgNC4wOTM3NSAtNC40NTcwMzEgNC4xNTYyNSAt%0D%0ANC4wOTM3NSBDIDQuMjE4NzUgLTMuNzM4MjgxIDQuMjUgLTMuMzI4MTI1IDQuMjUgLTIuODU5Mzc1%0D%0AIFogTSAzLjQ2ODc1IC0yLjc5Njg3NSBDIDMuNDY4NzUgLTMuMDg1OTM4IDMuNDU3MDMxIC0zLjM0%0D%0AMzc1IDMuNDM3NSAtMy41NjI1IEMgMy40MjU3ODEgLTMuNzg5MDYyIDMuMzk4NDM4IC0zLjk4ODI4%0D%0AMSAzLjM1OTM3NSAtNC4xNTYyNSBDIDMuMzE2NDA2IC00LjMzMjAzMSAzLjI2NTYyNSAtNC40ODQz%0D%0ANzUgMy4yMDMxMjUgLTQuNjA5Mzc1IEMgMy4xNDA2MjUgLTQuNzM0Mzc1IDMuMDYyNSAtNC44MzU5%0D%0AMzggMi45Njg3NSAtNC45MjE4NzUgQyAyLjg4MjgxMiAtNS4wMDM5MDYgMi43ODUxNTYgLTUuMDYy%0D%0ANSAyLjY3MTg3NSAtNS4wOTM3NSBDIDIuNTU0Njg4IC01LjEzMjgxMiAyLjQyOTY4OCAtNS4xNTYy%0D%0ANSAyLjI5Njg3NSAtNS4xNTYyNSBDIDIuMDQ2ODc1IC01LjE1NjI1IDEuODQzNzUgLTUuMDkzNzUg%0D%0AMS42ODc1IC00Ljk2ODc1IEMgMS41MzEyNSAtNC44NTE1NjIgMS40MDYyNSAtNC42OTE0MDYgMS4z%0D%0AMTI1IC00LjQ4NDM3NSBDIDEuMjI2NTYyIC00LjI4NTE1NiAxLjE3MTg3NSAtNC4wNDY4NzUgMS4x%0D%0ANDA2MjUgLTMuNzY1NjI1IEMgMS4xMDkzNzUgLTMuNDkyMTg4IDEuMDkzNzUgLTMuMjAzMTI1IDEu%0D%0AMDkzNzUgLTIuODkwNjI1IEMgMS4wOTM3NSAtMi40NjA5MzggMS4xMTMyODEgLTIuMDk3NjU2IDEu%0D%0AMTU2MjUgLTEuNzk2ODc1IEMgMS4xOTUzMTIgLTEuNTAzOTA2IDEuMjY1NjI1IC0xLjI2NTYyNSAx%0D%0ALjM1OTM3NSAtMS4wNzgxMjUgQyAxLjQ1MzEyNSAtMC44OTA2MjUgMS41NzAzMTIgLTAuNzUgMS43%0D%0AMTg3NSAtMC42NTYyNSBDIDEuODc1IC0wLjU3MDMxMiAyLjA1NDY4OCAtMC41MzEyNSAyLjI2NTYy%0D%0ANSAtMC41MzEyNSBDIDIuNDI5Njg4IC0wLjUzMTI1IDIuNTc4MTI1IC0wLjU1NDY4OCAyLjcwMzEy%0D%0ANSAtMC42MDkzNzUgQyAyLjgyODEyNSAtMC42NjAxNTYgMi45Mjk2ODggLTAuNzM0Mzc1IDMuMDE1%0D%0ANjI1IC0wLjgyODEyNSBDIDMuMTA5Mzc1IC0wLjkyOTY4OCAzLjE3OTY4OCAtMS4wNTA3ODEgMy4y%0D%0AMzQzNzUgLTEuMTg3NSBDIDMuMjk2ODc1IC0xLjMzMjAzMSAzLjM0Mzc1IC0xLjQ4ODI4MSAzLjM3%0D%0ANSAtMS42NTYyNSBDIDMuNDE0MDYyIC0xLjgyMDMxMiAzLjQ0MTQwNiAtMiAzLjQ1MzEyNSAtMi4x%0D%0AODc1IEMgMy40NjA5MzggLTIuMzgyODEyIDMuNDY4NzUgLTIuNTg1OTM4IDMuNDY4NzUgLTIuNzk2%0D%0AODc1IFogTSAzLjQ2ODc1IC0yLjc5Njg3NSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9%0D%0AInZpc2libGUiIGlkPSJnbHlwaDAtMiI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0g%0D%0ANC4wMzEyNSAtMS44MjgxMjUgQyA0LjAzMTI1IC0xLjUyMzQzOCAzLjk3NjU2MiAtMS4yNTM5MDYg%0D%0AMy44NzUgLTEuMDE1NjI1IEMgMy43ODEyNSAtMC43NzM0MzggMy42NDA2MjUgLTAuNTcwMzEyIDMu%0D%0ANDUzMTI1IC0wLjQwNjI1IEMgMy4yNjU2MjUgLTAuMjUgMy4wMzkwNjIgLTAuMTI4OTA2IDIuNzgx%0D%0AMjUgLTAuMDQ2ODc1IEMgMi41MTk1MzEgMC4wMzUxNTYyIDIuMjI2NTYyIDAuMDc4MTI1IDEuOTA2%0D%0AMjUgMC4wNzgxMjUgQyAxLjcyNjU2MiAwLjA3ODEyNSAxLjU2MjUgMC4wNjY0MDYyIDEuNDA2MjUg%0D%0AMC4wNDY4NzUgQyAxLjI1IDAuMDIzNDM3NSAxLjEwOTM3NSAwIDAuOTg0Mzc1IC0wLjAzMTI1IEMg%0D%0AMC44NTkzNzUgLTAuMDcwMzEyNSAwLjc1MzkwNiAtMC4xMDkzNzUgMC42NzE4NzUgLTAuMTQwNjI1%0D%0AIEMgMC41OTc2NTYgLTAuMTc5Njg4IDAuNTUwNzgxIC0wLjIxMDkzOCAwLjUzMTI1IC0wLjIzNDM3%0D%0ANSBDIDAuNTA3ODEyIC0wLjI1MzkwNiAwLjQ5MjE4OCAtMC4yNzM0MzggMC40ODQzNzUgLTAuMjk2%0D%0AODc1IEMgMC40NzI2NTYgLTAuMzE2NDA2IDAuNDYwOTM4IC0wLjMzNTkzOCAwLjQ1MzEyNSAtMC4z%0D%0ANTkzNzUgQyAwLjQ1MzEyNSAtMC4zNzg5MDYgMC40NDUzMTIgLTAuNDEwMTU2IDAuNDM3NSAtMC40%0D%0ANTMxMjUgQyAwLjQzNzUgLTAuNDkyMTg4IDAuNDM3NSAtMC41MzkwNjIgMC40Mzc1IC0wLjU5Mzc1%0D%0AIEMgMC40Mzc1IC0wLjYzMjgxMiAwLjQzNzUgLTAuNjc1NzgxIDAuNDM3NSAtMC43MTg3NSBDIDAu%0D%0ANDQ1MzEyIC0wLjc1NzgxMiAwLjQ1NzAzMSAtMC43ODkwNjIgMC40Njg3NSAtMC44MTI1IEMgMC40%0D%0AODgyODEgLTAuODQzNzUgMC41MDM5MDYgLTAuODYzMjgxIDAuNTE1NjI1IC0wLjg3NSBDIDAuNTM1%0D%0AMTU2IC0wLjg4MjgxMiAwLjU1NDY4OCAtMC44OTA2MjUgMC41NzgxMjUgLTAuODkwNjI1IEMgMC41%0D%0AOTc2NTYgLTAuODkwNjI1IDAuNjQwNjI1IC0wLjg2NzE4OCAwLjcwMzEyNSAtMC44MjgxMjUgQyAw%0D%0ALjc2NTYyNSAtMC43OTY4NzUgMC44NDc2NTYgLTAuNzU3ODEyIDAuOTUzMTI1IC0wLjcxODc1IEMg%0D%0AMS4wNTQ2ODggLTAuNjc1NzgxIDEuMTg3NSAtMC42MzI4MTIgMS4zNDM3NSAtMC41OTM3NSBDIDEu%0D%0ANSAtMC41NjI1IDEuNjc5Njg4IC0wLjU0Njg3NSAxLjg5MDYyNSAtMC41NDY4NzUgQyAyLjA4NTkz%0D%0AOCAtMC41NDY4NzUgMi4yNjU2MjUgLTAuNTY2NDA2IDIuNDIxODc1IC0wLjYwOTM3NSBDIDIuNTg1%0D%0AOTM4IC0wLjY2MDE1NiAyLjcyNjU2MiAtMC43MzQzNzUgMi44NDM3NSAtMC44MjgxMjUgQyAyLjk2%0D%0AODc1IC0wLjkyOTY4OCAzLjA2MjUgLTEuMDU0Njg4IDMuMTI1IC0xLjIwMzEyNSBDIDMuMTk1MzEy%0D%0AIC0xLjM1OTM3NSAzLjIzNDM3NSAtMS41NDY4NzUgMy4yMzQzNzUgLTEuNzY1NjI1IEMgMy4yMzQz%0D%0ANzUgLTEuOTQxNDA2IDMuMjAzMTI1IC0yLjA5NzY1NiAzLjE0MDYyNSAtMi4yMzQzNzUgQyAzLjA4%0D%0ANTkzOCAtMi4zNzg5MDYgMyAtMi40OTIxODggMi44NzUgLTIuNTc4MTI1IEMgMi43NTc4MTIgLTIu%0D%0ANjcxODc1IDIuNjA5Mzc1IC0yLjczODI4MSAyLjQyMTg3NSAtMi43ODEyNSBDIDIuMjQyMTg4IC0y%0D%0ALjgzMjAzMSAyLjAzMTI1IC0yLjg1OTM3NSAxLjc4MTI1IC0yLjg1OTM3NSBDIDEuNjAxNTYyIC0y%0D%0ALjg1OTM3NSAxLjQ0NTMxMiAtMi44NDc2NTYgMS4zMTI1IC0yLjgyODEyNSBDIDEuMTg3NSAtMi44%0D%0AMTY0MDYgMS4wNzAzMTIgLTIuODEyNSAwLjk2ODc1IC0yLjgxMjUgQyAwLjg4MjgxMiAtMi44MTI1%0D%0AIDAuODIwMzEyIC0yLjgyODEyNSAwLjc4MTI1IC0yLjg1OTM3NSBDIDAuNzUgLTIuODk4NDM4IDAu%0D%0ANzM0Mzc1IC0yLjk3MjY1NiAwLjczNDM3NSAtMy4wNzgxMjUgTCAwLjczNDM3NSAtNS40MDYyNSBD%0D%0AIDAuNzM0Mzc1IC01LjUgMC43NTM5MDYgLTUuNTY2NDA2IDAuNzk2ODc1IC01LjYwOTM3NSBDIDAu%0D%0AODQ3NjU2IC01LjY2MDE1NiAwLjkxNDA2MiAtNS42ODc1IDEgLTUuNjg3NSBMIDMuNTE1NjI1IC01%0D%0ALjY4NzUgQyAzLjUzNTE1NiAtNS42ODc1IDMuNTU0Njg4IC01LjY3NTc4MSAzLjU3ODEyNSAtNS42%0D%0ANTYyNSBDIDMuNTk3NjU2IC01LjY0NDUzMSAzLjYxMzI4MSAtNS42MjUgMy42MjUgLTUuNTkzNzUg%0D%0AQyAzLjY0NDUzMSAtNS41NzAzMTIgMy42NjAxNTYgLTUuNTM5MDYyIDMuNjcxODc1IC01LjUgQyAz%0D%0ALjY3OTY4OCAtNS40NTcwMzEgMy42ODc1IC01LjQxMDE1NiAzLjY4NzUgLTUuMzU5Mzc1IEMgMy42%0D%0AODc1IC01LjI1MzkwNiAzLjY3MTg3NSAtNS4xNzE4NzUgMy42NDA2MjUgLTUuMTA5Mzc1IEMgMy42%0D%0AMDkzNzUgLTUuMDU0Njg4IDMuNTY2NDA2IC01LjAzMTI1IDMuNTE1NjI1IC01LjAzMTI1IEwgMS4z%0D%0AOTA2MjUgLTUuMDMxMjUgTCAxLjM5MDYyNSAtMy40Mzc1IEMgMS40OTIxODggLTMuNDQ1MzEyIDEu%0D%0ANTk3NjU2IC0zLjQ1MzEyNSAxLjcwMzEyNSAtMy40NTMxMjUgQyAxLjgxNjQwNiAtMy40NjA5Mzgg%0D%0AMS45NDE0MDYgLTMuNDY4NzUgMi4wNzgxMjUgLTMuNDY4NzUgQyAyLjM5ODQzOCAtMy40Njg3NSAy%0D%0ALjY3OTY4OCAtMy40MjU3ODEgMi45MjE4NzUgLTMuMzQzNzUgQyAzLjE3MTg3NSAtMy4yNjk1MzEg%0D%0AMy4zNzUgLTMuMTYwMTU2IDMuNTMxMjUgLTMuMDE1NjI1IEMgMy42OTUzMTIgLTIuODY3MTg4IDMu%0D%0AODIwMzEyIC0yLjY5NTMxMiAzLjkwNjI1IC0yLjUgQyAzLjk4ODI4MSAtMi4zMDA3ODEgNC4wMzEy%0D%0ANSAtMi4wNzgxMjUgNC4wMzEyNSAtMS44MjgxMjUgWiBNIDQuMDMxMjUgLTEuODI4MTI1ICIvPgo8%0D%0AL3N5bWJvbD4KPHN5bWJvbCBvdmVyZmxvdz0idmlzaWJsZSIgaWQ9ImdseXBoMC0zIj4KPHBhdGgg%0D%0Ac3R5bGU9InN0cm9rZTpub25lOyIgZD0iTSA0LjA3ODEyNSAtMC4yOTY4NzUgQyA0LjA3ODEyNSAt%0D%0AMC4yNDIxODggNC4wNzAzMTIgLTAuMTk1MzEyIDQuMDYyNSAtMC4xNTYyNSBDIDQuMDUwNzgxIC0w%0D%0ALjExMzI4MSA0LjAzNTE1NiAtMC4wODIwMzEyIDQuMDE1NjI1IC0wLjA2MjUgQyA0LjAwMzkwNiAt%0D%0AMC4wMzkwNjI1IDMuOTg4MjgxIC0wLjAyMzQzNzUgMy45Njg3NSAtMC4wMTU2MjUgQyAzLjk1NzAz%0D%0AMSAtMC4wMDM5MDYyNSAzLjk0MTQwNiAwIDMuOTIxODc1IDAgTCAwLjkzNzUgMCBDIDAuOTI1Nzgx%0D%0AIDAgMC45MTAxNTYgLTAuMDAzOTA2MjUgMC44OTA2MjUgLTAuMDE1NjI1IEMgMC44NjcxODggLTAu%0D%0AMDIzNDM3NSAwLjg0NzY1NiAtMC4wMzkwNjI1IDAuODI4MTI1IC0wLjA2MjUgQyAwLjgxNjQwNiAt%0D%0AMC4wODIwMzEyIDAuODA0Njg4IC0wLjExMzI4MSAwLjc5Njg3NSAtMC4xNTYyNSBDIDAuNzg1MTU2%0D%0AIC0wLjE5NTMxMiAwLjc4MTI1IC0wLjI0MjE4OCAwLjc4MTI1IC0wLjI5Njg3NSBDIDAuNzgxMjUg%0D%0ALTAuMzQ3NjU2IDAuNzg1MTU2IC0wLjM5NDUzMSAwLjc5Njg3NSAtMC40Mzc1IEMgMC44MDQ2ODgg%0D%0ALTAuNDc2NTYyIDAuODE2NDA2IC0wLjUwNzgxMiAwLjgyODEyNSAtMC41MzEyNSBDIDAuODQ3NjU2%0D%0AIC0wLjU1MDc4MSAwLjg2MzI4MSAtMC41NjY0MDYgMC44NzUgLTAuNTc4MTI1IEMgMC44OTQ1MzEg%0D%0ALTAuNTk3NjU2IDAuOTE0MDYyIC0wLjYwOTM3NSAwLjkzNzUgLTAuNjA5Mzc1IEwgMi4xMjUgLTAu%0D%0ANjA5Mzc1IEwgMi4xMjUgLTQuOTM3NSBMIDEuMDMxMjUgLTQuMjgxMjUgQyAwLjk3NjU2MiAtNC4y%0D%0ANSAwLjkyOTY4OCAtNC4yMjY1NjIgMC44OTA2MjUgLTQuMjE4NzUgQyAwLjg1OTM3NSAtNC4yMTg3%0D%0ANSAwLjgzMjAzMSAtNC4yMjI2NTYgMC44MTI1IC00LjIzNDM3NSBDIDAuNzg5MDYyIC00LjI1Mzkw%0D%0ANiAwLjc3MzQzOCAtNC4yODUxNTYgMC43NjU2MjUgLTQuMzI4MTI1IEMgMC43NjU2MjUgLTQuMzc4%0D%0AOTA2IDAuNzY1NjI1IC00LjQzNzUgMC43NjU2MjUgLTQuNSBDIDAuNzY1NjI1IC00LjU1MDc4MSAw%0D%0ALjc2NTYyNSAtNC41OTM3NSAwLjc2NTYyNSAtNC42MjUgQyAwLjc3MzQzOCAtNC42NjQwNjIgMC43%0D%0AODEyNSAtNC42OTUzMTIgMC43ODEyNSAtNC43MTg3NSBDIDAuNzg5MDYyIC00LjczODI4MSAwLjgw%0D%0ANDY4OCAtNC43NTc4MTIgMC44MjgxMjUgLTQuNzgxMjUgQyAwLjg0NzY1NiAtNC44MDA3ODEgMC44%0D%0ANjcxODggLTQuODE2NDA2IDAuODkwNjI1IC00LjgyODEyNSBMIDIuMjAzMTI1IC01LjY3MTg3NSBD%0D%0AIDIuMjEwOTM4IC01LjY3OTY4OCAyLjIyMjY1NiAtNS42ODc1IDIuMjM0Mzc1IC01LjY4NzUgQyAy%0D%0ALjI1MzkwNiAtNS42OTUzMTIgMi4yNzM0MzggLTUuNzAzMTI1IDIuMjk2ODc1IC01LjcwMzEyNSBD%0D%0AIDIuMzI4MTI1IC01LjcxMDkzOCAyLjM1OTM3NSAtNS43MTg3NSAyLjM5MDYyNSAtNS43MTg3NSBD%0D%0AIDIuNDI5Njg4IC01LjcxODc1IDIuNDc2NTYyIC01LjcxODc1IDIuNTMxMjUgLTUuNzE4NzUgQyAy%0D%0ALjYwMTU2MiAtNS43MTg3NSAyLjY2MDE1NiAtNS43MTA5MzggMi43MDMxMjUgLTUuNzAzMTI1IEMg%0D%0AMi43NTM5MDYgLTUuNzAzMTI1IDIuNzg5MDYyIC01LjY5NTMxMiAyLjgxMjUgLTUuNjg3NSBDIDIu%0D%0AODQzNzUgLTUuNjc1NzgxIDIuODYzMjgxIC01LjY2MDE1NiAyLjg3NSAtNS42NDA2MjUgQyAyLjg4%0D%0AMjgxMiAtNS42Mjg5MDYgMi44OTA2MjUgLTUuNjE3MTg4IDIuODkwNjI1IC01LjYwOTM3NSBMIDIu%0D%0AODkwNjI1IC0wLjYwOTM3NSBMIDMuOTIxODc1IC0wLjYwOTM3NSBDIDMuOTQxNDA2IC0wLjYwOTM3%0D%0ANSAzLjk2MDkzOCAtMC41OTc2NTYgMy45ODQzNzUgLTAuNTc4MTI1IEMgNC4wMDM5MDYgLTAuNTY2%0D%0ANDA2IDQuMDE5NTMxIC0wLjU1MDc4MSA0LjAzMTI1IC0wLjUzMTI1IEMgNC4wMzkwNjIgLTAuNTA3%0D%0AODEyIDQuMDUwNzgxIC0wLjQ3NjU2MiA0LjA2MjUgLTAuNDM3NSBDIDQuMDcwMzEyIC0wLjM5NDUz%0D%0AMSA0LjA3ODEyNSAtMC4zNDc2NTYgNC4wNzgxMjUgLTAuMjk2ODc1IFogTSA0LjA3ODEyNSAtMC4y%0D%0AOTY4NzUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2aXNpYmxlIiBpZD0iZ2x5cGgw%0D%0ALTQiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDQuMDYyNSAtMC4zMjgxMjUgQyA0%0D%0ALjA2MjUgLTAuMjczNDM4IDQuMDU0Njg4IC0wLjIyNjU2MiA0LjA0Njg3NSAtMC4xODc1IEMgNC4w%0D%0AMzUxNTYgLTAuMTQ0NTMxIDQuMDIzNDM4IC0wLjEwOTM3NSA0LjAxNTYyNSAtMC4wNzgxMjUgQyA0%0D%0ALjAwMzkwNiAtMC4wNTQ2ODc1IDMuOTg4MjgxIC0wLjAzNTE1NjIgMy45Njg3NSAtMC4wMTU2MjUg%0D%0AQyAzLjk0NTMxMiAtMC4wMDM5MDYyNSAzLjkyMTg3NSAwIDMuODkwNjI1IDAgTCAwLjc1IDAgQyAw%0D%0ALjcwNzAzMSAwIDAuNjcxODc1IC0wLjAwMzkwNjI1IDAuNjQwNjI1IC0wLjAxNTYyNSBDIDAuNjA5%0D%0AMzc1IC0wLjAyMzQzNzUgMC41ODIwMzEgLTAuMDM5MDYyNSAwLjU2MjUgLTAuMDYyNSBDIDAuNTM5%0D%0AMDYyIC0wLjA4MjAzMTIgMC41MjM0MzggLTAuMTEzMjgxIDAuNTE1NjI1IC0wLjE1NjI1IEMgMC41%0D%0AMDM5MDYgLTAuMjA3MDMxIDAuNSAtMC4yNjU2MjUgMC41IC0wLjMyODEyNSBDIDAuNSAtMC4zOTA2%0D%0AMjUgMC41IC0wLjQ0MTQwNiAwLjUgLTAuNDg0Mzc1IEMgMC41MDc4MTIgLTAuNTIzNDM4IDAuNTE5%0D%0ANTMxIC0wLjU2MjUgMC41MzEyNSAtMC41OTM3NSBDIDAuNTUwNzgxIC0wLjYzMjgxMiAwLjU3MDMx%0D%0AMiAtMC42NzE4NzUgMC41OTM3NSAtMC43MDMxMjUgQyAwLjYxMzI4MSAtMC43MzQzNzUgMC42NDA2%0D%0AMjUgLTAuNzY5NTMxIDAuNjcxODc1IC0wLjgxMjUgTCAxLjc4MTI1IC0xLjk4NDM3NSBDIDIuMDMx%0D%0AMjUgLTIuMjQyMTg4IDIuMjM0Mzc1IC0yLjQ4NDM3NSAyLjM5MDYyNSAtMi43MDMxMjUgQyAyLjU0%0D%0ANjg3NSAtMi45MjE4NzUgMi42NjQwNjIgLTMuMTEzMjgxIDIuNzUgLTMuMjgxMjUgQyAyLjgzMjAz%0D%0AMSAtMy40NTcwMzEgMi44ODI4MTIgLTMuNjE3MTg4IDIuOTA2MjUgLTMuNzY1NjI1IEMgMi45Mzc1%0D%0AIC0zLjkxMDE1NiAyLjk1MzEyNSAtNC4wNDY4NzUgMi45NTMxMjUgLTQuMTcxODc1IEMgMi45NTMx%0D%0AMjUgLTQuMjk2ODc1IDIuOTI5Njg4IC00LjQxNDA2MiAyLjg5MDYyNSAtNC41MzEyNSBDIDIuODQ3%0D%0ANjU2IC00LjY0NDUzMSAyLjc4NTE1NiAtNC43NDIxODggMi43MDMxMjUgLTQuODI4MTI1IEMgMi42%0D%0AMjg5MDYgLTQuOTEwMTU2IDIuNTM1MTU2IC00Ljk3NjU2MiAyLjQyMTg3NSAtNS4wMzEyNSBDIDIu%0D%0AMzA0Njg4IC01LjA4MjAzMSAyLjE3MTg3NSAtNS4xMDkzNzUgMi4wMTU2MjUgLTUuMTA5Mzc1IEMg%0D%0AMS44MzU5MzggLTUuMTA5Mzc1IDEuNjc1NzgxIC01LjA4MjAzMSAxLjUzMTI1IC01LjAzMTI1IEMg%0D%0AMS4zOTQ1MzEgLTQuOTc2NTYyIDEuMjY5NTMxIC00LjkyMTg3NSAxLjE1NjI1IC00Ljg1OTM3NSBD%0D%0AIDEuMDUwNzgxIC00LjgwNDY4OCAwLjk2MDkzOCAtNC43NTM5MDYgMC44OTA2MjUgLTQuNzAzMTI1%0D%0AIEMgMC44MTY0MDYgLTQuNjQ4NDM4IDAuNzY1NjI1IC00LjYyNSAwLjczNDM3NSAtNC42MjUgQyAw%0D%0ALjcxMDkzOCAtNC42MjUgMC42OTE0MDYgLTQuNjI4OTA2IDAuNjcxODc1IC00LjY0MDYyNSBDIDAu%0D%0ANjYwMTU2IC00LjY0ODQzOCAwLjY0ODQzOCAtNC42NjQwNjIgMC42NDA2MjUgLTQuNjg3NSBDIDAu%0D%0ANjI4OTA2IC00LjcxODc1IDAuNjE3MTg4IC00Ljc1MzkwNiAwLjYwOTM3NSAtNC43OTY4NzUgQyAw%0D%0ALjYwOTM3NSAtNC44NDc2NTYgMC42MDkzNzUgLTQuOTA2MjUgMC42MDkzNzUgLTQuOTY4NzUgQyAw%0D%0ALjYwOTM3NSAtNS4wMDc4MTIgMC42MDkzNzUgLTUuMDQ2ODc1IDAuNjA5Mzc1IC01LjA3ODEyNSBD%0D%0AIDAuNjA5Mzc1IC01LjEwOTM3NSAwLjYxMzI4MSAtNS4xMzI4MTIgMC42MjUgLTUuMTU2MjUgQyAw%0D%0ALjYzMjgxMiAtNS4xODc1IDAuNjQ0NTMxIC01LjIxMDkzOCAwLjY1NjI1IC01LjIzNDM3NSBDIDAu%0D%0ANjY0MDYyIC01LjI1MzkwNiAwLjY4NzUgLTUuMjgxMjUgMC43MTg3NSAtNS4zMTI1IEMgMC43NTc4%0D%0AMTIgLTUuMzQzNzUgMC44MjAzMTIgLTUuMzgyODEyIDAuOTA2MjUgLTUuNDM3NSBDIDAuOTg4Mjgx%0D%0AIC01LjQ4ODI4MSAxLjA5Mzc1IC01LjUzOTA2MiAxLjIxODc1IC01LjU5Mzc1IEMgMS4zNTE1NjIg%0D%0ALTUuNjQ0NTMxIDEuNSAtNS42ODc1IDEuNjU2MjUgLTUuNzE4NzUgQyAxLjgyMDMxMiAtNS43NSAx%0D%0ALjk4ODI4MSAtNS43NjU2MjUgMi4xNTYyNSAtNS43NjU2MjUgQyAyLjQzNzUgLTUuNzY1NjI1IDIu%0D%0ANjc1NzgxIC01LjcyMjY1NiAyLjg3NSAtNS42NDA2MjUgQyAzLjA4MjAzMSAtNS41NjY0MDYgMy4y%0D%0ANTM5MDYgLTUuNDYwOTM4IDMuMzkwNjI1IC01LjMyODEyNSBDIDMuNTIzNDM4IC01LjIwMzEyNSAz%0D%0ALjYyNSAtNS4wNDY4NzUgMy42ODc1IC00Ljg1OTM3NSBDIDMuNzU3ODEyIC00LjY3OTY4OCAzLjc5%0D%0ANjg3NSAtNC40OTIxODggMy43OTY4NzUgLTQuMjk2ODc1IEMgMy43OTY4NzUgLTQuMTA5Mzc1IDMu%0D%0ANzgxMjUgLTMuOTI1NzgxIDMuNzUgLTMuNzUgQyAzLjcxODc1IC0zLjU3MDMxMiAzLjY0ODQzOCAt%0D%0AMy4zNzg5MDYgMy41NDY4NzUgLTMuMTcxODc1IEMgMy40NDE0MDYgLTIuOTYwOTM4IDMuMjg5MDYy%0D%0AIC0yLjcyNjU2MiAzLjA5Mzc1IC0yLjQ2ODc1IEMgMi44OTQ1MzEgLTIuMjA3MDMxIDIuNjMyODEy%0D%0AIC0xLjkxMDE1NiAyLjMxMjUgLTEuNTc4MTI1IEwgMS40MjE4NzUgLTAuNjQwNjI1IEwgMy44OTA2%0D%0AMjUgLTAuNjQwNjI1IEMgMy45MTAxNTYgLTAuNjQwNjI1IDMuOTI5Njg4IC0wLjYzMjgxMiAzLjk1%0D%0AMzEyNSAtMC42MjUgQyAzLjk3MjY1NiAtMC42MTMyODEgMy45ODgyODEgLTAuNTkzNzUgNCAtMC41%0D%0ANjI1IEMgNC4wMTk1MzEgLTAuNTMxMjUgNC4wMzUxNTYgLTAuNDkyMTg4IDQuMDQ2ODc1IC0wLjQ1%0D%0AMzEyNSBDIDQuMDU0Njg4IC0wLjQyMTg3NSA0LjA2MjUgLTAuMzc4OTA2IDQuMDYyNSAtMC4zMjgx%0D%0AMjUgWiBNIDQuMDYyNSAtMC4zMjgxMjUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2%0D%0AaXNpYmxlIiBpZD0iZ2x5cGgwLTUiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDQu%0D%0AMDE1NjI1IC0xLjYyNSBDIDQuMDE1NjI1IC0xLjM2MzI4MSAzLjk2ODc1IC0xLjEyODkwNiAzLjg3%0D%0ANSAtMC45MjE4NzUgQyAzLjc4OTA2MiAtMC43MTA5MzggMy42NjAxNTYgLTAuNTMxMjUgMy40ODQz%0D%0ANzUgLTAuMzc1IEMgMy4zMTY0MDYgLTAuMjI2NTYyIDMuMTA5Mzc1IC0wLjExMzI4MSAyLjg1OTM3%0D%0ANSAtMC4wMzEyNSBDIDIuNjA5Mzc1IDAuMDM5MDYyNSAyLjMyODEyNSAwLjA3ODEyNSAyLjAxNTYy%0D%0ANSAwLjA3ODEyNSBDIDEuODI4MTI1IDAuMDc4MTI1IDEuNjQ4NDM4IDAuMDYyNSAxLjQ4NDM3NSAw%0D%0ALjAzMTI1IEMgMS4zMTY0MDYgMCAxLjE2NDA2MiAtMC4wMzUxNTYyIDEuMDMxMjUgLTAuMDc4MTI1%0D%0AIEMgMC45MDYyNSAtMC4xMTcxODggMC44MDA3ODEgLTAuMTYwMTU2IDAuNzE4NzUgLTAuMjAzMTI1%0D%0AIEMgMC42MzI4MTIgLTAuMjQyMTg4IDAuNTgyMDMxIC0wLjI3MzQzOCAwLjU2MjUgLTAuMjk2ODc1%0D%0AIEMgMC41MzkwNjIgLTAuMzE2NDA2IDAuNTIzNDM4IC0wLjMzNTkzOCAwLjUxNTYyNSAtMC4zNTkz%0D%0ANzUgQyAwLjUwMzkwNiAtMC4zNzg5MDYgMC40OTIxODggLTAuNDA2MjUgMC40ODQzNzUgLTAuNDM3%0D%0ANSBDIDAuNDcyNjU2IC0wLjQ2ODc1IDAuNDYwOTM4IC0wLjUwMzkwNiAwLjQ1MzEyNSAtMC41NDY4%0D%0ANzUgQyAwLjQ1MzEyNSAtMC41ODU5MzggMC40NTMxMjUgLTAuNjQwNjI1IDAuNDUzMTI1IC0wLjcw%0D%0AMzEyNSBDIDAuNDUzMTI1IC0wLjgwNDY4OCAwLjQ2MDkzOCAtMC44Nzg5MDYgMC40ODQzNzUgLTAu%0D%0AOTIxODc1IEMgMC41MDM5MDYgLTAuOTYwOTM4IDAuNTM1MTU2IC0wLjk4NDM3NSAwLjU3ODEyNSAt%0D%0AMC45ODQzNzUgQyAwLjU5NzY1NiAtMC45ODQzNzUgMC42NDQ1MzEgLTAuOTU3MDMxIDAuNzE4NzUg%0D%0ALTAuOTA2MjUgQyAwLjgwMDc4MSAtMC44NjMyODEgMC44OTg0MzggLTAuODE2NDA2IDEuMDE1NjI1%0D%0AIC0wLjc2NTYyNSBDIDEuMTQwNjI1IC0wLjcxMDkzOCAxLjI4NTE1NiAtMC42NjAxNTYgMS40NTMx%0D%0AMjUgLTAuNjA5Mzc1IEMgMS42MTcxODggLTAuNTY2NDA2IDEuODAwNzgxIC0wLjU0Njg3NSAyIC0w%0D%0ALjU0Njg3NSBDIDIuMTg3NSAtMC41NDY4NzUgMi4zNTE1NjIgLTAuNTY2NDA2IDIuNSAtMC42MDkz%0D%0ANzUgQyAyLjY0NDUzMSAtMC42NjAxNTYgMi43Njk1MzEgLTAuNzM0Mzc1IDIuODc1IC0wLjgyODEy%0D%0ANSBDIDIuOTc2NTYyIC0wLjkyMTg3NSAzLjA1MDc4MSAtMS4wMzEyNSAzLjA5Mzc1IC0xLjE1NjI1%0D%0AIEMgMy4xNDQ1MzEgLTEuMjgxMjUgMy4xNzE4NzUgLTEuNDE0MDYyIDMuMTcxODc1IC0xLjU2MjUg%0D%0AQyAzLjE3MTg3NSAtMS43MTg3NSAzLjE0MDYyNSAtMS44NTkzNzUgMy4wNzgxMjUgLTEuOTg0Mzc1%0D%0AIEMgMy4wMTU2MjUgLTIuMTE3MTg4IDIuOTIxODc1IC0yLjIzNDM3NSAyLjc5Njg3NSAtMi4zMjgx%0D%0AMjUgQyAyLjY3OTY4OCAtMi40MjE4NzUgMi41MzUxNTYgLTIuNDg4MjgxIDIuMzU5Mzc1IC0yLjUz%0D%0AMTI1IEMgMi4xNzk2ODggLTIuNTgyMDMxIDEuOTc2NTYyIC0yLjYwOTM3NSAxLjc1IC0yLjYwOTM3%0D%0ANSBMIDEuMjAzMTI1IC0yLjYwOTM3NSBDIDEuMTc5Njg4IC0yLjYwOTM3NSAxLjE2MDE1NiAtMi42%0D%0AMTMyODEgMS4xNDA2MjUgLTIuNjI1IEMgMS4xMTcxODggLTIuNjMyODEyIDEuMDk3NjU2IC0yLjY0%0D%0AODQzOCAxLjA3ODEyNSAtMi42NzE4NzUgQyAxLjA2NjQwNiAtMi43MDMxMjUgMS4wNTQ2ODggLTIu%0D%0ANzM0Mzc1IDEuMDQ2ODc1IC0yLjc2NTYyNSBDIDEuMDM1MTU2IC0yLjgwNDY4OCAxLjAzMTI1IC0y%0D%0ALjg1OTM3NSAxLjAzMTI1IC0yLjkyMTg3NSBDIDEuMDMxMjUgLTIuOTcyNjU2IDEuMDM1MTU2IC0z%0D%0ALjAxNTYyNSAxLjA0Njg3NSAtMy4wNDY4NzUgQyAxLjA1NDY4OCAtMy4wODU5MzggMS4wNjY0MDYg%0D%0ALTMuMTE3MTg4IDEuMDc4MTI1IC0zLjE0MDYyNSBDIDEuMDk3NjU2IC0zLjE3MTg3NSAxLjExNzE4%0D%0AOCAtMy4xOTE0MDYgMS4xNDA2MjUgLTMuMjAzMTI1IEMgMS4xNjAxNTYgLTMuMjEwOTM4IDEuMTc5%0D%0ANjg4IC0zLjIxODc1IDEuMjAzMTI1IC0zLjIxODc1IEwgMS43MDMxMjUgLTMuMjE4NzUgQyAxLjg5%0D%0AMDYyNSAtMy4yMTg3NSAyLjA2MjUgLTMuMjM4MjgxIDIuMjE4NzUgLTMuMjgxMjUgQyAyLjM3NSAt%0D%0AMy4zMzIwMzEgMi41MDM5MDYgLTMuNDA2MjUgMi42MDkzNzUgLTMuNSBDIDIuNzEwOTM4IC0zLjU5%0D%0AMzc1IDIuNzg5MDYyIC0zLjcwMzEyNSAyLjg0Mzc1IC0zLjgyODEyNSBDIDIuOTA2MjUgLTMuOTYw%0D%0AOTM4IDIuOTM3NSAtNC4xMTMyODEgMi45Mzc1IC00LjI4MTI1IEMgMi45Mzc1IC00LjM5NDUzMSAy%0D%0ALjkxNDA2MiAtNC41MDM5MDYgMi44NzUgLTQuNjA5Mzc1IEMgMi44NDM3NSAtNC43MTA5MzggMi43%0D%0AODkwNjIgLTQuODA0Njg4IDIuNzE4NzUgLTQuODkwNjI1IEMgMi42NDQ1MzEgLTQuOTcyNjU2IDIu%0D%0ANTQ2ODc1IC01LjAzNTE1NiAyLjQyMTg3NSAtNS4wNzgxMjUgQyAyLjMwNDY4OCAtNS4xMTcxODgg%0D%0AMi4xNzE4NzUgLTUuMTQwNjI1IDIuMDE1NjI1IC01LjE0MDYyNSBDIDEuODQ3NjU2IC01LjE0MDYy%0D%0ANSAxLjY5MTQwNiAtNS4xMTMyODEgMS41NDY4NzUgLTUuMDYyNSBDIDEuMzk4NDM4IC01LjAxOTUz%0D%0AMSAxLjI2OTUzMSAtNC45Njg3NSAxLjE1NjI1IC00LjkwNjI1IEMgMS4wNTA3ODEgLTQuODQzNzUg%0D%0AMC45NjA5MzggLTQuNzg1MTU2IDAuODkwNjI1IC00LjczNDM3NSBDIDAuODE2NDA2IC00LjY3OTY4%0D%0AOCAwLjc2NTYyNSAtNC42NTYyNSAwLjczNDM3NSAtNC42NTYyNSBDIDAuNzEwOTM4IC00LjY1NjI1%0D%0AIDAuNjkxNDA2IC00LjY2MDE1NiAwLjY3MTg3NSAtNC42NzE4NzUgQyAwLjY2MDE1NiAtNC42Nzk2%0D%0AODggMC42NDg0MzggLTQuNjk1MzEyIDAuNjQwNjI1IC00LjcxODc1IEMgMC42Mjg5MDYgLTQuNzM4%0D%0AMjgxIDAuNjE3MTg4IC00Ljc2OTUzMSAwLjYwOTM3NSAtNC44MTI1IEMgMC42MDkzNzUgLTQuODUx%0D%0ANTYyIDAuNjA5Mzc1IC00LjkwNjI1IDAuNjA5Mzc1IC00Ljk2ODc1IEMgMC42MDkzNzUgLTUuMDA3%0D%0AODEyIDAuNjA5Mzc1IC01LjA0Njg3NSAwLjYwOTM3NSAtNS4wNzgxMjUgQyAwLjYxNzE4OCAtNS4x%0D%0AMTcxODggMC42MjUgLTUuMTQ4NDM4IDAuNjI1IC01LjE3MTg3NSBDIDAuNjMyODEyIC01LjE5MTQw%0D%0ANiAwLjY0NDUzMSAtNS4yMTA5MzggMC42NTYyNSAtNS4yMzQzNzUgQyAwLjY3NTc4MSAtNS4yNTM5%0D%0AMDYgMC42OTUzMTIgLTUuMjgxMjUgMC43MTg3NSAtNS4zMTI1IEMgMC43NSAtNS4zNDM3NSAwLjgw%0D%0ANDY4OCAtNS4zODI4MTIgMC44OTA2MjUgLTUuNDM3NSBDIDAuOTcyNjU2IC01LjQ4ODI4MSAxLjA3%0D%0AODEyNSAtNS41MzkwNjIgMS4yMDMxMjUgLTUuNTkzNzUgQyAxLjMyODEyNSAtNS42NDQ1MzEgMS40%0D%0ANjg3NSAtNS42ODc1IDEuNjI1IC01LjcxODc1IEMgMS43ODkwNjIgLTUuNzUgMS45Njg3NSAtNS43%0D%0ANjU2MjUgMi4xNTYyNSAtNS43NjU2MjUgQyAyLjQyNTc4MSAtNS43NjU2MjUgMi42NjAxNTYgLTUu%0D%0ANzI2NTYyIDIuODU5Mzc1IC01LjY1NjI1IEMgMy4wNTQ2ODggLTUuNTkzNzUgMy4yMjI2NTYgLTUu%0D%0ANSAzLjM1OTM3NSAtNS4zNzUgQyAzLjQ5MjE4OCAtNS4yNTc4MTIgMy41OTM3NSAtNS4xMTMyODEg%0D%0AMy42NTYyNSAtNC45Mzc1IEMgMy43MTg3NSAtNC43Njk1MzEgMy43NSAtNC41ODIwMzEgMy43NSAt%0D%0ANC4zNzUgQyAzLjc1IC00LjE5NTMxMiAzLjcyMjY1NiAtNC4wMzEyNSAzLjY3MTg3NSAtMy44NzUg%0D%0AQyAzLjYyODkwNiAtMy43MTg3NSAzLjU2MjUgLTMuNTc4MTI1IDMuNDY4NzUgLTMuNDUzMTI1IEMg%0D%0AMy4zODI4MTIgLTMuMzM1OTM4IDMuMjczNDM4IC0zLjIzODI4MSAzLjE0MDYyNSAtMy4xNTYyNSBD%0D%0AIDMuMDAzOTA2IC0zLjA3MDMxMiAyLjg1MTU2MiAtMy4wMTU2MjUgMi42ODc1IC0yLjk4NDM3NSBD%0D%0AIDIuODgyODEyIC0yLjk2MDkzOCAzLjA2MjUgLTIuOTEwMTU2IDMuMjE4NzUgLTIuODI4MTI1IEMg%0D%0AMy4zODI4MTIgLTIuNzUzOTA2IDMuNTIzNDM4IC0yLjY1NjI1IDMuNjQwNjI1IC0yLjUzMTI1IEMg%0D%0AMy43NTM5MDYgLTIuNDE0MDYyIDMuODQzNzUgLTIuMjgxMjUgMy45MDYyNSAtMi4xMjUgQyAzLjk3%0D%0ANjU2MiAtMS45Njg3NSA0LjAxNTYyNSAtMS44MDA3ODEgNC4wMTU2MjUgLTEuNjI1IFogTSA0LjAx%0D%0ANTYyNSAtMS42MjUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2aXNpYmxlIiBpZD0i%0D%0AZ2x5cGgwLTYiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDQuMjk2ODc1IC0xLjYy%0D%0ANSBDIDQuMjk2ODc1IC0xLjUxOTUzMSA0LjI4MTI1IC0xLjQ0MTQwNiA0LjI1IC0xLjM5MDYyNSBD%0D%0AIDQuMjI2NTYyIC0xLjMzNTkzOCA0LjE5MTQwNiAtMS4zMTI1IDQuMTQwNjI1IC0xLjMxMjUgTCAz%0D%0ALjUgLTEuMzEyNSBMIDMuNSAtMC4xMDkzNzUgQyAzLjUgLTAuMDg1OTM3NSAzLjQ4ODI4MSAtMC4w%0D%0ANjY0MDYyIDMuNDY4NzUgLTAuMDQ2ODc1IEMgMy40NTcwMzEgLTAuMDM1MTU2MiAzLjQzNzUgLTAu%0D%0AMDE5NTMxMiAzLjQwNjI1IDAgQyAzLjM4MjgxMiAwLjAwNzgxMjUgMy4zNDc2NTYgMC4wMTU2MjUg%0D%0AMy4yOTY4NzUgMC4wMTU2MjUgQyAzLjI1MzkwNiAwLjAyMzQzNzUgMy4xOTUzMTIgMC4wMzEyNSAz%0D%0ALjEyNSAwLjAzMTI1IEMgMy4wNTA3ODEgMC4wMzEyNSAyLjk4ODI4MSAwLjAyMzQzNzUgMi45Mzc1%0D%0AIDAuMDE1NjI1IEMgMi44OTQ1MzEgMC4wMTU2MjUgMi44NTkzNzUgMC4wMDc4MTI1IDIuODI4MTI1%0D%0AIDAgQyAyLjgwNDY4OCAtMC4wMTk1MzEyIDIuNzg5MDYyIC0wLjAzNTE1NjIgMi43ODEyNSAtMC4w%0D%0ANDY4NzUgQyAyLjc2OTUzMSAtMC4wNjY0MDYyIDIuNzY1NjI1IC0wLjA4NTkzNzUgMi43NjU2MjUg%0D%0ALTAuMTA5Mzc1IEwgMi43NjU2MjUgLTEuMzEyNSBMIDAuNDA2MjUgLTEuMzEyNSBDIDAuMzc1IC0x%0D%0ALjMxMjUgMC4zNDc2NTYgLTEuMzEyNSAwLjMyODEyNSAtMS4zMTI1IEMgMC4zMDQ2ODggLTEuMzIw%0D%0AMzEyIDAuMjg1MTU2IC0xLjMzNTkzOCAwLjI2NTYyNSAtMS4zNTkzNzUgQyAwLjI0MjE4OCAtMS4z%0D%0AOTA2MjUgMC4yMjY1NjIgLTEuNDI1NzgxIDAuMjE4NzUgLTEuNDY4NzUgQyAwLjIxODc1IC0xLjUw%0D%0ANzgxMiAwLjIxODc1IC0xLjU2NjQwNiAwLjIxODc1IC0xLjY0MDYyNSBDIDAuMjE4NzUgLTEuNjkx%0D%0ANDA2IDAuMjE4NzUgLTEuNzM4MjgxIDAuMjE4NzUgLTEuNzgxMjUgQyAwLjIxODc1IC0xLjgzMjAz%0D%0AMSAwLjIyMjY1NiAtMS44NzUgMC4yMzQzNzUgLTEuOTA2MjUgQyAwLjI0MjE4OCAtMS45NDUzMTIg%0D%0AMC4yNTM5MDYgLTEuOTg0Mzc1IDAuMjY1NjI1IC0yLjAxNTYyNSBDIDAuMjg1MTU2IC0yLjA0Njg3%0D%0ANSAwLjMwNDY4OCAtMi4wODIwMzEgMC4zMjgxMjUgLTIuMTI1IEwgMi4zNzUgLTUuNTc4MTI1IEMg%0D%0AMi4zODI4MTIgLTUuNTk3NjU2IDIuMzk4NDM4IC01LjYxNzE4OCAyLjQyMTg3NSAtNS42NDA2MjUg%0D%0AQyAyLjQ1MzEyNSAtNS42NjAxNTYgMi40ODgyODEgLTUuNjc1NzgxIDIuNTMxMjUgLTUuNjg3NSBD%0D%0AIDIuNTgyMDMxIC01LjY5NTMxMiAyLjY0MDYyNSAtNS43MDMxMjUgMi43MDMxMjUgLTUuNzAzMTI1%0D%0AIEMgMi43NjU2MjUgLTUuNzEwOTM4IDIuODQzNzUgLTUuNzE4NzUgMi45Mzc1IC01LjcxODc1IEMg%0D%0AMy4wMzEyNSAtNS43MTg3NSAzLjExMzI4MSAtNS43MTA5MzggMy4xODc1IC01LjcwMzEyNSBDIDMu%0D%0AMjU3ODEyIC01LjY5MTQwNiAzLjMxNjQwNiAtNS42Nzk2ODggMy4zNTkzNzUgLTUuNjcxODc1IEMg%0D%0AMy4zOTg0MzggLTUuNjYwMTU2IDMuNDI5Njg4IC01LjY0NDUzMSAzLjQ1MzEyNSAtNS42MjUgQyAz%0D%0ALjQ4NDM3NSAtNS42MDE1NjIgMy41IC01LjU4MjAzMSAzLjUgLTUuNTYyNSBMIDMuNSAtMS45MjE4%0D%0ANzUgTCA0LjE0MDYyNSAtMS45MjE4NzUgQyA0LjE3OTY4OCAtMS45MjE4NzUgNC4yMTg3NSAtMS44%0D%0AOTQ1MzEgNC4yNSAtMS44NDM3NSBDIDQuMjgxMjUgLTEuODAwNzgxIDQuMjk2ODc1IC0xLjcyNjU2%0D%0AMiA0LjI5Njg3NSAtMS42MjUgWiBNIDIuNzY1NjI1IC01LjA0Njg3NSBMIDIuNzUgLTUuMDQ2ODc1%0D%0AIEwgMC45MDYyNSAtMS45MjE4NzUgTCAyLjc2NTYyNSAtMS45MjE4NzUgWiBNIDIuNzY1NjI1IC01%0D%0ALjA0Njg3NSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlw%0D%0AaDEtMCI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0gNC4yOTY4NzUgLTUuNjg3NSBM%0D%0AIDQuMjk2ODc1IDAgTCAwLjE4NzUgMCBMIDAuMTg3NSAtNS42ODc1IFogTSAzLjkyMTg3NSAtMC4z%0D%0ANTkzNzUgTCAzLjkyMTg3NSAtNS4zMjgxMjUgTCAwLjU2MjUgLTUuMzI4MTI1IEwgMC41NjI1IC0w%0D%0ALjM1OTM3NSBaIE0gMy4zNzUgLTMuNzUgQyAzLjM3NSAtMy41ODIwMzEgMy4zNDc2NTYgLTMuNDI5%0D%0ANjg4IDMuMjk2ODc1IC0zLjI5Njg3NSBDIDMuMjQyMTg4IC0zLjE2MDE1NiAzLjE3MTg3NSAtMy4w%0D%0ANDY4NzUgMy4wNzgxMjUgLTIuOTUzMTI1IEMgMi45ODQzNzUgLTIuODU5Mzc1IDIuODc1IC0yLjc4%0D%0ANTE1NiAyLjc1IC0yLjczNDM3NSBDIDIuNjI1IC0yLjY3OTY4OCAyLjQ4ODI4MSAtMi42NDg0Mzgg%0D%0AMi4zNDM3NSAtMi42NDA2MjUgTCAyLjMxMjUgLTEuOTg0Mzc1IEMgMi4zMTI1IC0xLjk1MzEyNSAy%0D%0ALjI4OTA2MiAtMS45Mjk2ODggMi4yNSAtMS45MjE4NzUgQyAyLjIwNzAzMSAtMS45MTAxNTYgMi4x%0D%0ANDQ1MzEgLTEuOTA2MjUgMi4wNjI1IC0xLjkwNjI1IEMgMi4wMTk1MzEgLTEuOTA2MjUgMS45ODQz%0D%0ANzUgLTEuOTA2MjUgMS45NTMxMjUgLTEuOTA2MjUgQyAxLjkyOTY4OCAtMS45MDYyNSAxLjkxMDE1%0D%0ANiAtMS45MDYyNSAxLjg5MDYyNSAtMS45MDYyNSBDIDEuODY3MTg4IC0xLjkxNDA2MiAxLjg1MTU2%0D%0AMiAtMS45MjU3ODEgMS44NDM3NSAtMS45Mzc1IEMgMS44MzIwMzEgLTEuOTQ1MzEyIDEuODI4MTI1%0D%0AIC0xLjk2MDkzOCAxLjgyODEyNSAtMS45ODQzNzUgTCAxLjgxMjUgLTIuNzgxMjUgQyAxLjgwMDc4%0D%0AMSAtMi44NzUgMS44MTY0MDYgLTIuOTM3NSAxLjg1OTM3NSAtMi45Njg3NSBDIDEuODk4NDM4IC0z%0D%0ALjAwNzgxMiAxLjk1NzAzMSAtMy4wMzEyNSAyLjAzMTI1IC0zLjAzMTI1IEwgMi4xMDkzNzUgLTMu%0D%0AMDMxMjUgQyAyLjI0MjE4OCAtMy4wMzEyNSAyLjM1MTU2MiAtMy4wNDY4NzUgMi40Mzc1IC0zLjA3%0D%0AODEyNSBDIDIuNTMxMjUgLTMuMTE3MTg4IDIuNjAxNTYyIC0zLjE2NDA2MiAyLjY1NjI1IC0zLjIx%0D%0AODc1IEMgMi43MDcwMzEgLTMuMjgxMjUgMi43NDIxODggLTMuMzUxNTYyIDIuNzY1NjI1IC0zLjQz%0D%0ANzUgQyAyLjc4NTE1NiAtMy41MzEyNSAyLjc5Njg3NSAtMy42MjUgMi43OTY4NzUgLTMuNzE4NzUg%0D%0AQyAyLjc5Njg3NSAtMy45Mzc1IDIuNzM0Mzc1IC00LjEwOTM3NSAyLjYwOTM3NSAtNC4yMzQzNzUg%0D%0AQyAyLjQ5MjE4OCAtNC4zNTkzNzUgMi4zMDQ2ODggLTQuNDIxODc1IDIuMDQ2ODc1IC00LjQyMTg3%0D%0ANSBDIDEuOTI5Njg4IC00LjQyMTg3NSAxLjgyODEyNSAtNC40MTAxNTYgMS43MzQzNzUgLTQuMzkw%0D%0ANjI1IEMgMS42NDg0MzggLTQuMzY3MTg4IDEuNTcwMzEyIC00LjM0Mzc1IDEuNSAtNC4zMTI1IEMg%0D%0AMS40Mzc1IC00LjI4MTI1IDEuMzgyODEyIC00LjI1MzkwNiAxLjM0Mzc1IC00LjIzNDM3NSBDIDEu%0D%0AMzAwNzgxIC00LjIxMDkzOCAxLjI2OTUzMSAtNC4yMDMxMjUgMS4yNSAtNC4yMDMxMjUgQyAxLjIz%0D%0AODI4MSAtNC4yMDMxMjUgMS4yMjY1NjIgLTQuMjAzMTI1IDEuMjE4NzUgLTQuMjAzMTI1IEMgMS4y%0D%0AMDcwMzEgLTQuMjEwOTM4IDEuMTk1MzEyIC00LjIyMjY1NiAxLjE4NzUgLTQuMjM0Mzc1IEMgMS4x%0D%0AODc1IC00LjI1MzkwNiAxLjE3OTY4OCAtNC4yNzM0MzggMS4xNzE4NzUgLTQuMjk2ODc1IEMgMS4x%0D%0ANzE4NzUgLTQuMzI4MTI1IDEuMTcxODc1IC00LjM2NzE4OCAxLjE3MTg3NSAtNC40MjE4NzUgQyAx%0D%0ALjE3MTg3NSAtNC40NzI2NTYgMS4xNzE4NzUgLTQuNTE1NjI1IDEuMTcxODc1IC00LjU0Njg3NSBD%0D%0AIDEuMTc5Njg4IC00LjU3ODEyNSAxLjE5NTMxMiAtNC42MDE1NjIgMS4yMTg3NSAtNC42MjUgQyAx%0D%0ALjIzODI4MSAtNC42NDQ1MzEgMS4yODEyNSAtNC42NjQwNjIgMS4zNDM3NSAtNC42ODc1IEMgMS40%0D%0AMDYyNSAtNC43MTg3NSAxLjQ3MjY1NiAtNC43NDIxODggMS41NDY4NzUgLTQuNzY1NjI1IEMgMS42%0D%0AMjg5MDYgLTQuNzk2ODc1IDEuNzE4NzUgLTQuODE2NDA2IDEuODEyNSAtNC44MjgxMjUgQyAxLjkx%0D%0ANDA2MiAtNC44NDc2NTYgMi4wMTU2MjUgLTQuODU5Mzc1IDIuMTA5Mzc1IC00Ljg1OTM3NSBDIDIu%0D%0AMzI4MTI1IC00Ljg1OTM3NSAyLjUxNTYyNSAtNC44MjgxMjUgMi42NzE4NzUgLTQuNzY1NjI1IEMg%0D%0AMi44MzU5MzggLTQuNzEwOTM4IDIuOTY4NzUgLTQuNjMyODEyIDMuMDYyNSAtNC41MzEyNSBDIDMu%0D%0AMTY0MDYyIC00LjQzNzUgMy4yNDIxODggLTQuMzIwMzEyIDMuMjk2ODc1IC00LjE4NzUgQyAzLjM0%0D%0ANzY1NiAtNC4wNTA3ODEgMy4zNzUgLTMuOTA2MjUgMy4zNzUgLTMuNzUgWiBNIDIuNDA2MjUgLTEu%0D%0AMjAzMTI1IEMgMi40MDYyNSAtMS4xNDA2MjUgMi4zOTg0MzggLTEuMDg1OTM4IDIuMzkwNjI1IC0x%0D%0ALjA0Njg3NSBDIDIuMzc4OTA2IC0xLjAwMzkwNiAyLjM2MzI4MSAtMC45Njg3NSAyLjM0Mzc1IC0w%0D%0ALjkzNzUgQyAyLjMyMDMxMiAtMC45MTQwNjIgMi4yODkwNjIgLTAuODk4NDM4IDIuMjUgLTAuODkw%0D%0ANjI1IEMgMi4yMDcwMzEgLTAuODc4OTA2IDIuMTU2MjUgLTAuODc1IDIuMDkzNzUgLTAuODc1IEMg%0D%0AMi4wMTk1MzEgLTAuODc1IDEuOTYwOTM4IC0wLjg3ODkwNiAxLjkyMTg3NSAtMC44OTA2MjUgQyAx%0D%0ALjg3ODkwNiAtMC44OTg0MzggMS44NDM3NSAtMC45MTQwNjIgMS44MTI1IC0wLjkzNzUgQyAxLjc4%0D%0AOTA2MiAtMC45Njg3NSAxLjc3MzQzOCAtMS4wMDM5MDYgMS43NjU2MjUgLTEuMDQ2ODc1IEMgMS43%0D%0ANTM5MDYgLTEuMDg1OTM4IDEuNzUgLTEuMTQwNjI1IDEuNzUgLTEuMjAzMTI1IEMgMS43NSAtMS4y%0D%0ANjU2MjUgMS43NTM5MDYgLTEuMzE2NDA2IDEuNzY1NjI1IC0xLjM1OTM3NSBDIDEuNzczNDM4IC0x%0D%0ALjM5ODQzOCAxLjc4OTA2MiAtMS40Mjk2ODggMS44MTI1IC0xLjQ1MzEyNSBDIDEuODQzNzUgLTEu%0D%0ANDg0Mzc1IDEuODc4OTA2IC0xLjUwMzkwNiAxLjkyMTg3NSAtMS41MTU2MjUgQyAxLjk2MDkzOCAt%0D%0AMS41MjM0MzggMi4wMTk1MzEgLTEuNTMxMjUgMi4wOTM3NSAtMS41MzEyNSBDIDIuMTU2MjUgLTEu%0D%0ANTMxMjUgMi4yMDcwMzEgLTEuNTIzNDM4IDIuMjUgLTEuNTE1NjI1IEMgMi4yODkwNjIgLTEuNTAz%0D%0AOTA2IDIuMzIwMzEyIC0xLjQ4NDM3NSAyLjM0Mzc1IC0xLjQ1MzEyNSBDIDIuMzYzMjgxIC0xLjQy%0D%0AOTY4OCAyLjM3ODkwNiAtMS4zOTg0MzggMi4zOTA2MjUgLTEuMzU5Mzc1IEMgMi4zOTg0MzggLTEu%0D%0AMzE2NDA2IDIuNDA2MjUgLTEuMjY1NjI1IDIuNDA2MjUgLTEuMjAzMTI1IFogTSAwIDEuNzUgWiBN%0D%0AIDAgMS43NSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlw%0D%0AaDEtMSI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0gNC4wNzgxMjUgLTAuMjk2ODc1%0D%0AIEMgNC4wNzgxMjUgLTAuMjQyMTg4IDQuMDcwMzEyIC0wLjE5NTMxMiA0LjA2MjUgLTAuMTU2MjUg%0D%0AQyA0LjA1MDc4MSAtMC4xMTMyODEgNC4wMzUxNTYgLTAuMDgyMDMxMiA0LjAxNTYyNSAtMC4wNjI1%0D%0AIEMgNC4wMDM5MDYgLTAuMDM5MDYyNSAzLjk4ODI4MSAtMC4wMjM0Mzc1IDMuOTY4NzUgLTAuMDE1%0D%0ANjI1IEMgMy45NTcwMzEgLTAuMDAzOTA2MjUgMy45NDE0MDYgMCAzLjkyMTg3NSAwIEwgMC45Mzc1%0D%0AIDAgQyAwLjkyNTc4MSAwIDAuOTEwMTU2IC0wLjAwMzkwNjI1IDAuODkwNjI1IC0wLjAxNTYyNSBD%0D%0AIDAuODY3MTg4IC0wLjAyMzQzNzUgMC44NDc2NTYgLTAuMDM5MDYyNSAwLjgyODEyNSAtMC4wNjI1%0D%0AIEMgMC44MTY0MDYgLTAuMDgyMDMxMiAwLjgwNDY4OCAtMC4xMTMyODEgMC43OTY4NzUgLTAuMTU2%0D%0AMjUgQyAwLjc4NTE1NiAtMC4xOTUzMTIgMC43ODEyNSAtMC4yNDIxODggMC43ODEyNSAtMC4yOTY4%0D%0ANzUgQyAwLjc4MTI1IC0wLjM0NzY1NiAwLjc4NTE1NiAtMC4zOTQ1MzEgMC43OTY4NzUgLTAuNDM3%0D%0ANSBDIDAuODA0Njg4IC0wLjQ3NjU2MiAwLjgxNjQwNiAtMC41MDc4MTIgMC44MjgxMjUgLTAuNTMx%0D%0AMjUgQyAwLjg0NzY1NiAtMC41NTA3ODEgMC44NjMyODEgLTAuNTY2NDA2IDAuODc1IC0wLjU3ODEy%0D%0ANSBDIDAuODk0NTMxIC0wLjU5NzY1NiAwLjkxNDA2MiAtMC42MDkzNzUgMC45Mzc1IC0wLjYwOTM3%0D%0ANSBMIDIuMTI1IC0wLjYwOTM3NSBMIDIuMTI1IC00LjkzNzUgTCAxLjAzMTI1IC00LjI4MTI1IEMg%0D%0AMC45NzY1NjIgLTQuMjUgMC45Mjk2ODggLTQuMjI2NTYyIDAuODkwNjI1IC00LjIxODc1IEMgMC44%0D%0ANTkzNzUgLTQuMjE4NzUgMC44MzIwMzEgLTQuMjIyNjU2IDAuODEyNSAtNC4yMzQzNzUgQyAwLjc4%0D%0AOTA2MiAtNC4yNTM5MDYgMC43NzM0MzggLTQuMjg1MTU2IDAuNzY1NjI1IC00LjMyODEyNSBDIDAu%0D%0ANzY1NjI1IC00LjM3ODkwNiAwLjc2NTYyNSAtNC40Mzc1IDAuNzY1NjI1IC00LjUgQyAwLjc2NTYy%0D%0ANSAtNC41NTA3ODEgMC43NjU2MjUgLTQuNTkzNzUgMC43NjU2MjUgLTQuNjI1IEMgMC43NzM0Mzgg%0D%0ALTQuNjY0MDYyIDAuNzgxMjUgLTQuNjk1MzEyIDAuNzgxMjUgLTQuNzE4NzUgQyAwLjc4OTA2MiAt%0D%0ANC43MzgyODEgMC44MDQ2ODggLTQuNzU3ODEyIDAuODI4MTI1IC00Ljc4MTI1IEMgMC44NDc2NTYg%0D%0ALTQuODAwNzgxIDAuODY3MTg4IC00LjgxNjQwNiAwLjg5MDYyNSAtNC44MjgxMjUgTCAyLjIwMzEy%0D%0ANSAtNS42NzE4NzUgQyAyLjIxMDkzOCAtNS42Nzk2ODggMi4yMjI2NTYgLTUuNjg3NSAyLjIzNDM3%0D%0ANSAtNS42ODc1IEMgMi4yNTM5MDYgLTUuNjk1MzEyIDIuMjczNDM4IC01LjcwMzEyNSAyLjI5Njg3%0D%0ANSAtNS43MDMxMjUgQyAyLjMyODEyNSAtNS43MTA5MzggMi4zNTkzNzUgLTUuNzE4NzUgMi4zOTA2%0D%0AMjUgLTUuNzE4NzUgQyAyLjQyOTY4OCAtNS43MTg3NSAyLjQ3NjU2MiAtNS43MTg3NSAyLjUzMTI1%0D%0AIC01LjcxODc1IEMgMi42MDE1NjIgLTUuNzE4NzUgMi42NjAxNTYgLTUuNzEwOTM4IDIuNzAzMTI1%0D%0AIC01LjcwMzEyNSBDIDIuNzUzOTA2IC01LjcwMzEyNSAyLjc4OTA2MiAtNS42OTUzMTIgMi44MTI1%0D%0AIC01LjY4NzUgQyAyLjg0Mzc1IC01LjY3NTc4MSAyLjg2MzI4MSAtNS42NjAxNTYgMi44NzUgLTUu%0D%0ANjQwNjI1IEMgMi44ODI4MTIgLTUuNjI4OTA2IDIuODkwNjI1IC01LjYxNzE4OCAyLjg5MDYyNSAt%0D%0ANS42MDkzNzUgTCAyLjg5MDYyNSAtMC42MDkzNzUgTCAzLjkyMTg3NSAtMC42MDkzNzUgQyAzLjk0%0D%0AMTQwNiAtMC42MDkzNzUgMy45NjA5MzggLTAuNTk3NjU2IDMuOTg0Mzc1IC0wLjU3ODEyNSBDIDQu%0D%0AMDAzOTA2IC0wLjU2NjQwNiA0LjAxOTUzMSAtMC41NTA3ODEgNC4wMzEyNSAtMC41MzEyNSBDIDQu%0D%0AMDM5MDYyIC0wLjUwNzgxMiA0LjA1MDc4MSAtMC40NzY1NjIgNC4wNjI1IC0wLjQzNzUgQyA0LjA3%0D%0AMDMxMiAtMC4zOTQ1MzEgNC4wNzgxMjUgLTAuMzQ3NjU2IDQuMDc4MTI1IC0wLjI5Njg3NSBaIE0g%0D%0ANC4wNzgxMjUgLTAuMjk2ODc1ICIvPgo8L3N5bWJvbD4KPHN5bWJvbCBvdmVyZmxvdz0idmlzaWJs%0D%0AZSIgaWQ9ImdseXBoMS0yIj4KPHBhdGggc3R5bGU9InN0cm9rZTpub25lOyIgZD0iTSA0LjA2MjUg%0D%0ALTAuMzI4MTI1IEMgNC4wNjI1IC0wLjI3MzQzOCA0LjA1NDY4OCAtMC4yMjY1NjIgNC4wNDY4NzUg%0D%0ALTAuMTg3NSBDIDQuMDM1MTU2IC0wLjE0NDUzMSA0LjAyMzQzOCAtMC4xMDkzNzUgNC4wMTU2MjUg%0D%0ALTAuMDc4MTI1IEMgNC4wMDM5MDYgLTAuMDU0Njg3NSAzLjk4ODI4MSAtMC4wMzUxNTYyIDMuOTY4%0D%0ANzUgLTAuMDE1NjI1IEMgMy45NDUzMTIgLTAuMDAzOTA2MjUgMy45MjE4NzUgMCAzLjg5MDYyNSAw%0D%0AIEwgMC43NSAwIEMgMC43MDcwMzEgMCAwLjY3MTg3NSAtMC4wMDM5MDYyNSAwLjY0MDYyNSAtMC4w%0D%0AMTU2MjUgQyAwLjYwOTM3NSAtMC4wMjM0Mzc1IDAuNTgyMDMxIC0wLjAzOTA2MjUgMC41NjI1IC0w%0D%0ALjA2MjUgQyAwLjUzOTA2MiAtMC4wODIwMzEyIDAuNTIzNDM4IC0wLjExMzI4MSAwLjUxNTYyNSAt%0D%0AMC4xNTYyNSBDIDAuNTAzOTA2IC0wLjIwNzAzMSAwLjUgLTAuMjY1NjI1IDAuNSAtMC4zMjgxMjUg%0D%0AQyAwLjUgLTAuMzkwNjI1IDAuNSAtMC40NDE0MDYgMC41IC0wLjQ4NDM3NSBDIDAuNTA3ODEyIC0w%0D%0ALjUyMzQzOCAwLjUxOTUzMSAtMC41NjI1IDAuNTMxMjUgLTAuNTkzNzUgQyAwLjU1MDc4MSAtMC42%0D%0AMzI4MTIgMC41NzAzMTIgLTAuNjcxODc1IDAuNTkzNzUgLTAuNzAzMTI1IEMgMC42MTMyODEgLTAu%0D%0ANzM0Mzc1IDAuNjQwNjI1IC0wLjc2OTUzMSAwLjY3MTg3NSAtMC44MTI1IEwgMS43ODEyNSAtMS45%0D%0AODQzNzUgQyAyLjAzMTI1IC0yLjI0MjE4OCAyLjIzNDM3NSAtMi40ODQzNzUgMi4zOTA2MjUgLTIu%0D%0ANzAzMTI1IEMgMi41NDY4NzUgLTIuOTIxODc1IDIuNjY0MDYyIC0zLjExMzI4MSAyLjc1IC0zLjI4%0D%0AMTI1IEMgMi44MzIwMzEgLTMuNDU3MDMxIDIuODgyODEyIC0zLjYxNzE4OCAyLjkwNjI1IC0zLjc2%0D%0ANTYyNSBDIDIuOTM3NSAtMy45MTAxNTYgMi45NTMxMjUgLTQuMDQ2ODc1IDIuOTUzMTI1IC00LjE3%0D%0AMTg3NSBDIDIuOTUzMTI1IC00LjI5Njg3NSAyLjkyOTY4OCAtNC40MTQwNjIgMi44OTA2MjUgLTQu%0D%0ANTMxMjUgQyAyLjg0NzY1NiAtNC42NDQ1MzEgMi43ODUxNTYgLTQuNzQyMTg4IDIuNzAzMTI1IC00%0D%0ALjgyODEyNSBDIDIuNjI4OTA2IC00LjkxMDE1NiAyLjUzNTE1NiAtNC45NzY1NjIgMi40MjE4NzUg%0D%0ALTUuMDMxMjUgQyAyLjMwNDY4OCAtNS4wODIwMzEgMi4xNzE4NzUgLTUuMTA5Mzc1IDIuMDE1NjI1%0D%0AIC01LjEwOTM3NSBDIDEuODM1OTM4IC01LjEwOTM3NSAxLjY3NTc4MSAtNS4wODIwMzEgMS41MzEy%0D%0ANSAtNS4wMzEyNSBDIDEuMzk0NTMxIC00Ljk3NjU2MiAxLjI2OTUzMSAtNC45MjE4NzUgMS4xNTYy%0D%0ANSAtNC44NTkzNzUgQyAxLjA1MDc4MSAtNC44MDQ2ODggMC45NjA5MzggLTQuNzUzOTA2IDAuODkw%0D%0ANjI1IC00LjcwMzEyNSBDIDAuODE2NDA2IC00LjY0ODQzOCAwLjc2NTYyNSAtNC42MjUgMC43MzQz%0D%0ANzUgLTQuNjI1IEMgMC43MTA5MzggLTQuNjI1IDAuNjkxNDA2IC00LjYyODkwNiAwLjY3MTg3NSAt%0D%0ANC42NDA2MjUgQyAwLjY2MDE1NiAtNC42NDg0MzggMC42NDg0MzggLTQuNjY0MDYyIDAuNjQwNjI1%0D%0AIC00LjY4NzUgQyAwLjYyODkwNiAtNC43MTg3NSAwLjYxNzE4OCAtNC43NTM5MDYgMC42MDkzNzUg%0D%0ALTQuNzk2ODc1IEMgMC42MDkzNzUgLTQuODQ3NjU2IDAuNjA5Mzc1IC00LjkwNjI1IDAuNjA5Mzc1%0D%0AIC00Ljk2ODc1IEMgMC42MDkzNzUgLTUuMDA3ODEyIDAuNjA5Mzc1IC01LjA0Njg3NSAwLjYwOTM3%0D%0ANSAtNS4wNzgxMjUgQyAwLjYwOTM3NSAtNS4xMDkzNzUgMC42MTMyODEgLTUuMTMyODEyIDAuNjI1%0D%0AIC01LjE1NjI1IEMgMC42MzI4MTIgLTUuMTg3NSAwLjY0NDUzMSAtNS4yMTA5MzggMC42NTYyNSAt%0D%0ANS4yMzQzNzUgQyAwLjY2NDA2MiAtNS4yNTM5MDYgMC42ODc1IC01LjI4MTI1IDAuNzE4NzUgLTUu%0D%0AMzEyNSBDIDAuNzU3ODEyIC01LjM0Mzc1IDAuODIwMzEyIC01LjM4MjgxMiAwLjkwNjI1IC01LjQz%0D%0ANzUgQyAwLjk4ODI4MSAtNS40ODgyODEgMS4wOTM3NSAtNS41MzkwNjIgMS4yMTg3NSAtNS41OTM3%0D%0ANSBDIDEuMzUxNTYyIC01LjY0NDUzMSAxLjUgLTUuNjg3NSAxLjY1NjI1IC01LjcxODc1IEMgMS44%0D%0AMjAzMTIgLTUuNzUgMS45ODgyODEgLTUuNzY1NjI1IDIuMTU2MjUgLTUuNzY1NjI1IEMgMi40Mzc1%0D%0AIC01Ljc2NTYyNSAyLjY3NTc4MSAtNS43MjI2NTYgMi44NzUgLTUuNjQwNjI1IEMgMy4wODIwMzEg%0D%0ALTUuNTY2NDA2IDMuMjUzOTA2IC01LjQ2MDkzOCAzLjM5MDYyNSAtNS4zMjgxMjUgQyAzLjUyMzQz%0D%0AOCAtNS4yMDMxMjUgMy42MjUgLTUuMDQ2ODc1IDMuNjg3NSAtNC44NTkzNzUgQyAzLjc1NzgxMiAt%0D%0ANC42Nzk2ODggMy43OTY4NzUgLTQuNDkyMTg4IDMuNzk2ODc1IC00LjI5Njg3NSBDIDMuNzk2ODc1%0D%0AIC00LjEwOTM3NSAzLjc4MTI1IC0zLjkyNTc4MSAzLjc1IC0zLjc1IEMgMy43MTg3NSAtMy41NzAz%0D%0AMTIgMy42NDg0MzggLTMuMzc4OTA2IDMuNTQ2ODc1IC0zLjE3MTg3NSBDIDMuNDQxNDA2IC0yLjk2%0D%0AMDkzOCAzLjI4OTA2MiAtMi43MjY1NjIgMy4wOTM3NSAtMi40Njg3NSBDIDIuODk0NTMxIC0yLjIw%0D%0ANzAzMSAyLjYzMjgxMiAtMS45MTAxNTYgMi4zMTI1IC0xLjU3ODEyNSBMIDEuNDIxODc1IC0wLjY0%0D%0AMDYyNSBMIDMuODkwNjI1IC0wLjY0MDYyNSBDIDMuOTEwMTU2IC0wLjY0MDYyNSAzLjkyOTY4OCAt%0D%0AMC42MzI4MTIgMy45NTMxMjUgLTAuNjI1IEMgMy45NzI2NTYgLTAuNjEzMjgxIDMuOTg4MjgxIC0w%0D%0ALjU5Mzc1IDQgLTAuNTYyNSBDIDQuMDE5NTMxIC0wLjUzMTI1IDQuMDM1MTU2IC0wLjQ5MjE4OCA0%0D%0ALjA0Njg3NSAtMC40NTMxMjUgQyA0LjA1NDY4OCAtMC40MjE4NzUgNC4wNjI1IC0wLjM3ODkwNiA0%0D%0ALjA2MjUgLTAuMzI4MTI1IFogTSA0LjA2MjUgLTAuMzI4MTI1ICIvPgo8L3N5bWJvbD4KPHN5bWJv%0D%0AbCBvdmVyZmxvdz0idmlzaWJsZSIgaWQ9ImdseXBoMS0zIj4KPHBhdGggc3R5bGU9InN0cm9rZTpu%0D%0Ab25lOyIgZD0iTSA0LjAxNTYyNSAtMS42MjUgQyA0LjAxNTYyNSAtMS4zNjMyODEgMy45Njg3NSAt%0D%0AMS4xMjg5MDYgMy44NzUgLTAuOTIxODc1IEMgMy43ODkwNjIgLTAuNzEwOTM4IDMuNjYwMTU2IC0w%0D%0ALjUzMTI1IDMuNDg0Mzc1IC0wLjM3NSBDIDMuMzE2NDA2IC0wLjIyNjU2MiAzLjEwOTM3NSAtMC4x%0D%0AMTMyODEgMi44NTkzNzUgLTAuMDMxMjUgQyAyLjYwOTM3NSAwLjAzOTA2MjUgMi4zMjgxMjUgMC4w%0D%0ANzgxMjUgMi4wMTU2MjUgMC4wNzgxMjUgQyAxLjgyODEyNSAwLjA3ODEyNSAxLjY0ODQzOCAwLjA2%0D%0AMjUgMS40ODQzNzUgMC4wMzEyNSBDIDEuMzE2NDA2IDAgMS4xNjQwNjIgLTAuMDM1MTU2MiAxLjAz%0D%0AMTI1IC0wLjA3ODEyNSBDIDAuOTA2MjUgLTAuMTE3MTg4IDAuODAwNzgxIC0wLjE2MDE1NiAwLjcx%0D%0AODc1IC0wLjIwMzEyNSBDIDAuNjMyODEyIC0wLjI0MjE4OCAwLjU4MjAzMSAtMC4yNzM0MzggMC41%0D%0ANjI1IC0wLjI5Njg3NSBDIDAuNTM5MDYyIC0wLjMxNjQwNiAwLjUyMzQzOCAtMC4zMzU5MzggMC41%0D%0AMTU2MjUgLTAuMzU5Mzc1IEMgMC41MDM5MDYgLTAuMzc4OTA2IDAuNDkyMTg4IC0wLjQwNjI1IDAu%0D%0ANDg0Mzc1IC0wLjQzNzUgQyAwLjQ3MjY1NiAtMC40Njg3NSAwLjQ2MDkzOCAtMC41MDM5MDYgMC40%0D%0ANTMxMjUgLTAuNTQ2ODc1IEMgMC40NTMxMjUgLTAuNTg1OTM4IDAuNDUzMTI1IC0wLjY0MDYyNSAw%0D%0ALjQ1MzEyNSAtMC43MDMxMjUgQyAwLjQ1MzEyNSAtMC44MDQ2ODggMC40NjA5MzggLTAuODc4OTA2%0D%0AIDAuNDg0Mzc1IC0wLjkyMTg3NSBDIDAuNTAzOTA2IC0wLjk2MDkzOCAwLjUzNTE1NiAtMC45ODQz%0D%0ANzUgMC41NzgxMjUgLTAuOTg0Mzc1IEMgMC41OTc2NTYgLTAuOTg0Mzc1IDAuNjQ0NTMxIC0wLjk1%0D%0ANzAzMSAwLjcxODc1IC0wLjkwNjI1IEMgMC44MDA3ODEgLTAuODYzMjgxIDAuODk4NDM4IC0wLjgx%0D%0ANjQwNiAxLjAxNTYyNSAtMC43NjU2MjUgQyAxLjE0MDYyNSAtMC43MTA5MzggMS4yODUxNTYgLTAu%0D%0ANjYwMTU2IDEuNDUzMTI1IC0wLjYwOTM3NSBDIDEuNjE3MTg4IC0wLjU2NjQwNiAxLjgwMDc4MSAt%0D%0AMC41NDY4NzUgMiAtMC41NDY4NzUgQyAyLjE4NzUgLTAuNTQ2ODc1IDIuMzUxNTYyIC0wLjU2NjQw%0D%0ANiAyLjUgLTAuNjA5Mzc1IEMgMi42NDQ1MzEgLTAuNjYwMTU2IDIuNzY5NTMxIC0wLjczNDM3NSAy%0D%0ALjg3NSAtMC44MjgxMjUgQyAyLjk3NjU2MiAtMC45MjE4NzUgMy4wNTA3ODEgLTEuMDMxMjUgMy4w%0D%0AOTM3NSAtMS4xNTYyNSBDIDMuMTQ0NTMxIC0xLjI4MTI1IDMuMTcxODc1IC0xLjQxNDA2MiAzLjE3%0D%0AMTg3NSAtMS41NjI1IEMgMy4xNzE4NzUgLTEuNzE4NzUgMy4xNDA2MjUgLTEuODU5Mzc1IDMuMDc4%0D%0AMTI1IC0xLjk4NDM3NSBDIDMuMDE1NjI1IC0yLjExNzE4OCAyLjkyMTg3NSAtMi4yMzQzNzUgMi43%0D%0AOTY4NzUgLTIuMzI4MTI1IEMgMi42Nzk2ODggLTIuNDIxODc1IDIuNTM1MTU2IC0yLjQ4ODI4MSAy%0D%0ALjM1OTM3NSAtMi41MzEyNSBDIDIuMTc5Njg4IC0yLjU4MjAzMSAxLjk3NjU2MiAtMi42MDkzNzUg%0D%0AMS43NSAtMi42MDkzNzUgTCAxLjIwMzEyNSAtMi42MDkzNzUgQyAxLjE3OTY4OCAtMi42MDkzNzUg%0D%0AMS4xNjAxNTYgLTIuNjEzMjgxIDEuMTQwNjI1IC0yLjYyNSBDIDEuMTE3MTg4IC0yLjYzMjgxMiAx%0D%0ALjA5NzY1NiAtMi42NDg0MzggMS4wNzgxMjUgLTIuNjcxODc1IEMgMS4wNjY0MDYgLTIuNzAzMTI1%0D%0AIDEuMDU0Njg4IC0yLjczNDM3NSAxLjA0Njg3NSAtMi43NjU2MjUgQyAxLjAzNTE1NiAtMi44MDQ2%0D%0AODggMS4wMzEyNSAtMi44NTkzNzUgMS4wMzEyNSAtMi45MjE4NzUgQyAxLjAzMTI1IC0yLjk3MjY1%0D%0ANiAxLjAzNTE1NiAtMy4wMTU2MjUgMS4wNDY4NzUgLTMuMDQ2ODc1IEMgMS4wNTQ2ODggLTMuMDg1%0D%0AOTM4IDEuMDY2NDA2IC0zLjExNzE4OCAxLjA3ODEyNSAtMy4xNDA2MjUgQyAxLjA5NzY1NiAtMy4x%0D%0ANzE4NzUgMS4xMTcxODggLTMuMTkxNDA2IDEuMTQwNjI1IC0zLjIwMzEyNSBDIDEuMTYwMTU2IC0z%0D%0ALjIxMDkzOCAxLjE3OTY4OCAtMy4yMTg3NSAxLjIwMzEyNSAtMy4yMTg3NSBMIDEuNzAzMTI1IC0z%0D%0ALjIxODc1IEMgMS44OTA2MjUgLTMuMjE4NzUgMi4wNjI1IC0zLjIzODI4MSAyLjIxODc1IC0zLjI4%0D%0AMTI1IEMgMi4zNzUgLTMuMzMyMDMxIDIuNTAzOTA2IC0zLjQwNjI1IDIuNjA5Mzc1IC0zLjUgQyAy%0D%0ALjcxMDkzOCAtMy41OTM3NSAyLjc4OTA2MiAtMy43MDMxMjUgMi44NDM3NSAtMy44MjgxMjUgQyAy%0D%0ALjkwNjI1IC0zLjk2MDkzOCAyLjkzNzUgLTQuMTEzMjgxIDIuOTM3NSAtNC4yODEyNSBDIDIuOTM3%0D%0ANSAtNC4zOTQ1MzEgMi45MTQwNjIgLTQuNTAzOTA2IDIuODc1IC00LjYwOTM3NSBDIDIuODQzNzUg%0D%0ALTQuNzEwOTM4IDIuNzg5MDYyIC00LjgwNDY4OCAyLjcxODc1IC00Ljg5MDYyNSBDIDIuNjQ0NTMx%0D%0AIC00Ljk3MjY1NiAyLjU0Njg3NSAtNS4wMzUxNTYgMi40MjE4NzUgLTUuMDc4MTI1IEMgMi4zMDQ2%0D%0AODggLTUuMTE3MTg4IDIuMTcxODc1IC01LjE0MDYyNSAyLjAxNTYyNSAtNS4xNDA2MjUgQyAxLjg0%0D%0ANzY1NiAtNS4xNDA2MjUgMS42OTE0MDYgLTUuMTEzMjgxIDEuNTQ2ODc1IC01LjA2MjUgQyAxLjM5%0D%0AODQzOCAtNS4wMTk1MzEgMS4yNjk1MzEgLTQuOTY4NzUgMS4xNTYyNSAtNC45MDYyNSBDIDEuMDUw%0D%0ANzgxIC00Ljg0Mzc1IDAuOTYwOTM4IC00Ljc4NTE1NiAwLjg5MDYyNSAtNC43MzQzNzUgQyAwLjgx%0D%0ANjQwNiAtNC42Nzk2ODggMC43NjU2MjUgLTQuNjU2MjUgMC43MzQzNzUgLTQuNjU2MjUgQyAwLjcx%0D%0AMDkzOCAtNC42NTYyNSAwLjY5MTQwNiAtNC42NjAxNTYgMC42NzE4NzUgLTQuNjcxODc1IEMgMC42%0D%0ANjAxNTYgLTQuNjc5Njg4IDAuNjQ4NDM4IC00LjY5NTMxMiAwLjY0MDYyNSAtNC43MTg3NSBDIDAu%0D%0ANjI4OTA2IC00LjczODI4MSAwLjYxNzE4OCAtNC43Njk1MzEgMC42MDkzNzUgLTQuODEyNSBDIDAu%0D%0ANjA5Mzc1IC00Ljg1MTU2MiAwLjYwOTM3NSAtNC45MDYyNSAwLjYwOTM3NSAtNC45Njg3NSBDIDAu%0D%0ANjA5Mzc1IC01LjAwNzgxMiAwLjYwOTM3NSAtNS4wNDY4NzUgMC42MDkzNzUgLTUuMDc4MTI1IEMg%0D%0AMC42MTcxODggLTUuMTE3MTg4IDAuNjI1IC01LjE0ODQzOCAwLjYyNSAtNS4xNzE4NzUgQyAwLjYz%0D%0AMjgxMiAtNS4xOTE0MDYgMC42NDQ1MzEgLTUuMjEwOTM4IDAuNjU2MjUgLTUuMjM0Mzc1IEMgMC42%0D%0ANzU3ODEgLTUuMjUzOTA2IDAuNjk1MzEyIC01LjI4MTI1IDAuNzE4NzUgLTUuMzEyNSBDIDAuNzUg%0D%0ALTUuMzQzNzUgMC44MDQ2ODggLTUuMzgyODEyIDAuODkwNjI1IC01LjQzNzUgQyAwLjk3MjY1NiAt%0D%0ANS40ODgyODEgMS4wNzgxMjUgLTUuNTM5MDYyIDEuMjAzMTI1IC01LjU5Mzc1IEMgMS4zMjgxMjUg%0D%0ALTUuNjQ0NTMxIDEuNDY4NzUgLTUuNjg3NSAxLjYyNSAtNS43MTg3NSBDIDEuNzg5MDYyIC01Ljc1%0D%0AIDEuOTY4NzUgLTUuNzY1NjI1IDIuMTU2MjUgLTUuNzY1NjI1IEMgMi40MjU3ODEgLTUuNzY1NjI1%0D%0AIDIuNjYwMTU2IC01LjcyNjU2MiAyLjg1OTM3NSAtNS42NTYyNSBDIDMuMDU0Njg4IC01LjU5Mzc1%0D%0AIDMuMjIyNjU2IC01LjUgMy4zNTkzNzUgLTUuMzc1IEMgMy40OTIxODggLTUuMjU3ODEyIDMuNTkz%0D%0ANzUgLTUuMTEzMjgxIDMuNjU2MjUgLTQuOTM3NSBDIDMuNzE4NzUgLTQuNzY5NTMxIDMuNzUgLTQu%0D%0ANTgyMDMxIDMuNzUgLTQuMzc1IEMgMy43NSAtNC4xOTUzMTIgMy43MjI2NTYgLTQuMDMxMjUgMy42%0D%0ANzE4NzUgLTMuODc1IEMgMy42Mjg5MDYgLTMuNzE4NzUgMy41NjI1IC0zLjU3ODEyNSAzLjQ2ODc1%0D%0AIC0zLjQ1MzEyNSBDIDMuMzgyODEyIC0zLjMzNTkzOCAzLjI3MzQzOCAtMy4yMzgyODEgMy4xNDA2%0D%0AMjUgLTMuMTU2MjUgQyAzLjAwMzkwNiAtMy4wNzAzMTIgMi44NTE1NjIgLTMuMDE1NjI1IDIuNjg3%0D%0ANSAtMi45ODQzNzUgQyAyLjg4MjgxMiAtMi45NjA5MzggMy4wNjI1IC0yLjkxMDE1NiAzLjIxODc1%0D%0AIC0yLjgyODEyNSBDIDMuMzgyODEyIC0yLjc1MzkwNiAzLjUyMzQzOCAtMi42NTYyNSAzLjY0MDYy%0D%0ANSAtMi41MzEyNSBDIDMuNzUzOTA2IC0yLjQxNDA2MiAzLjg0Mzc1IC0yLjI4MTI1IDMuOTA2MjUg%0D%0ALTIuMTI1IEMgMy45NzY1NjIgLTEuOTY4NzUgNC4wMTU2MjUgLTEuODAwNzgxIDQuMDE1NjI1IC0x%0D%0ALjYyNSBaIE0gNC4wMTU2MjUgLTEuNjI1ICIvPgo8L3N5bWJvbD4KPHN5bWJvbCBvdmVyZmxvdz0i%0D%0AdmlzaWJsZSIgaWQ9ImdseXBoMS00Ij4KPHBhdGggc3R5bGU9InN0cm9rZTpub25lOyIgZD0iTSA0%0D%0ALjI5Njg3NSAtMS42MjUgQyA0LjI5Njg3NSAtMS41MTk1MzEgNC4yODEyNSAtMS40NDE0MDYgNC4y%0D%0ANSAtMS4zOTA2MjUgQyA0LjIyNjU2MiAtMS4zMzU5MzggNC4xOTE0MDYgLTEuMzEyNSA0LjE0MDYy%0D%0ANSAtMS4zMTI1IEwgMy41IC0xLjMxMjUgTCAzLjUgLTAuMTA5Mzc1IEMgMy41IC0wLjA4NTkzNzUg%0D%0AMy40ODgyODEgLTAuMDY2NDA2MiAzLjQ2ODc1IC0wLjA0Njg3NSBDIDMuNDU3MDMxIC0wLjAzNTE1%0D%0ANjIgMy40Mzc1IC0wLjAxOTUzMTIgMy40MDYyNSAwIEMgMy4zODI4MTIgMC4wMDc4MTI1IDMuMzQ3%0D%0ANjU2IDAuMDE1NjI1IDMuMjk2ODc1IDAuMDE1NjI1IEMgMy4yNTM5MDYgMC4wMjM0Mzc1IDMuMTk1%0D%0AMzEyIDAuMDMxMjUgMy4xMjUgMC4wMzEyNSBDIDMuMDUwNzgxIDAuMDMxMjUgMi45ODgyODEgMC4w%0D%0AMjM0Mzc1IDIuOTM3NSAwLjAxNTYyNSBDIDIuODk0NTMxIDAuMDE1NjI1IDIuODU5Mzc1IDAuMDA3%0D%0AODEyNSAyLjgyODEyNSAwIEMgMi44MDQ2ODggLTAuMDE5NTMxMiAyLjc4OTA2MiAtMC4wMzUxNTYy%0D%0AIDIuNzgxMjUgLTAuMDQ2ODc1IEMgMi43Njk1MzEgLTAuMDY2NDA2MiAyLjc2NTYyNSAtMC4wODU5%0D%0AMzc1IDIuNzY1NjI1IC0wLjEwOTM3NSBMIDIuNzY1NjI1IC0xLjMxMjUgTCAwLjQwNjI1IC0xLjMx%0D%0AMjUgQyAwLjM3NSAtMS4zMTI1IDAuMzQ3NjU2IC0xLjMxMjUgMC4zMjgxMjUgLTEuMzEyNSBDIDAu%0D%0AMzA0Njg4IC0xLjMyMDMxMiAwLjI4NTE1NiAtMS4zMzU5MzggMC4yNjU2MjUgLTEuMzU5Mzc1IEMg%0D%0AMC4yNDIxODggLTEuMzkwNjI1IDAuMjI2NTYyIC0xLjQyNTc4MSAwLjIxODc1IC0xLjQ2ODc1IEMg%0D%0AMC4yMTg3NSAtMS41MDc4MTIgMC4yMTg3NSAtMS41NjY0MDYgMC4yMTg3NSAtMS42NDA2MjUgQyAw%0D%0ALjIxODc1IC0xLjY5MTQwNiAwLjIxODc1IC0xLjczODI4MSAwLjIxODc1IC0xLjc4MTI1IEMgMC4y%0D%0AMTg3NSAtMS44MzIwMzEgMC4yMjI2NTYgLTEuODc1IDAuMjM0Mzc1IC0xLjkwNjI1IEMgMC4yNDIx%0D%0AODggLTEuOTQ1MzEyIDAuMjUzOTA2IC0xLjk4NDM3NSAwLjI2NTYyNSAtMi4wMTU2MjUgQyAwLjI4%0D%0ANTE1NiAtMi4wNDY4NzUgMC4zMDQ2ODggLTIuMDgyMDMxIDAuMzI4MTI1IC0yLjEyNSBMIDIuMzc1%0D%0AIC01LjU3ODEyNSBDIDIuMzgyODEyIC01LjU5NzY1NiAyLjM5ODQzOCAtNS42MTcxODggMi40MjE4%0D%0ANzUgLTUuNjQwNjI1IEMgMi40NTMxMjUgLTUuNjYwMTU2IDIuNDg4MjgxIC01LjY3NTc4MSAyLjUz%0D%0AMTI1IC01LjY4NzUgQyAyLjU4MjAzMSAtNS42OTUzMTIgMi42NDA2MjUgLTUuNzAzMTI1IDIuNzAz%0D%0AMTI1IC01LjcwMzEyNSBDIDIuNzY1NjI1IC01LjcxMDkzOCAyLjg0Mzc1IC01LjcxODc1IDIuOTM3%0D%0ANSAtNS43MTg3NSBDIDMuMDMxMjUgLTUuNzE4NzUgMy4xMTMyODEgLTUuNzEwOTM4IDMuMTg3NSAt%0D%0ANS43MDMxMjUgQyAzLjI1NzgxMiAtNS42OTE0MDYgMy4zMTY0MDYgLTUuNjc5Njg4IDMuMzU5Mzc1%0D%0AIC01LjY3MTg3NSBDIDMuMzk4NDM4IC01LjY2MDE1NiAzLjQyOTY4OCAtNS42NDQ1MzEgMy40NTMx%0D%0AMjUgLTUuNjI1IEMgMy40ODQzNzUgLTUuNjAxNTYyIDMuNSAtNS41ODIwMzEgMy41IC01LjU2MjUg%0D%0ATCAzLjUgLTEuOTIxODc1IEwgNC4xNDA2MjUgLTEuOTIxODc1IEMgNC4xNzk2ODggLTEuOTIxODc1%0D%0AIDQuMjE4NzUgLTEuODk0NTMxIDQuMjUgLTEuODQzNzUgQyA0LjI4MTI1IC0xLjgwMDc4MSA0LjI5%0D%0ANjg3NSAtMS43MjY1NjIgNC4yOTY4NzUgLTEuNjI1IFogTSAyLjc2NTYyNSAtNS4wNDY4NzUgTCAy%0D%0ALjc1IC01LjA0Njg3NSBMIDAuOTA2MjUgLTEuOTIxODc1IEwgMi43NjU2MjUgLTEuOTIxODc1IFog%0D%0ATSAyLjc2NTYyNSAtNS4wNDY4NzUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2aXNp%0D%0AYmxlIiBpZD0iZ2x5cGgxLTUiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDQuMDMx%0D%0AMjUgLTEuODI4MTI1IEMgNC4wMzEyNSAtMS41MjM0MzggMy45NzY1NjIgLTEuMjUzOTA2IDMuODc1%0D%0AIC0xLjAxNTYyNSBDIDMuNzgxMjUgLTAuNzczNDM4IDMuNjQwNjI1IC0wLjU3MDMxMiAzLjQ1MzEy%0D%0ANSAtMC40MDYyNSBDIDMuMjY1NjI1IC0wLjI1IDMuMDM5MDYyIC0wLjEyODkwNiAyLjc4MTI1IC0w%0D%0ALjA0Njg3NSBDIDIuNTE5NTMxIDAuMDM1MTU2MiAyLjIyNjU2MiAwLjA3ODEyNSAxLjkwNjI1IDAu%0D%0AMDc4MTI1IEMgMS43MjY1NjIgMC4wNzgxMjUgMS41NjI1IDAuMDY2NDA2MiAxLjQwNjI1IDAuMDQ2%0D%0AODc1IEMgMS4yNSAwLjAyMzQzNzUgMS4xMDkzNzUgMCAwLjk4NDM3NSAtMC4wMzEyNSBDIDAuODU5%0D%0AMzc1IC0wLjA3MDMxMjUgMC43NTM5MDYgLTAuMTA5Mzc1IDAuNjcxODc1IC0wLjE0MDYyNSBDIDAu%0D%0ANTk3NjU2IC0wLjE3OTY4OCAwLjU1MDc4MSAtMC4yMTA5MzggMC41MzEyNSAtMC4yMzQzNzUgQyAw%0D%0ALjUwNzgxMiAtMC4yNTM5MDYgMC40OTIxODggLTAuMjczNDM4IDAuNDg0Mzc1IC0wLjI5Njg3NSBD%0D%0AIDAuNDcyNjU2IC0wLjMxNjQwNiAwLjQ2MDkzOCAtMC4zMzU5MzggMC40NTMxMjUgLTAuMzU5Mzc1%0D%0AIEMgMC40NTMxMjUgLTAuMzc4OTA2IDAuNDQ1MzEyIC0wLjQxMDE1NiAwLjQzNzUgLTAuNDUzMTI1%0D%0AIEMgMC40Mzc1IC0wLjQ5MjE4OCAwLjQzNzUgLTAuNTM5MDYyIDAuNDM3NSAtMC41OTM3NSBDIDAu%0D%0ANDM3NSAtMC42MzI4MTIgMC40Mzc1IC0wLjY3NTc4MSAwLjQzNzUgLTAuNzE4NzUgQyAwLjQ0NTMx%0D%0AMiAtMC43NTc4MTIgMC40NTcwMzEgLTAuNzg5MDYyIDAuNDY4NzUgLTAuODEyNSBDIDAuNDg4Mjgx%0D%0AIC0wLjg0Mzc1IDAuNTAzOTA2IC0wLjg2MzI4MSAwLjUxNTYyNSAtMC44NzUgQyAwLjUzNTE1NiAt%0D%0AMC44ODI4MTIgMC41NTQ2ODggLTAuODkwNjI1IDAuNTc4MTI1IC0wLjg5MDYyNSBDIDAuNTk3NjU2%0D%0AIC0wLjg5MDYyNSAwLjY0MDYyNSAtMC44NjcxODggMC43MDMxMjUgLTAuODI4MTI1IEMgMC43NjU2%0D%0AMjUgLTAuNzk2ODc1IDAuODQ3NjU2IC0wLjc1NzgxMiAwLjk1MzEyNSAtMC43MTg3NSBDIDEuMDU0%0D%0ANjg4IC0wLjY3NTc4MSAxLjE4NzUgLTAuNjMyODEyIDEuMzQzNzUgLTAuNTkzNzUgQyAxLjUgLTAu%0D%0ANTYyNSAxLjY3OTY4OCAtMC41NDY4NzUgMS44OTA2MjUgLTAuNTQ2ODc1IEMgMi4wODU5MzggLTAu%0D%0ANTQ2ODc1IDIuMjY1NjI1IC0wLjU2NjQwNiAyLjQyMTg3NSAtMC42MDkzNzUgQyAyLjU4NTkzOCAt%0D%0AMC42NjAxNTYgMi43MjY1NjIgLTAuNzM0Mzc1IDIuODQzNzUgLTAuODI4MTI1IEMgMi45Njg3NSAt%0D%0AMC45Mjk2ODggMy4wNjI1IC0xLjA1NDY4OCAzLjEyNSAtMS4yMDMxMjUgQyAzLjE5NTMxMiAtMS4z%0D%0ANTkzNzUgMy4yMzQzNzUgLTEuNTQ2ODc1IDMuMjM0Mzc1IC0xLjc2NTYyNSBDIDMuMjM0Mzc1IC0x%0D%0ALjk0MTQwNiAzLjIwMzEyNSAtMi4wOTc2NTYgMy4xNDA2MjUgLTIuMjM0Mzc1IEMgMy4wODU5Mzgg%0D%0ALTIuMzc4OTA2IDMgLTIuNDkyMTg4IDIuODc1IC0yLjU3ODEyNSBDIDIuNzU3ODEyIC0yLjY3MTg3%0D%0ANSAyLjYwOTM3NSAtMi43MzgyODEgMi40MjE4NzUgLTIuNzgxMjUgQyAyLjI0MjE4OCAtMi44MzIw%0D%0AMzEgMi4wMzEyNSAtMi44NTkzNzUgMS43ODEyNSAtMi44NTkzNzUgQyAxLjYwMTU2MiAtMi44NTkz%0D%0ANzUgMS40NDUzMTIgLTIuODQ3NjU2IDEuMzEyNSAtMi44MjgxMjUgQyAxLjE4NzUgLTIuODE2NDA2%0D%0AIDEuMDcwMzEyIC0yLjgxMjUgMC45Njg3NSAtMi44MTI1IEMgMC44ODI4MTIgLTIuODEyNSAwLjgy%0D%0AMDMxMiAtMi44MjgxMjUgMC43ODEyNSAtMi44NTkzNzUgQyAwLjc1IC0yLjg5ODQzOCAwLjczNDM3%0D%0ANSAtMi45NzI2NTYgMC43MzQzNzUgLTMuMDc4MTI1IEwgMC43MzQzNzUgLTUuNDA2MjUgQyAwLjcz%0D%0ANDM3NSAtNS41IDAuNzUzOTA2IC01LjU2NjQwNiAwLjc5Njg3NSAtNS42MDkzNzUgQyAwLjg0NzY1%0D%0ANiAtNS42NjAxNTYgMC45MTQwNjIgLTUuNjg3NSAxIC01LjY4NzUgTCAzLjUxNTYyNSAtNS42ODc1%0D%0AIEMgMy41MzUxNTYgLTUuNjg3NSAzLjU1NDY4OCAtNS42NzU3ODEgMy41NzgxMjUgLTUuNjU2MjUg%0D%0AQyAzLjU5NzY1NiAtNS42NDQ1MzEgMy42MTMyODEgLTUuNjI1IDMuNjI1IC01LjU5Mzc1IEMgMy42%0D%0ANDQ1MzEgLTUuNTcwMzEyIDMuNjYwMTU2IC01LjUzOTA2MiAzLjY3MTg3NSAtNS41IEMgMy42Nzk2%0D%0AODggLTUuNDU3MDMxIDMuNjg3NSAtNS40MTAxNTYgMy42ODc1IC01LjM1OTM3NSBDIDMuNjg3NSAt%0D%0ANS4yNTM5MDYgMy42NzE4NzUgLTUuMTcxODc1IDMuNjQwNjI1IC01LjEwOTM3NSBDIDMuNjA5Mzc1%0D%0AIC01LjA1NDY4OCAzLjU2NjQwNiAtNS4wMzEyNSAzLjUxNTYyNSAtNS4wMzEyNSBMIDEuMzkwNjI1%0D%0AIC01LjAzMTI1IEwgMS4zOTA2MjUgLTMuNDM3NSBDIDEuNDkyMTg4IC0zLjQ0NTMxMiAxLjU5NzY1%0D%0ANiAtMy40NTMxMjUgMS43MDMxMjUgLTMuNDUzMTI1IEMgMS44MTY0MDYgLTMuNDYwOTM4IDEuOTQx%0D%0ANDA2IC0zLjQ2ODc1IDIuMDc4MTI1IC0zLjQ2ODc1IEMgMi4zOTg0MzggLTMuNDY4NzUgMi42Nzk2%0D%0AODggLTMuNDI1NzgxIDIuOTIxODc1IC0zLjM0Mzc1IEMgMy4xNzE4NzUgLTMuMjY5NTMxIDMuMzc1%0D%0AIC0zLjE2MDE1NiAzLjUzMTI1IC0zLjAxNTYyNSBDIDMuNjk1MzEyIC0yLjg2NzE4OCAzLjgyMDMx%0D%0AMiAtMi42OTUzMTIgMy45MDYyNSAtMi41IEMgMy45ODgyODEgLTIuMzAwNzgxIDQuMDMxMjUgLTIu%0D%0AMDc4MTI1IDQuMDMxMjUgLTEuODI4MTI1IFogTSA0LjAzMTI1IC0xLjgyODEyNSAiLz4KPC9zeW1i%0D%0Ab2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlwaDEtNiI+CjxwYXRoIHN0eWxl%0D%0APSJzdHJva2U6bm9uZTsiIGQ9Ik0gNC4yMDMxMjUgLTEuODQzNzUgQyA0LjIwMzEyNSAtMS41OTM3%0D%0ANSA0LjE2NDA2MiAtMS4zNDc2NTYgNC4wOTM3NSAtMS4xMDkzNzUgQyA0LjAxOTUzMSAtMC44Nzg5%0D%0AMDYgMy44OTg0MzggLTAuNjc1NzgxIDMuNzM0Mzc1IC0wLjUgQyAzLjU3ODEyNSAtMC4zMjAzMTIg%0D%0AMy4zNzUgLTAuMTc5Njg4IDMuMTI1IC0wLjA3ODEyNSBDIDIuODgyODEyIDAuMDIzNDM3NSAyLjYw%0D%0AMTU2MiAwLjA3ODEyNSAyLjI4MTI1IDAuMDc4MTI1IEMgMi4wMzkwNjIgMC4wNzgxMjUgMS44Mjgx%0D%0AMjUgMC4wNTA3ODEyIDEuNjQwNjI1IDAgQyAxLjQ2MDkzOCAtMC4wNjI1IDEuMzA0Njg4IC0wLjE0%0D%0ANDUzMSAxLjE3MTg3NSAtMC4yNSBDIDEuMDM1MTU2IC0wLjM2MzI4MSAwLjkyMTg3NSAtMC41IDAu%0D%0AODI4MTI1IC0wLjY1NjI1IEMgMC43NDIxODggLTAuODEyNSAwLjY3NTc4MSAtMC45ODgyODEgMC42%0D%0AMjUgLTEuMTg3NSBDIDAuNTcwMzEyIC0xLjM5NDUzMSAwLjUzMTI1IC0xLjYxNzE4OCAwLjUgLTEu%0D%0AODU5Mzc1IEMgMC40NzY1NjIgLTIuMDk3NjU2IDAuNDY4NzUgLTIuMzUxNTYyIDAuNDY4NzUgLTIu%0D%0ANjI1IEMgMC40Njg3NSAtMi44NzUgMC40NzY1NjIgLTMuMTI1IDAuNSAtMy4zNzUgQyAwLjUzMTI1%0D%0AIC0zLjYyNSAwLjU3ODEyNSAtMy44NjMyODEgMC42NDA2MjUgLTQuMDkzNzUgQyAwLjcxMDkzOCAt%0D%0ANC4zMjAzMTIgMC44MDQ2ODggLTQuNTM1MTU2IDAuOTIxODc1IC00LjczNDM3NSBDIDEuMDM1MTU2%0D%0AIC00Ljk0MTQwNiAxLjE3OTY4OCAtNS4xMTcxODggMS4zNTkzNzUgLTUuMjY1NjI1IEMgMS41MzUx%0D%0ANTYgLTUuNDIxODc1IDEuNzQyMTg4IC01LjUzOTA2MiAxLjk4NDM3NSAtNS42MjUgQyAyLjIyMjY1%0D%0ANiAtNS43MTg3NSAyLjUwMzkwNiAtNS43NjU2MjUgMi44MjgxMjUgLTUuNzY1NjI1IEMgMi45NDE0%0D%0AMDYgLTUuNzY1NjI1IDMuMDUwNzgxIC01Ljc1MzkwNiAzLjE1NjI1IC01LjczNDM3NSBDIDMuMjY5%0D%0ANTMxIC01LjcyMjY1NiAzLjM3NSAtNS43MDcwMzEgMy40Njg3NSAtNS42ODc1IEMgMy41NjI1IC01%0D%0ALjY2NDA2MiAzLjY0MDYyNSAtNS42NDA2MjUgMy43MDMxMjUgLTUuNjA5Mzc1IEMgMy43NzM0Mzgg%0D%0ALTUuNTg1OTM4IDMuODIwMzEyIC01LjU2NjQwNiAzLjg0Mzc1IC01LjU0Njg3NSBDIDMuODYzMjgx%0D%0AIC01LjUzNTE1NiAzLjg3NSAtNS41MTk1MzEgMy44NzUgLTUuNSBDIDMuODgyODEyIC01LjQ3NjU2%0D%0AMiAzLjg5NDUzMSAtNS40NTcwMzEgMy45MDYyNSAtNS40Mzc1IEMgMy45MTQwNjIgLTUuNDE0MDYy%0D%0AIDMuOTIxODc1IC01LjM5MDYyNSAzLjkyMTg3NSAtNS4zNTkzNzUgQyAzLjkyMTg3NSAtNS4zMzU5%0D%0AMzggMy45MjE4NzUgLTUuMzA0Njg4IDMuOTIxODc1IC01LjI2NTYyNSBDIDMuOTIxODc1IC01LjIx%0D%0AMDkzOCAzLjkyMTg3NSAtNS4xNjQwNjIgMy45MjE4NzUgLTUuMTI1IEMgMy45MjE4NzUgLTUuMDkz%0D%0ANzUgMy45MTQwNjIgLTUuMDYyNSAzLjkwNjI1IC01LjAzMTI1IEMgMy44OTQ1MzEgLTUuMDA3ODEy%0D%0AIDMuODc4OTA2IC00Ljk5MjE4OCAzLjg1OTM3NSAtNC45ODQzNzUgQyAzLjg0NzY1NiAtNC45NzI2%0D%0ANTYgMy44MjgxMjUgLTQuOTY4NzUgMy43OTY4NzUgLTQuOTY4NzUgQyAzLjc2NTYyNSAtNC45Njg3%0D%0ANSAzLjcyMjY1NiAtNC45NzY1NjIgMy42NzE4NzUgLTUgQyAzLjYxNzE4OCAtNS4wMTk1MzEgMy41%0D%0ANTA3ODEgLTUuMDM5MDYyIDMuNDY4NzUgLTUuMDYyNSBDIDMuMzgyODEyIC01LjA5Mzc1IDMuMjg1%0D%0AMTU2IC01LjExMzI4MSAzLjE3MTg3NSAtNS4xMjUgQyAzLjA2NjQwNiAtNS4xNDQ1MzEgMi45Mzc1%0D%0AIC01LjE1NjI1IDIuNzgxMjUgLTUuMTU2MjUgQyAyLjUwNzgxMiAtNS4xNTYyNSAyLjI3MzQzOCAt%0D%0ANS4wOTc2NTYgMi4wNzgxMjUgLTQuOTg0Mzc1IEMgMS44Nzg5MDYgLTQuODc4OTA2IDEuNzE4NzUg%0D%0ALTQuNzM0Mzc1IDEuNTkzNzUgLTQuNTQ2ODc1IEMgMS40NzY1NjIgLTQuMzU5Mzc1IDEuMzkwNjI1%0D%0AIC00LjE0MDYyNSAxLjMyODEyNSAtMy44OTA2MjUgQyAxLjI2NTYyNSAtMy42NDg0MzggMS4yMjY1%0D%0ANjIgLTMuMzk0NTMxIDEuMjE4NzUgLTMuMTI1IEMgMS4zMDA3ODEgLTMuMTY0MDYyIDEuMzkwNjI1%0D%0AIC0zLjIwNzAzMSAxLjQ4NDM3NSAtMy4yNSBDIDEuNTc4MTI1IC0zLjI4OTA2MiAxLjY3NTc4MSAt%0D%0AMy4zMjgxMjUgMS43ODEyNSAtMy4zNTkzNzUgQyAxLjg5NDUzMSAtMy4zOTg0MzggMi4wMTU2MjUg%0D%0ALTMuNDI5Njg4IDIuMTQwNjI1IC0zLjQ1MzEyNSBDIDIuMjczNDM4IC0zLjQ3MjY1NiAyLjQxMDE1%0D%0ANiAtMy40ODQzNzUgMi41NDY4NzUgLTMuNDg0Mzc1IEMgMi44NTkzNzUgLTMuNDg0Mzc1IDMuMTE3%0D%0AMTg4IC0zLjQ0MTQwNiAzLjMyODEyNSAtMy4zNTkzNzUgQyAzLjUzNTE1NiAtMy4yNzM0MzggMy43%0D%0AMDMxMjUgLTMuMTYwMTU2IDMuODI4MTI1IC0zLjAxNTYyNSBDIDMuOTYwOTM4IC0yLjg2NzE4OCA0%0D%0ALjA1NDY4OCAtMi42OTE0MDYgNC4xMDkzNzUgLTIuNDg0Mzc1IEMgNC4xNzE4NzUgLTIuMjg1MTU2%0D%0AIDQuMjAzMTI1IC0yLjA3MDMxMiA0LjIwMzEyNSAtMS44NDM3NSBaIE0gMy40Mzc1IC0xLjc2NTYy%0D%0ANSBDIDMuNDM3NSAtMS45NDE0MDYgMy40MjE4NzUgLTIuMDk3NjU2IDMuMzkwNjI1IC0yLjIzNDM3%0D%0ANSBDIDMuMzU5Mzc1IC0yLjM2NzE4OCAzLjMwMDc4MSAtMi40ODQzNzUgMy4yMTg3NSAtMi41Nzgx%0D%0AMjUgQyAzLjE0NDUzMSAtMi42Nzk2ODggMy4wMzkwNjIgLTIuNzU3ODEyIDIuOTA2MjUgLTIuODEy%0D%0ANSBDIDIuNzY5NTMxIC0yLjg2MzI4MSAyLjYwOTM3NSAtMi44OTA2MjUgMi40MjE4NzUgLTIuODkw%0D%0ANjI1IEMgMi4zMDQ2ODggLTIuODkwNjI1IDIuMTkxNDA2IC0yLjg3ODkwNiAyLjA3ODEyNSAtMi44%0D%0ANTkzNzUgQyAxLjk3MjY1NiAtMi44MzU5MzggMS44NjcxODggLTIuODEyNSAxLjc2NTYyNSAtMi43%0D%0AODEyNSBDIDEuNjcxODc1IC0yLjc1IDEuNTc4MTI1IC0yLjcwNzAzMSAxLjQ4NDM3NSAtMi42NTYy%0D%0ANSBDIDEuMzkwNjI1IC0yLjYxMzI4MSAxLjMwNDY4OCAtMi41NjY0MDYgMS4yMzQzNzUgLTIuNTE1%0D%0ANjI1IEMgMS4yMzQzNzUgLTIuMTQwNjI1IDEuMjUzOTA2IC0xLjgyMDMxMiAxLjI5Njg3NSAtMS41%0D%0ANjI1IEMgMS4zNDc2NTYgLTEuMzEyNSAxLjQyMTg3NSAtMS4xMDkzNzUgMS41MTU2MjUgLTAuOTUz%0D%0AMTI1IEMgMS42MDkzNzUgLTAuNzk2ODc1IDEuNzIyNjU2IC0wLjY4NzUgMS44NTkzNzUgLTAuNjI1%0D%0AIEMgMS45OTIxODggLTAuNTYyNSAyLjE1NjI1IC0wLjUzMTI1IDIuMzQzNzUgLTAuNTMxMjUgQyAy%0D%0ALjUzMTI1IC0wLjUzMTI1IDIuNjkxNDA2IC0wLjU2MjUgMi44MjgxMjUgLTAuNjI1IEMgMi45NzI2%0D%0ANTYgLTAuNjk1MzEyIDMuMDg1OTM4IC0wLjc4OTA2MiAzLjE3MTg3NSAtMC45MDYyNSBDIDMuMjY1%0D%0ANjI1IC0xLjAzMTI1IDMuMzMyMDMxIC0xLjE2NDA2MiAzLjM3NSAtMS4zMTI1IEMgMy40MTQwNjIg%0D%0ALTEuNDU3MDMxIDMuNDM3NSAtMS42MDkzNzUgMy40Mzc1IC0xLjc2NTYyNSBaIE0gMy40Mzc1IC0x%0D%0ALjc2NTYyNSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlw%0D%0AaDEtNyI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0gNC4xNDA2MjUgLTUuMzU5Mzc1%0D%0AIEMgNC4xNDA2MjUgLTUuMzA0Njg4IDQuMTMyODEyIC01LjI1NzgxMiA0LjEyNSAtNS4yMTg3NSBD%0D%0AIDQuMTI1IC01LjE3NTc4MSA0LjExNzE4OCAtNS4xMzI4MTIgNC4xMDkzNzUgLTUuMDkzNzUgQyA0%0D%0ALjEwOTM3NSAtNS4wNTA3ODEgNC4wOTc2NTYgLTUuMDA3ODEyIDQuMDc4MTI1IC00Ljk2ODc1IEMg%0D%0ANC4wNjY0MDYgLTQuOTM3NSA0LjA1NDY4OCAtNC44OTg0MzggNC4wNDY4NzUgLTQuODU5Mzc1IEwg%0D%0AMiAtMC4xNDA2MjUgQyAxLjk3NjU2MiAtMC4xMDkzNzUgMS45NTcwMzEgLTAuMDgyMDMxMiAxLjkz%0D%0ANzUgLTAuMDYyNSBDIDEuOTE0MDYyIC0wLjAzOTA2MjUgMS44OTA2MjUgLTAuMDE5NTMxMiAxLjg1%0D%0AOTM3NSAwIEMgMS44MjgxMjUgMC4wMDc4MTI1IDEuNzgxMjUgMC4wMTU2MjUgMS43MTg3NSAwLjAx%0D%0ANTYyNSBDIDEuNjY0MDYyIDAuMDIzNDM3NSAxLjYwMTU2MiAwLjAzMTI1IDEuNTMxMjUgMC4wMzEy%0D%0ANSBDIDEuNDM3NSAwLjAzMTI1IDEuMzU5Mzc1IDAuMDIzNDM3NSAxLjI5Njg3NSAwLjAxNTYyNSBD%0D%0AIDEuMjQyMTg4IDAuMDAzOTA2MjUgMS4yMDMxMjUgLTAuMDA3ODEyNSAxLjE3MTg3NSAtMC4wMzEy%0D%0ANSBDIDEuMTQ4NDM4IC0wLjA1MDc4MTIgMS4xNDA2MjUgLTAuMDcwMzEyNSAxLjE0MDYyNSAtMC4w%0D%0AOTM3NSBDIDEuMTQ4NDM4IC0wLjEyNSAxLjE2MDE1NiAtMC4xNjAxNTYgMS4xNzE4NzUgLTAuMjAz%0D%0AMTI1IEwgMy4zMjgxMjUgLTUuMDE1NjI1IEwgMC42MDkzNzUgLTUuMDE1NjI1IEMgMC41NDY4NzUg%0D%0ALTUuMDE1NjI1IDAuNSAtNS4wNDY4NzUgMC40Njg3NSAtNS4xMDkzNzUgQyAwLjQ0NTMxMiAtNS4x%0D%0ANzE4NzUgMC40Mzc1IC01LjI1MzkwNiAwLjQzNzUgLTUuMzU5Mzc1IEMgMC40Mzc1IC01LjQxMDE1%0D%0ANiAwLjQzNzUgLTUuNDU3MDMxIDAuNDM3NSAtNS41IEMgMC40NDUzMTIgLTUuNTM5MDYyIDAuNDU3%0D%0AMDMxIC01LjU3MDMxMiAwLjQ2ODc1IC01LjU5Mzc1IEMgMC40ODgyODEgLTUuNjI1IDAuNTA3ODEy%0D%0AIC01LjY0NDUzMSAwLjUzMTI1IC01LjY1NjI1IEMgMC41NTA3ODEgLTUuNjc1NzgxIDAuNTc4MTI1%0D%0AIC01LjY4NzUgMC42MDkzNzUgLTUuNjg3NSBMIDMuOTIxODc1IC01LjY4NzUgQyAzLjk2MDkzOCAt%0D%0ANS42ODc1IDMuOTkyMTg4IC01LjY3OTY4OCA0LjAxNTYyNSAtNS42NzE4NzUgQyA0LjA0Njg3NSAt%0D%0ANS42NjAxNTYgNC4wNzAzMTIgLTUuNjQwNjI1IDQuMDkzNzUgLTUuNjA5Mzc1IEMgNC4xMTMyODEg%0D%0ALTUuNTg1OTM4IDQuMTI1IC01LjU1NDY4OCA0LjEyNSAtNS41MTU2MjUgQyA0LjEzMjgxMiAtNS40%0D%0ANzI2NTYgNC4xNDA2MjUgLTUuNDIxODc1IDQuMTQwNjI1IC01LjM1OTM3NSBaIE0gNC4xNDA2MjUg%0D%0ALTUuMzU5Mzc1ICIvPgo8L3N5bWJvbD4KPHN5bWJvbCBvdmVyZmxvdz0idmlzaWJsZSIgaWQ9Imds%0D%0AeXBoMS04Ij4KPHBhdGggc3R5bGU9InN0cm9rZTpub25lOyIgZD0iTSA0LjE4NzUgLTEuNDUzMTI1%0D%0AIEMgNC4xODc1IC0xLjIxMDkzOCA0LjE0MDYyNSAtMC45OTIxODggNC4wNDY4NzUgLTAuNzk2ODc1%0D%0AIEMgMy45NjA5MzggLTAuNjA5Mzc1IDMuODM1OTM4IC0wLjQ0NTMxMiAzLjY3MTg3NSAtMC4zMTI1%0D%0AIEMgMy41MTU2MjUgLTAuMTg3NSAzLjMxMjUgLTAuMDg1OTM3NSAzLjA2MjUgLTAuMDE1NjI1IEMg%0D%0AMi44MjAzMTIgMC4wNDY4NzUgMi41NDY4NzUgMC4wNzgxMjUgMi4yMzQzNzUgMC4wNzgxMjUgQyAx%0D%0ALjk0MTQwNiAwLjA3ODEyNSAxLjY3NTc4MSAwLjA1MDc4MTIgMS40Mzc1IDAgQyAxLjIwNzAzMSAt%0D%0AMC4wNjI1IDEuMDA3ODEyIC0wLjE1NjI1IDAuODQzNzUgLTAuMjgxMjUgQyAwLjY4NzUgLTAuNDA2%0D%0AMjUgMC41NjY0MDYgLTAuNTUwNzgxIDAuNDg0Mzc1IC0wLjcxODc1IEMgMC4zOTg0MzggLTAuODk0%0D%0ANTMxIDAuMzU5Mzc1IC0xLjA5NzY1NiAwLjM1OTM3NSAtMS4zMjgxMjUgQyAwLjM1OTM3NSAtMS41%0D%0AMDM5MDYgMC4zODI4MTIgLTEuNjY0MDYyIDAuNDM3NSAtMS44MTI1IEMgMC41IC0xLjk2ODc1IDAu%0D%0ANTgyMDMxIC0yLjEwOTM3NSAwLjY4NzUgLTIuMjM0Mzc1IEMgMC44MDA3ODEgLTIuMzU5Mzc1IDAu%0D%0AOTM3NSAtMi40NzI2NTYgMS4wOTM3NSAtMi41NzgxMjUgQyAxLjI1NzgxMiAtMi42OTE0MDYgMS40%0D%0ANDUzMTIgLTIuODAwNzgxIDEuNjU2MjUgLTIuOTA2MjUgQyAxLjQ3NjU2MiAtMi45ODgyODEgMS4z%0D%0AMjAzMTIgLTMuMDgyMDMxIDEuMTg3NSAtMy4xODc1IEMgMS4wNTA3ODEgLTMuMjg5MDYyIDAuOTM3%0D%0ANSAtMy4zOTg0MzggMC44NDM3NSAtMy41MTU2MjUgQyAwLjc1IC0zLjY0MDYyNSAwLjY3NTc4MSAt%0D%0AMy43NjU2MjUgMC42MjUgLTMuODkwNjI1IEMgMC41ODIwMzEgLTQuMDIzNDM4IDAuNTYyNSAtNC4x%0D%0ANzU3ODEgMC41NjI1IC00LjM0Mzc1IEMgMC41NjI1IC00LjUzOTA2MiAwLjU5NzY1NiAtNC43MjI2%0D%0ANTYgMC42NzE4NzUgLTQuODkwNjI1IEMgMC43NDIxODggLTUuMDY2NDA2IDAuODQ3NjU2IC01LjIx%0D%0AODc1IDAuOTg0Mzc1IC01LjM0Mzc1IEMgMS4xMjg5MDYgLTUuNDc2NTYyIDEuMzEyNSAtNS41ODIw%0D%0AMzEgMS41MzEyNSAtNS42NTYyNSBDIDEuNzU3ODEyIC01LjcyNjU2MiAyLjAyMzQzOCAtNS43NjU2%0D%0AMjUgMi4zMjgxMjUgLTUuNzY1NjI1IEMgMi42MTcxODggLTUuNzY1NjI1IDIuODY3MTg4IC01Ljcy%0D%0ANjU2MiAzLjA3ODEyNSAtNS42NTYyNSBDIDMuMjg1MTU2IC01LjU5Mzc1IDMuNDU3MDMxIC01LjUg%0D%0AMy41OTM3NSAtNS4zNzUgQyAzLjcyNjU2MiAtNS4yNTc4MTIgMy44MjgxMjUgLTUuMTE3MTg4IDMu%0D%0AODkwNjI1IC00Ljk1MzEyNSBDIDMuOTUzMTI1IC00Ljc5Njg3NSAzLjk4NDM3NSAtNC42Mjg5MDYg%0D%0AMy45ODQzNzUgLTQuNDUzMTI1IEMgMy45ODQzNzUgLTQuMjk2ODc1IDMuOTU3MDMxIC00LjE0ODQz%0D%0AOCAzLjkwNjI1IC00LjAxNTYyNSBDIDMuODYzMjgxIC0zLjg3ODkwNiAzLjc5Njg3NSAtMy43NSAz%0D%0ALjcwMzEyNSAtMy42MjUgQyAzLjYwOTM3NSAtMy41IDMuNDkyMTg4IC0zLjM4MjgxMiAzLjM1OTM3%0D%0ANSAtMy4yODEyNSBDIDMuMjIyNjU2IC0zLjE3NTc4MSAzLjA2NjQwNiAtMy4wNzgxMjUgMi44OTA2%0D%0AMjUgLTIuOTg0Mzc1IEMgMy4wOTc2NTYgLTIuODc4OTA2IDMuMjgxMjUgLTIuNzY5NTMxIDMuNDM3%0D%0ANSAtMi42NTYyNSBDIDMuNjAxNTYyIC0yLjU1MDc4MSAzLjczODI4MSAtMi40Mzc1IDMuODQzNzUg%0D%0ALTIuMzEyNSBDIDMuOTU3MDMxIC0yLjE4NzUgNC4wMzkwNjIgLTIuMDUwNzgxIDQuMDkzNzUgLTEu%0D%0AOTA2MjUgQyA0LjE1NjI1IC0xLjc2OTUzMSA0LjE4NzUgLTEuNjE3MTg4IDQuMTg3NSAtMS40NTMx%0D%0AMjUgWiBNIDMuMjUgLTQuMzc1IEMgMy4yNSAtNC41IDMuMjIyNjU2IC00LjYwOTM3NSAzLjE3MTg3%0D%0ANSAtNC43MDMxMjUgQyAzLjEyODkwNiAtNC44MDQ2ODggMy4wNjY0MDYgLTQuODk0NTMxIDIuOTg0%0D%0AMzc1IC00Ljk2ODc1IEMgMi45MTAxNTYgLTUuMDM5MDYyIDIuODEyNSAtNS4wOTM3NSAyLjY4NzUg%0D%0ALTUuMTI1IEMgMi41NjI1IC01LjE2NDA2MiAyLjQyMTg3NSAtNS4xODc1IDIuMjY1NjI1IC01LjE4%0D%0ANzUgQyAxLjk1MzEyNSAtNS4xODc1IDEuNzEwOTM4IC01LjExMzI4MSAxLjU0Njg3NSAtNC45Njg3%0D%0ANSBDIDEuMzkwNjI1IC00LjgyMDMxMiAxLjMxMjUgLTQuNjI1IDEuMzEyNSAtNC4zNzUgQyAxLjMx%0D%0AMjUgLTQuMjU3ODEyIDEuMzI4MTI1IC00LjE0ODQzOCAxLjM1OTM3NSAtNC4wNDY4NzUgQyAxLjM5%0D%0AODQzOCAtMy45NTMxMjUgMS40NjA5MzggLTMuODU5Mzc1IDEuNTQ2ODc1IC0zLjc2NTYyNSBDIDEu%0D%0ANjI4OTA2IC0zLjY3OTY4OCAxLjczNDM3NSAtMy41OTc2NTYgMS44NTkzNzUgLTMuNTE1NjI1IEMg%0D%0AMS45OTIxODggLTMuNDI5Njg4IDIuMTQ4NDM4IC0zLjM0NzY1NiAyLjMyODEyNSAtMy4yNjU2MjUg%0D%0AQyAyLjYxNzE4OCAtMy40MTAxNTYgMi44NDM3NSAtMy41NzgxMjUgMyAtMy43NjU2MjUgQyAzLjE2%0D%0ANDA2MiAtMy45NTMxMjUgMy4yNSAtNC4xNTYyNSAzLjI1IC00LjM3NSBaIE0gMy40MjE4NzUgLTEu%0D%0AMzc1IEMgMy40MjE4NzUgLTEuNTA3ODEyIDMuMzk0NTMxIC0xLjYyODkwNiAzLjM0Mzc1IC0xLjcz%0D%0ANDM3NSBDIDMuMzAwNzgxIC0xLjg0NzY1NiAzLjIyNjU2MiAtMS45NTMxMjUgMy4xMjUgLTIuMDQ2%0D%0AODc1IEMgMy4wMTk1MzEgLTIuMTQ4NDM4IDIuODkwNjI1IC0yLjI0MjE4OCAyLjczNDM3NSAtMi4z%0D%0AMjgxMjUgQyAyLjU4NTkzOCAtMi40MjE4NzUgMi40MTQwNjIgLTIuNTE1NjI1IDIuMjE4NzUgLTIu%0D%0ANjA5Mzc1IEMgMi4wMzkwNjIgLTIuNTE1NjI1IDEuODc4OTA2IC0yLjQyMTg3NSAxLjczNDM3NSAt%0D%0AMi4zMjgxMjUgQyAxLjU5NzY1NiAtMi4yNDIxODggMS40ODQzNzUgLTIuMTQ4NDM4IDEuMzkwNjI1%0D%0AIC0yLjA0Njg3NSBDIDEuMzA0Njg4IC0xLjk1MzEyNSAxLjIzODI4MSAtMS44NTE1NjIgMS4xODc1%0D%0AIC0xLjc1IEMgMS4xNDQ1MzEgLTEuNjQ0NTMxIDEuMTI1IC0xLjUyMzQzOCAxLjEyNSAtMS4zOTA2%0D%0AMjUgQyAxLjEyNSAtMS4xMDkzNzUgMS4yMTg3NSAtMC44OTA2MjUgMS40MDYyNSAtMC43MzQzNzUg%0D%0AQyAxLjYwMTU2MiAtMC41NzgxMjUgMS44OTg0MzggLTAuNSAyLjI5Njg3NSAtMC41IEMgMi42NzE4%0D%0ANzUgLTAuNSAyLjk1MzEyNSAtMC41NzgxMjUgMy4xNDA2MjUgLTAuNzM0Mzc1IEMgMy4zMjgxMjUg%0D%0ALTAuODkwNjI1IDMuNDIxODc1IC0xLjEwMTU2MiAzLjQyMTg3NSAtMS4zNzUgWiBNIDMuNDIxODc1%0D%0AIC0xLjM3NSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlw%0D%0AaDEtOSI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0gNC4xMDkzNzUgLTMuMDYyNSBD%0D%0AIDQuMTA5Mzc1IC0yLjgyMDMxMiA0LjA5Mzc1IC0yLjU3MDMxMiA0LjA2MjUgLTIuMzEyNSBDIDQu%0D%0AMDMxMjUgLTIuMDYyNSAzLjk3NjU2MiAtMS44MjAzMTIgMy45MDYyNSAtMS41OTM3NSBDIDMuODQz%0D%0ANzUgLTEuMzYzMjgxIDMuNzUgLTEuMTQ0NTMxIDMuNjI1IC0wLjkzNzUgQyAzLjUwNzgxMiAtMC43%0D%0AMzgyODEgMy4zNTkzNzUgLTAuNTYyNSAzLjE3MTg3NSAtMC40MDYyNSBDIDIuOTkyMTg4IC0wLjI1%0D%0AIDIuNzgxMjUgLTAuMTI4OTA2IDIuNTMxMjUgLTAuMDQ2ODc1IEMgMi4yODkwNjIgMC4wMzUxNTYy%0D%0AIDIuMDAzOTA2IDAuMDc4MTI1IDEuNjcxODc1IDAuMDc4MTI1IEMgMS41NDY4NzUgMC4wNzgxMjUg%0D%0AMS40MjE4NzUgMC4wNjY0MDYyIDEuMjk2ODc1IDAuMDQ2ODc1IEMgMS4xNzk2ODggMC4wMzUxNTYy%0D%0AIDEuMDc4MTI1IDAuMDE5NTMxMiAwLjk4NDM3NSAwIEMgMC44OTA2MjUgLTAuMDE5NTMxMiAwLjgx%0D%0AMjUgLTAuMDM5MDYyNSAwLjc1IC0wLjA2MjUgQyAwLjY4NzUgLTAuMDkzNzUgMC42NDA2MjUgLTAu%0D%0AMTE3MTg4IDAuNjA5Mzc1IC0wLjE0MDYyNSBDIDAuNTc4MTI1IC0wLjE3MTg3NSAwLjU1MDc4MSAt%0D%0AMC4yMDcwMzEgMC41MzEyNSAtMC4yNSBDIDAuNTE5NTMxIC0wLjMwMDc4MSAwLjUxNTYyNSAtMC4z%0D%0ANjMyODEgMC41MTU2MjUgLTAuNDM3NSBDIDAuNTE1NjI1IC0wLjUwNzgxMiAwLjUxNTYyNSAtMC41%0D%0ANjI1IDAuNTE1NjI1IC0wLjU5Mzc1IEMgMC41MjM0MzggLTAuNjMyODEyIDAuNTM1MTU2IC0wLjY2%0D%0ANDA2MiAwLjU0Njg3NSAtMC42ODc1IEMgMC41NTQ2ODggLTAuNzA3MDMxIDAuNTY2NDA2IC0wLjcx%0D%0AODc1IDAuNTc4MTI1IC0wLjcxODc1IEMgMC41OTc2NTYgLTAuNzI2NTYyIDAuNjE3MTg4IC0wLjcz%0D%0ANDM3NSAwLjY0MDYyNSAtMC43MzQzNzUgQyAwLjY3MTg3NSAtMC43MzQzNzUgMC43MTg3NSAtMC43%0D%0AMjI2NTYgMC43ODEyNSAtMC43MDMxMjUgQyAwLjg0Mzc1IC0wLjY3OTY4OCAwLjkxNDA2MiAtMC42%0D%0ANTYyNSAxIC0wLjYyNSBDIDEuMDkzNzUgLTAuNjAxNTYyIDEuMTk1MzEyIC0wLjU4MjAzMSAxLjMx%0D%0AMjUgLTAuNTYyNSBDIDEuNDM3NSAtMC41MzkwNjIgMS41NzAzMTIgLTAuNTMxMjUgMS43MTg3NSAt%0D%0AMC41MzEyNSBDIDIuMDA3ODEyIC0wLjUzMTI1IDIuMjUzOTA2IC0wLjU4MjAzMSAyLjQ1MzEyNSAt%0D%0AMC42ODc1IEMgMi42NjAxNTYgLTAuODAwNzgxIDIuODI4MTI1IC0wLjk0NTMxMiAyLjk1MzEyNSAt%0D%0AMS4xMjUgQyAzLjA3ODEyNSAtMS4zMTI1IDMuMTcxODc1IC0xLjUyMzQzOCAzLjIzNDM3NSAtMS43%0D%0ANjU2MjUgQyAzLjMwNDY4OCAtMi4wMTU2MjUgMy4zNDM3NSAtMi4yNzM0MzggMy4zNDM3NSAtMi41%0D%0ANDY4NzUgQyAzLjE4NzUgLTIuNDUzMTI1IDMgLTIuMzY3MTg4IDIuNzgxMjUgLTIuMjk2ODc1IEMg%0D%0AMi41NzAzMTIgLTIuMjM0Mzc1IDIuMzI4MTI1IC0yLjIwMzEyNSAyLjA0Njg3NSAtMi4yMDMxMjUg%0D%0AQyAxLjcyMjY1NiAtMi4yMDMxMjUgMS40NTMxMjUgLTIuMjQyMTg4IDEuMjM0Mzc1IC0yLjMyODEy%0D%0ANSBDIDEuMDIzNDM4IC0yLjQxMDE1NiAwLjg1MTU2MiAtMi41MjM0MzggMC43MTg3NSAtMi42NzE4%0D%0ANzUgQyAwLjU5Mzc1IC0yLjgyODEyNSAwLjUgLTMuMDAzOTA2IDAuNDM3NSAtMy4yMDMxMjUgQyAw%0D%0ALjM4MjgxMiAtMy40MTAxNTYgMC4zNTkzNzUgLTMuNjQwNjI1IDAuMzU5Mzc1IC0zLjg5MDYyNSBD%0D%0AIDAuMzU5Mzc1IC00LjE0MDYyNSAwLjM5NDUzMSAtNC4zNzUgMC40Njg3NSAtNC41OTM3NSBDIDAu%0D%0ANTM5MDYyIC00LjgyMDMxMiAwLjY1NjI1IC01LjAyMzQzOCAwLjgxMjUgLTUuMjAzMTI1IEMgMC45%0D%0ANjg3NSAtNS4zNzg5MDYgMS4xNjQwNjIgLTUuNTE1NjI1IDEuNDA2MjUgLTUuNjA5Mzc1IEMgMS42%0D%0ANDQ1MzEgLTUuNzEwOTM4IDEuOTI5Njg4IC01Ljc2NTYyNSAyLjI2NTYyNSAtNS43NjU2MjUgQyAy%0D%0ALjUwMzkwNiAtNS43NjU2MjUgMi43MTA5MzggLTUuNzM0Mzc1IDIuODkwNjI1IC01LjY3MTg3NSBD%0D%0AIDMuMDc4MTI1IC01LjYxNzE4OCAzLjIzODI4MSAtNS41MzUxNTYgMy4zNzUgLTUuNDIxODc1IEMg%0D%0AMy41MDc4MTIgLTUuMzA0Njg4IDMuNjI1IC01LjE2NDA2MiAzLjcxODc1IC01IEMgMy44MTI1IC00%0D%0ALjg0Mzc1IDMuODgyODEyIC00LjY2NDA2MiAzLjkzNzUgLTQuNDY4NzUgQyA0IC00LjI2OTUzMSA0%0D%0ALjAzOTA2MiAtNC4wNTA3ODEgNC4wNjI1IC0zLjgxMjUgQyA0LjA5Mzc1IC0zLjU3MDMxMiA0LjEw%0D%0AOTM3NSAtMy4zMjAzMTIgNC4xMDkzNzUgLTMuMDYyNSBaIE0gMy4zMjgxMjUgLTMuMTU2MjUgQyAz%0D%0ALjMyODEyNSAtMy41MzEyNSAzLjMwMDc4MSAtMy44NDM3NSAzLjI1IC00LjA5Mzc1IEMgMy4yMDcw%0D%0AMzEgLTQuMzUxNTYyIDMuMTMyODEyIC00LjU2MjUgMy4wMzEyNSAtNC43MTg3NSBDIDIuOTM3NSAt%0D%0ANC44NzUgMi44MTY0MDYgLTQuOTg4MjgxIDIuNjcxODc1IC01LjA2MjUgQyAyLjUzNTE1NiAtNS4x%0D%0AMzI4MTIgMi4zNzg5MDYgLTUuMTcxODc1IDIuMjAzMTI1IC01LjE3MTg3NSBDIDIuMDAzOTA2IC01%0D%0ALjE3MTg3NSAxLjgzNTkzOCAtNS4xMzI4MTIgMS43MDMxMjUgLTUuMDYyNSBDIDEuNTY2NDA2IC01%0D%0AIDEuNDU3MDMxIC00LjkwNjI1IDEuMzc1IC00Ljc4MTI1IEMgMS4yODkwNjIgLTQuNjY0MDYyIDEu%0D%0AMjI2NTYyIC00LjUzNTE1NiAxLjE4NzUgLTQuMzkwNjI1IEMgMS4xNDQ1MzEgLTQuMjQyMTg4IDEu%0D%0AMTI1IC00LjA5Mzc1IDEuMTI1IC0zLjkzNzUgQyAxLjEyNSAtMy43Njk1MzEgMS4xNDA2MjUgLTMu%0D%0ANjEzMjgxIDEuMTcxODc1IC0zLjQ2ODc1IEMgMS4yMTA5MzggLTMuMzIwMzEyIDEuMjczNDM4IC0z%0D%0ALjE5NTMxMiAxLjM1OTM3NSAtMy4wOTM3NSBDIDEuNDQxNDA2IC0zIDEuNTQ2ODc1IC0yLjkyNTc4%0D%0AMSAxLjY3MTg3NSAtMi44NzUgQyAxLjgwNDY4OCAtMi44MjAzMTIgMS45NzI2NTYgLTIuNzk2ODc1%0D%0AIDIuMTcxODc1IC0yLjc5Njg3NSBDIDIuMzkwNjI1IC0yLjc5Njg3NSAyLjU5NzY1NiAtMi44Mjgx%0D%0AMjUgMi43OTY4NzUgLTIuODkwNjI1IEMgMy4wMDM5MDYgLTIuOTUzMTI1IDMuMTc5Njg4IC0zLjAz%0D%0AOTA2MiAzLjMyODEyNSAtMy4xNTYyNSBaIE0gMy4zMjgxMjUgLTMuMTU2MjUgIi8+Cjwvc3ltYm9s%0D%0APgo8c3ltYm9sIG92ZXJmbG93PSJ2aXNpYmxlIiBpZD0iZ2x5cGgxLTEwIj4KPHBhdGggc3R5bGU9%0D%0AInN0cm9rZTpub25lOyIgZD0iTSA0LjI1IC0yLjg1OTM3NSBDIDQuMjUgLTIuNDIxODc1IDQuMjEw%0D%0AOTM4IC0yLjAxOTUzMSA0LjE0MDYyNSAtMS42NTYyNSBDIDQuMDc4MTI1IC0xLjMwMDc4MSAzLjk2%0D%0AMDkzOCAtMC45OTIxODggMy43OTY4NzUgLTAuNzM0Mzc1IEMgMy42NDA2MjUgLTAuNDcyNjU2IDMu%0D%0ANDI5Njg4IC0wLjI2OTUzMSAzLjE3MTg3NSAtMC4xMjUgQyAyLjkyMTg3NSAwLjAwNzgxMjUgMi42%0D%0AMDE1NjIgMC4wNzgxMjUgMi4yMTg3NSAwLjA3ODEyNSBDIDEuODYzMjgxIDAuMDc4MTI1IDEuNTYy%0D%0ANSAwLjAxNTYyNSAxLjMxMjUgLTAuMTA5Mzc1IEMgMS4wNzAzMTIgLTAuMjM0Mzc1IDAuODc1IC0w%0D%0ALjQyMTg3NSAwLjcxODc1IC0wLjY3MTg3NSBDIDAuNTcwMzEyIC0wLjkyMTg3NSAwLjQ2ODc1IC0x%0D%0ALjIyMjY1NiAwLjQwNjI1IC0xLjU3ODEyNSBDIDAuMzQzNzUgLTEuOTQxNDA2IDAuMzEyNSAtMi4z%0D%0ANTkzNzUgMC4zMTI1IC0yLjgyODEyNSBDIDAuMzEyNSAtMy4yNjU2MjUgMC4zNDc2NTYgLTMuNjYw%0D%0AMTU2IDAuNDIxODc1IC00LjAxNTYyNSBDIDAuNDkyMTg4IC00LjM3ODkwNiAwLjYwOTM3NSAtNC42%0D%0AOTE0MDYgMC43NjU2MjUgLTQuOTUzMTI1IEMgMC45MjE4NzUgLTUuMjEwOTM4IDEuMTI4OTA2IC01%0D%0ALjQxMDE1NiAxLjM5MDYyNSAtNS41NDY4NzUgQyAxLjY0ODQzOCAtNS42OTE0MDYgMS45Njg3NSAt%0D%0ANS43NjU2MjUgMi4zNDM3NSAtNS43NjU2MjUgQyAyLjY5NTMxMiAtNS43NjU2MjUgMi45OTIxODgg%0D%0ALTUuNzAzMTI1IDMuMjM0Mzc1IC01LjU3ODEyNSBDIDMuNDg0Mzc1IC01LjQ1MzEyNSAzLjY3OTY4%0D%0AOCAtNS4yNjU2MjUgMy44MjgxMjUgLTUuMDE1NjI1IEMgMy45ODQzNzUgLTQuNzY1NjI1IDQuMDkz%0D%0ANzUgLTQuNDU3MDMxIDQuMTU2MjUgLTQuMDkzNzUgQyA0LjIxODc1IC0zLjczODI4MSA0LjI1IC0z%0D%0ALjMyODEyNSA0LjI1IC0yLjg1OTM3NSBaIE0gMy40Njg3NSAtMi43OTY4NzUgQyAzLjQ2ODc1IC0z%0D%0ALjA4NTkzOCAzLjQ1NzAzMSAtMy4zNDM3NSAzLjQzNzUgLTMuNTYyNSBDIDMuNDI1NzgxIC0zLjc4%0D%0AOTA2MiAzLjM5ODQzOCAtMy45ODgyODEgMy4zNTkzNzUgLTQuMTU2MjUgQyAzLjMxNjQwNiAtNC4z%0D%0AMzIwMzEgMy4yNjU2MjUgLTQuNDg0Mzc1IDMuMjAzMTI1IC00LjYwOTM3NSBDIDMuMTQwNjI1IC00%0D%0ALjczNDM3NSAzLjA2MjUgLTQuODM1OTM4IDIuOTY4NzUgLTQuOTIxODc1IEMgMi44ODI4MTIgLTUu%0D%0AMDAzOTA2IDIuNzg1MTU2IC01LjA2MjUgMi42NzE4NzUgLTUuMDkzNzUgQyAyLjU1NDY4OCAtNS4x%0D%0AMzI4MTIgMi40Mjk2ODggLTUuMTU2MjUgMi4yOTY4NzUgLTUuMTU2MjUgQyAyLjA0Njg3NSAtNS4x%0D%0ANTYyNSAxLjg0Mzc1IC01LjA5Mzc1IDEuNjg3NSAtNC45Njg3NSBDIDEuNTMxMjUgLTQuODUxNTYy%0D%0AIDEuNDA2MjUgLTQuNjkxNDA2IDEuMzEyNSAtNC40ODQzNzUgQyAxLjIyNjU2MiAtNC4yODUxNTYg%0D%0AMS4xNzE4NzUgLTQuMDQ2ODc1IDEuMTQwNjI1IC0zLjc2NTYyNSBDIDEuMTA5Mzc1IC0zLjQ5MjE4%0D%0AOCAxLjA5Mzc1IC0zLjIwMzEyNSAxLjA5Mzc1IC0yLjg5MDYyNSBDIDEuMDkzNzUgLTIuNDYwOTM4%0D%0AIDEuMTEzMjgxIC0yLjA5NzY1NiAxLjE1NjI1IC0xLjc5Njg3NSBDIDEuMTk1MzEyIC0xLjUwMzkw%0D%0ANiAxLjI2NTYyNSAtMS4yNjU2MjUgMS4zNTkzNzUgLTEuMDc4MTI1IEMgMS40NTMxMjUgLTAuODkw%0D%0ANjI1IDEuNTcwMzEyIC0wLjc1IDEuNzE4NzUgLTAuNjU2MjUgQyAxLjg3NSAtMC41NzAzMTIgMi4w%0D%0ANTQ2ODggLTAuNTMxMjUgMi4yNjU2MjUgLTAuNTMxMjUgQyAyLjQyOTY4OCAtMC41MzEyNSAyLjU3%0D%0AODEyNSAtMC41NTQ2ODggMi43MDMxMjUgLTAuNjA5Mzc1IEMgMi44MjgxMjUgLTAuNjYwMTU2IDIu%0D%0AOTI5Njg4IC0wLjczNDM3NSAzLjAxNTYyNSAtMC44MjgxMjUgQyAzLjEwOTM3NSAtMC45Mjk2ODgg%0D%0AMy4xNzk2ODggLTEuMDUwNzgxIDMuMjM0Mzc1IC0xLjE4NzUgQyAzLjI5Njg3NSAtMS4zMzIwMzEg%0D%0AMy4zNDM3NSAtMS40ODgyODEgMy4zNzUgLTEuNjU2MjUgQyAzLjQxNDA2MiAtMS44MjAzMTIgMy40%0D%0ANDE0MDYgLTIgMy40NTMxMjUgLTIuMTg3NSBDIDMuNDYwOTM4IC0yLjM4MjgxMiAzLjQ2ODc1IC0y%0D%0ALjU4NTkzOCAzLjQ2ODc1IC0yLjc5Njg3NSBaIE0gMy40Njg3NSAtMi43OTY4NzUgIi8+Cjwvc3lt%0D%0AYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2aXNpYmxlIiBpZD0iZ2x5cGgyLTAiPgo8cGF0aCBzdHls%0D%0AZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDYuNjg3NSAtOC44NDM3NSBMIDYuNjg3NSAwIEwgMC4yOTY4%0D%0ANzUgMCBMIDAuMjk2ODc1IC04Ljg0Mzc1IFogTSA2LjA5Mzc1IC0wLjU0Njg3NSBMIDYuMDkzNzUg%0D%0ALTguMjk2ODc1IEwgMC44NzUgLTguMjk2ODc1IEwgMC44NzUgLTAuNTQ2ODc1IFogTSA1LjI1IC01%0D%0ALjg0Mzc1IEMgNS4yNSAtNS41NzAzMTIgNS4yMDcwMzEgLTUuMzMyMDMxIDUuMTI1IC01LjEyNSBD%0D%0AIDUuMDM5MDYyIC00LjkxNDA2MiA0LjkyNTc4MSAtNC43MzgyODEgNC43ODEyNSAtNC41OTM3NSBD%0D%0AIDQuNjQ0NTMxIC00LjQ0NTMxMiA0LjQ3NjU2MiAtNC4zMzIwMzEgNC4yODEyNSAtNC4yNSBDIDQu%0D%0AMDgyMDMxIC00LjE3NTc4MSAzLjg2NzE4OCAtNC4xMjg5MDYgMy42NDA2MjUgLTQuMTA5Mzc1IEwg%0D%0AMy42MDkzNzUgLTMuMDc4MTI1IEMgMy42MDkzNzUgLTMuMDM1MTU2IDMuNTcwMzEyIC0zLjAwMzkw%0D%0ANiAzLjUgLTIuOTg0Mzc1IEMgMy40MjU3ODEgLTIuOTYwOTM4IDMuMzMyMDMxIC0yLjk1MzEyNSAz%0D%0ALjIxODc1IC0yLjk1MzEyNSBDIDMuMTU2MjUgLTIuOTUzMTI1IDMuMDk3NjU2IC0yLjk1MzEyNSAz%0D%0ALjA0Njg3NSAtMi45NTMxMjUgQyAzLjAwMzkwNiAtMi45NjA5MzggMi45Njg3NSAtMi45NzI2NTYg%0D%0AMi45Mzc1IC0yLjk4NDM3NSBDIDIuOTA2MjUgLTIuOTkyMTg4IDIuODc4OTA2IC0zLjAwMzkwNiAy%0D%0ALjg1OTM3NSAtMy4wMTU2MjUgQyAyLjg0NzY1NiAtMy4wMjM0MzggMi44NDM3NSAtMy4wNDY4NzUg%0D%0AMi44NDM3NSAtMy4wNzgxMjUgTCAyLjgxMjUgLTQuMzI4MTI1IEMgMi44MDA3ODEgLTQuNDcyNjU2%0D%0AIDIuODI4MTI1IC00LjU3MDMxMiAyLjg5MDYyNSAtNC42MjUgQyAyLjk2MDkzOCAtNC42NzU3ODEg%0D%0AMy4wNTA3ODEgLTQuNzAzMTI1IDMuMTU2MjUgLTQuNzAzMTI1IEwgMy4yODEyNSAtNC43MDMxMjUg%0D%0AQyAzLjQ4ODI4MSAtNC43MDMxMjUgMy42NjAxNTYgLTQuNzI2NTYyIDMuNzk2ODc1IC00Ljc4MTI1%0D%0AIEMgMy45Mjk2ODggLTQuODQzNzUgNC4wMzkwNjIgLTQuOTIxODc1IDQuMTI1IC01LjAxNTYyNSBD%0D%0AIDQuMjA3MDMxIC01LjEwOTM3NSA0LjI2NTYyNSAtNS4yMjI2NTYgNC4yOTY4NzUgLTUuMzU5Mzc1%0D%0AIEMgNC4zMzU5MzggLTUuNDkyMTg4IDQuMzU5Mzc1IC01LjY0MDYyNSA0LjM1OTM3NSAtNS43OTY4%0D%0ANzUgQyA0LjM1OTM3NSAtNi4xMjg5MDYgNC4yNjU2MjUgLTYuMzkwNjI1IDQuMDc4MTI1IC02LjU3%0D%0AODEyNSBDIDMuODkwNjI1IC02Ljc3MzQzOCAzLjU4NTkzOCAtNi44NzUgMy4xNzE4NzUgLTYuODc1%0D%0AIEMgMy4wMDM5MDYgLTYuODc1IDIuODUxNTYyIC02Ljg1OTM3NSAyLjcxODc1IC02LjgyODEyNSBD%0D%0AIDIuNTgyMDMxIC02Ljc5Njg3NSAyLjQ1NzAzMSAtNi43NTM5MDYgMi4zNDM3NSAtNi43MDMxMjUg%0D%0AQyAyLjIzODI4MSAtNi42NjAxNTYgMi4xNTYyNSAtNi42MTcxODggMi4wOTM3NSAtNi41NzgxMjUg%0D%0AQyAyLjAzMTI1IC02LjU0Njg3NSAxLjk4NDM3NSAtNi41MzEyNSAxLjk1MzEyNSAtNi41MzEyNSBD%0D%0AIDEuOTI5Njg4IC02LjUzMTI1IDEuOTEwMTU2IC02LjUzNTE1NiAxLjg5MDYyNSAtNi41NDY4NzUg%0D%0AQyAxLjg3ODkwNiAtNi41NTQ2ODggMS44NjcxODggLTYuNTcwMzEyIDEuODU5Mzc1IC02LjU5Mzc1%0D%0AIEMgMS44NDc2NTYgLTYuNjI1IDEuODM1OTM4IC02LjY2MDE1NiAxLjgyODEyNSAtNi43MDMxMjUg%0D%0AQyAxLjgxNjQwNiAtNi43NDIxODggMS44MTI1IC02LjgwMDc4MSAxLjgxMjUgLTYuODc1IEMgMS44%0D%0AMTI1IC02Ljk1NzAzMSAxLjgxNjQwNiAtNy4wMTk1MzEgMS44MjgxMjUgLTcuMDYyNSBDIDEuODQ3%0D%0ANjU2IC03LjExMzI4MSAxLjg3NSAtNy4xNjAxNTYgMS45MDYyNSAtNy4yMDMxMjUgQyAxLjkzNzUg%0D%0ALTcuMjM0Mzc1IDIgLTcuMjY5NTMxIDIuMDkzNzUgLTcuMzEyNSBDIDIuMTg3NSAtNy4zNTE1NjIg%0D%0AMi4yOTY4NzUgLTcuMzkwNjI1IDIuNDIxODc1IC03LjQyMTg3NSBDIDIuNTQ2ODc1IC03LjQ2MDkz%0D%0AOCAyLjY3OTY4OCAtNy40OTIxODggMi44MjgxMjUgLTcuNTE1NjI1IEMgMi45ODQzNzUgLTcuNTQ2%0D%0AODc1IDMuMTMyODEyIC03LjU2MjUgMy4yODEyNSAtNy41NjI1IEMgMy42MjUgLTcuNTYyNSAzLjky%0D%0AMTg3NSAtNy41MTU2MjUgNC4xNzE4NzUgLTcuNDIxODc1IEMgNC40MjE4NzUgLTcuMzM1OTM4IDQu%0D%0ANjI1IC03LjIxODc1IDQuNzgxMjUgLTcuMDYyNSBDIDQuOTM3NSAtNi45MDYyNSA1LjA1MDc4MSAt%0D%0ANi43MjI2NTYgNS4xMjUgLTYuNTE1NjI1IEMgNS4yMDcwMzEgLTYuMzA0Njg4IDUuMjUgLTYuMDgy%0D%0AMDMxIDUuMjUgLTUuODQzNzUgWiBNIDMuNzUgLTEuODU5Mzc1IEMgMy43NSAtMS43NjU2MjUgMy43%0D%0AMzgyODEgLTEuNjg3NSAzLjcxODc1IC0xLjYyNSBDIDMuNzA3MDMxIC0xLjU2MjUgMy42Nzk2ODgg%0D%0ALTEuNTA3ODEyIDMuNjQwNjI1IC0xLjQ2ODc1IEMgMy42MDkzNzUgLTEuNDI1NzgxIDMuNTU0Njg4%0D%0AIC0xLjM5NDUzMSAzLjQ4NDM3NSAtMS4zNzUgQyAzLjQyMTg3NSAtMS4zNjMyODEgMy4zNDM3NSAt%0D%0AMS4zNTkzNzUgMy4yNSAtMS4zNTkzNzUgQyAzLjE0NDUzMSAtMS4zNTkzNzUgMy4wNjI1IC0xLjM2%0D%0AMzI4MSAzIC0xLjM3NSBDIDIuOTM3NSAtMS4zOTQ1MzEgMi44ODI4MTIgLTEuNDI1NzgxIDIuODQz%0D%0ANzUgLTEuNDY4NzUgQyAyLjgwMDc4MSAtMS41MDc4MTIgMi43Njk1MzEgLTEuNTYyNSAyLjc1IC0x%0D%0ALjYyNSBDIDIuNzI2NTYyIC0xLjY4NzUgMi43MTg3NSAtMS43NjU2MjUgMi43MTg3NSAtMS44NTkz%0D%0ANzUgQyAyLjcxODc1IC0xLjk2MDkzOCAyLjcyNjU2MiAtMi4wNDY4NzUgMi43NSAtMi4xMDkzNzUg%0D%0AQyAyLjc2OTUzMSAtMi4xNzE4NzUgMi44MDA3ODEgLTIuMjIyNjU2IDIuODQzNzUgLTIuMjY1NjI1%0D%0AIEMgMi44ODI4MTIgLTIuMzA0Njg4IDIuOTM3NSAtMi4zMzU5MzggMyAtMi4zNTkzNzUgQyAzLjA2%0D%0AMjUgLTIuMzc4OTA2IDMuMTQ0NTMxIC0yLjM5MDYyNSAzLjI1IC0yLjM5MDYyNSBDIDMuMzQzNzUg%0D%0ALTIuMzkwNjI1IDMuNDIxODc1IC0yLjM3ODkwNiAzLjQ4NDM3NSAtMi4zNTkzNzUgQyAzLjU1NDY4%0D%0AOCAtMi4zMzU5MzggMy42MDkzNzUgLTIuMzA0Njg4IDMuNjQwNjI1IC0yLjI2NTYyNSBDIDMuNjc5%0D%0ANjg4IC0yLjIyMjY1NiAzLjcwNzAzMSAtMi4xNzE4NzUgMy43MTg3NSAtMi4xMDkzNzUgQyAzLjcz%0D%0AODI4MSAtMi4wNDY4NzUgMy43NSAtMS45NjA5MzggMy43NSAtMS44NTkzNzUgWiBNIDAgMi43MTg3%0D%0ANSBaIE0gMCAyLjcxODc1ICIvPgo8L3N5bWJvbD4KPHN5bWJvbCBvdmVyZmxvdz0idmlzaWJsZSIg%0D%0AaWQ9ImdseXBoMi0xIj4KPHBhdGggc3R5bGU9InN0cm9rZTpub25lOyIgZD0iTSAyLjE3MTg3NSAt%0D%0AMC4xNzE4NzUgQyAyLjE3MTg3NSAtMC4xMjg5MDYgMi4xNjAxNTYgLTAuMDk3NjU2MiAyLjE0MDYy%0D%0ANSAtMC4wNzgxMjUgQyAyLjEyODkwNiAtMC4wNTQ2ODc1IDIuMTAxNTYyIC0wLjAzNTE1NjIgMi4w%0D%0ANjI1IC0wLjAxNTYyNSBDIDIuMDE5NTMxIDAuMDAzOTA2MjUgMS45NjA5MzggMC4wMTk1MzEyIDEu%0D%0AODkwNjI1IDAuMDMxMjUgQyAxLjgxNjQwNiAwLjAzOTA2MjUgMS43MjI2NTYgMC4wNDY4NzUgMS42%0D%0AMDkzNzUgMC4wNDY4NzUgQyAxLjQ5MjE4OCAwLjA0Njg3NSAxLjM5ODQzOCAwLjAzOTA2MjUgMS4z%0D%0AMjgxMjUgMC4wMzEyNSBDIDEuMjUzOTA2IDAuMDE5NTMxMiAxLjE5NTMxMiAwLjAwMzkwNjI1IDEu%0D%0AMTU2MjUgLTAuMDE1NjI1IEMgMS4xMTMyODEgLTAuMDM1MTU2MiAxLjA4MjAzMSAtMC4wNTQ2ODc1%0D%0AIDEuMDYyNSAtMC4wNzgxMjUgQyAxLjA1MDc4MSAtMC4wOTc2NTYyIDEuMDQ2ODc1IC0wLjEyODkw%0D%0ANiAxLjA0Njg3NSAtMC4xNzE4NzUgTCAxLjA0Njg3NSAtOS4yOTY4NzUgQyAxLjA0Njg3NSAtOS4z%0D%0AMzU5MzggMS4wNTA3ODEgLTkuMzY3MTg4IDEuMDYyNSAtOS4zOTA2MjUgQyAxLjA4MjAzMSAtOS40%0D%0AMjE4NzUgMS4xMTMyODEgLTkuNDQ1MzEyIDEuMTU2MjUgLTkuNDY4NzUgQyAxLjE5NTMxMiAtOS40%0D%0AODgyODEgMS4yNTM5MDYgLTkuNSAxLjMyODEyNSAtOS41IEMgMS4zOTg0MzggLTkuNTA3ODEyIDEu%0D%0ANDkyMTg4IC05LjUxNTYyNSAxLjYwOTM3NSAtOS41MTU2MjUgQyAxLjcyMjY1NiAtOS41MTU2MjUg%0D%0AMS44MTY0MDYgLTkuNTA3ODEyIDEuODkwNjI1IC05LjUgQyAxLjk2MDkzOCAtOS41IDIuMDE5NTMx%0D%0AIC05LjQ4ODI4MSAyLjA2MjUgLTkuNDY4NzUgQyAyLjEwMTU2MiAtOS40NDUzMTIgMi4xMjg5MDYg%0D%0ALTkuNDIxODc1IDIuMTQwNjI1IC05LjM5MDYyNSBDIDIuMTYwMTU2IC05LjM2NzE4OCAyLjE3MTg3%0D%0ANSAtOS4zMzU5MzggMi4xNzE4NzUgLTkuMjk2ODc1IFogTSAyLjE3MTg3NSAtMC4xNzE4NzUgIi8+%0D%0ACjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2aXNpYmxlIiBpZD0iZ2x5cGgyLTIiPgo8cGF0%0D%0AaCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDYuNzY1NjI1IC0zLjMxMjUgQyA2Ljc2NTYyNSAt%0D%0AMi44MTI1IDYuNjk1MzEyIC0yLjM0NzY1NiA2LjU2MjUgLTEuOTIxODc1IEMgNi40Mzc1IC0xLjUw%0D%0AMzkwNiA2LjIzODI4MSAtMS4xNDQ1MzEgNS45Njg3NSAtMC44NDM3NSBDIDUuNzA3MDMxIC0wLjUz%0D%0AOTA2MiA1LjM3ODkwNiAtMC4zMDA3ODEgNC45ODQzNzUgLTAuMTI1IEMgNC41OTc2NTYgMC4wMzkw%0D%0ANjI1IDQuMTQ0NTMxIDAuMTI1IDMuNjI1IDAuMTI1IEMgMy4xMTMyODEgMC4xMjUgMi42NzE4NzUg%0D%0AMC4wNTA3ODEyIDIuMjk2ODc1IC0wLjA5Mzc1IEMgMS45MjE4NzUgLTAuMjUgMS42MDkzNzUgLTAu%0D%0ANDY4NzUgMS4zNTkzNzUgLTAuNzUgQyAxLjEwOTM3NSAtMS4wMzkwNjIgMC45MjE4NzUgLTEuMzkw%0D%0ANjI1IDAuNzk2ODc1IC0xLjc5Njg3NSBDIDAuNjcxODc1IC0yLjIwMzEyNSAwLjYwOTM3NSAtMi42%0D%0ANjQwNjIgMC42MDkzNzUgLTMuMTg3NSBDIDAuNjA5Mzc1IC0zLjY4NzUgMC42NzE4NzUgLTQuMTQ0%0D%0ANTMxIDAuNzk2ODc1IC00LjU2MjUgQyAwLjkyOTY4OCAtNC45ODgyODEgMS4xMjg5MDYgLTUuMzUx%0D%0ANTYyIDEuMzkwNjI1IC01LjY1NjI1IEMgMS42NjAxNTYgLTUuOTY4NzUgMS45ODgyODEgLTYuMjA3%0D%0AMDMxIDIuMzc1IC02LjM3NSBDIDIuNzY5NTMxIC02LjUzOTA2MiAzLjIyNjU2MiAtNi42MjUgMy43%0D%0ANSAtNi42MjUgQyA0LjI1NzgxMiAtNi42MjUgNC43MDMxMjUgLTYuNTUwNzgxIDUuMDc4MTI1IC02%0D%0ALjQwNjI1IEMgNS40NTMxMjUgLTYuMjU3ODEyIDUuNzY1NjI1IC02LjAzOTA2MiA2LjAxNTYyNSAt%0D%0ANS43NSBDIDYuMjY1NjI1IC01LjQ1NzAzMSA2LjQ1MzEyNSAtNS4xMDkzNzUgNi41NzgxMjUgLTQu%0D%0ANzAzMTI1IEMgNi43MDMxMjUgLTQuMjk2ODc1IDYuNzY1NjI1IC0zLjgzMjAzMSA2Ljc2NTYyNSAt%0D%0AMy4zMTI1IFogTSA1LjU3ODEyNSAtMy4yMzQzNzUgQyA1LjU3ODEyNSAtMy41NjY0MDYgNS41NDY4%0D%0ANzUgLTMuODc4OTA2IDUuNDg0Mzc1IC00LjE3MTg3NSBDIDUuNDIxODc1IC00LjQ3MjY1NiA1LjMx%0D%0ANjQwNiAtNC43MzQzNzUgNS4xNzE4NzUgLTQuOTUzMTI1IEMgNS4wMzUxNTYgLTUuMTc5Njg4IDQu%0D%0AODQ3NjU2IC01LjM1OTM3NSA0LjYwOTM3NSAtNS40ODQzNzUgQyA0LjM2NzE4OCAtNS42MTcxODgg%0D%0ANC4wNjY0MDYgLTUuNjg3NSAzLjcwMzEyNSAtNS42ODc1IEMgMy4zNjcxODggLTUuNjg3NSAzLjA4%0D%0AMjAzMSAtNS42MjUgMi44NDM3NSAtNS41IEMgMi42MDE1NjIgLTUuMzgyODEyIDIuNDA2MjUgLTUu%0D%0AMjE4NzUgMi4yNSAtNSBDIDIuMTAxNTYyIC00Ljc4OTA2MiAxLjk4ODI4MSAtNC41MzUxNTYgMS45%0D%0AMDYyNSAtNC4yMzQzNzUgQyAxLjgzMjAzMSAtMy45NDE0MDYgMS43OTY4NzUgLTMuNjE3MTg4IDEu%0D%0ANzk2ODc1IC0zLjI2NTYyNSBDIDEuNzk2ODc1IC0yLjkyOTY4OCAxLjgyODEyNSAtMi42MTMyODEg%0D%0AMS44OTA2MjUgLTIuMzEyNSBDIDEuOTUzMTI1IC0yLjAxOTUzMSAyLjA1MDc4MSAtMS43NTc4MTIg%0D%0AMi4xODc1IC0xLjUzMTI1IEMgMi4zMzIwMzEgLTEuMzEyNSAyLjUyMzQzOCAtMS4xNDA2MjUgMi43%0D%0ANjU2MjUgLTEuMDE1NjI1IEMgMy4wMTU2MjUgLTAuODkwNjI1IDMuMzE2NDA2IC0wLjgyODEyNSAz%0D%0ALjY3MTg3NSAtMC44MjgxMjUgQyA0LjAwMzkwNiAtMC44MjgxMjUgNC4yODkwNjIgLTAuODgyODEy%0D%0AIDQuNTMxMjUgLTEgQyA0Ljc2OTUzMSAtMS4xMTMyODEgNC45Njg3NSAtMS4yNzM0MzggNS4xMjUg%0D%0ALTEuNDg0Mzc1IEMgNS4yODEyNSAtMS43MDMxMjUgNS4zOTQ1MzEgLTEuOTU3MDMxIDUuNDY4NzUg%0D%0ALTIuMjUgQyA1LjUzOTA2MiAtMi41NTA3ODEgNS41NzgxMjUgLTIuODc4OTA2IDUuNTc4MTI1IC0z%0D%0ALjIzNDM3NSBaIE0gNS41NzgxMjUgLTMuMjM0Mzc1ICIvPgo8L3N5bWJvbD4KPHN5bWJvbCBvdmVy%0D%0AZmxvdz0idmlzaWJsZSIgaWQ9ImdseXBoMi0zIj4KPHBhdGggc3R5bGU9InN0cm9rZTpub25lOyIg%0D%0AZD0iTSA0LjkwNjI1IC0xLjg0Mzc1IEMgNC45MDYyNSAtMS41MzEyNSA0Ljg0NzY1NiAtMS4yNSA0%0D%0ALjczNDM3NSAtMSBDIDQuNjE3MTg4IC0wLjc1NzgxMiA0LjQ1MzEyNSAtMC41NTA3ODEgNC4yMzQz%0D%0ANzUgLTAuMzc1IEMgNC4wMjM0MzggLTAuMjA3MDMxIDMuNzY5NTMxIC0wLjA4MjAzMTIgMy40Njg3%0D%0ANSAwIEMgMy4xNzU3ODEgMC4wODIwMzEyIDIuODUxNTYyIDAuMTI1IDIuNSAwLjEyNSBDIDIuMjgx%0D%0AMjUgMC4xMjUgMi4wNzAzMTIgMC4xMDkzNzUgMS44NzUgMC4wNzgxMjUgQyAxLjY3NTc4MSAwLjA0%0D%0ANjg3NSAxLjQ5MjE4OCAwLjAwMzkwNjI1IDEuMzI4MTI1IC0wLjA0Njg3NSBDIDEuMTcxODc1IC0w%0D%0ALjA5NzY1NjIgMS4wMzUxNTYgLTAuMTQ4NDM4IDAuOTIxODc1IC0wLjIwMzEyNSBDIDAuODE2NDA2%0D%0AIC0wLjI2NTYyNSAwLjczODI4MSAtMC4zMTY0MDYgMC42ODc1IC0wLjM1OTM3NSBDIDAuNjQ0NTMx%0D%0AIC0wLjQxMDE1NiAwLjYwOTM3NSAtMC40NzY1NjIgMC41NzgxMjUgLTAuNTYyNSBDIDAuNTU0Njg4%0D%0AIC0wLjY0NDUzMSAwLjU0Njg3NSAtMC43NTM5MDYgMC41NDY4NzUgLTAuODkwNjI1IEMgMC41NDY4%0D%0ANzUgLTAuOTg0Mzc1IDAuNTUwNzgxIC0xLjA1NDY4OCAwLjU2MjUgLTEuMTA5Mzc1IEMgMC41NzAz%0D%0AMTIgLTEuMTcxODc1IDAuNTgyMDMxIC0xLjIxODc1IDAuNTkzNzUgLTEuMjUgQyAwLjYxMzI4MSAt%0D%0AMS4yODkwNjIgMC42MzI4MTIgLTEuMzE2NDA2IDAuNjU2MjUgLTEuMzI4MTI1IEMgMC42ODc1IC0x%0D%0ALjM0NzY1NiAwLjcxODc1IC0xLjM1OTM3NSAwLjc1IC0xLjM1OTM3NSBDIDAuODAwNzgxIC0xLjM1%0D%0AOTM3NSAwLjg3NSAtMS4zMjgxMjUgMC45Njg3NSAtMS4yNjU2MjUgQyAxLjA2MjUgLTEuMjAzMTI1%0D%0AIDEuMTc5Njg4IC0xLjEzMjgxMiAxLjMyODEyNSAtMS4wNjI1IEMgMS40NzI2NTYgLTAuOTg4Mjgx%0D%0AIDEuNjQ0NTMxIC0wLjkyMTg3NSAxLjg0Mzc1IC0wLjg1OTM3NSBDIDIuMDM5MDYyIC0wLjc5Njg3%0D%0ANSAyLjI2NTYyNSAtMC43NjU2MjUgMi41MTU2MjUgLTAuNzY1NjI1IEMgMi43MDMxMjUgLTAuNzY1%0D%0ANjI1IDIuODc1IC0wLjc4NTE1NiAzLjAzMTI1IC0wLjgyODEyNSBDIDMuMTg3NSAtMC44NjcxODgg%0D%0AMy4zMjAzMTIgLTAuOTI5Njg4IDMuNDM3NSAtMS4wMTU2MjUgQyAzLjU1MDc4MSAtMS4wOTc2NTYg%0D%0AMy42NDA2MjUgLTEuMTk1MzEyIDMuNzAzMTI1IC0xLjMxMjUgQyAzLjc2NTYyNSAtMS40Mzc1IDMu%0D%0ANzk2ODc1IC0xLjU4MjAzMSAzLjc5Njg3NSAtMS43NSBDIDMuNzk2ODc1IC0xLjkyNTc4MSAzLjc1%0D%0AIC0yLjA3MDMxMiAzLjY1NjI1IC0yLjE4NzUgQyAzLjU3MDMxMiAtMi4zMTI1IDMuNDU3MDMxIC0y%0D%0ALjQxNDA2MiAzLjMxMjUgLTIuNSBDIDMuMTY0MDYyIC0yLjU5Mzc1IDMgLTIuNjc1NzgxIDIuODEy%0D%0ANSAtMi43NSBDIDIuNjMyODEyIC0yLjgyMDMxMiAyLjQ0NTMxMiAtMi44OTQ1MzEgMi4yNSAtMi45%0D%0ANjg3NSBDIDIuMDYyNSAtMy4wNTA3ODEgMS44NzUgLTMuMTQwNjI1IDEuNjg3NSAtMy4yMzQzNzUg%0D%0AQyAxLjUgLTMuMzM1OTM4IDEuMzMyMDMxIC0zLjQ1NzAzMSAxLjE4NzUgLTMuNTkzNzUgQyAxLjAz%0D%0AOTA2MiAtMy43MzgyODEgMC45MjE4NzUgLTMuOTA2MjUgMC44MjgxMjUgLTQuMDkzNzUgQyAwLjc0%0D%0AMjE4OCAtNC4yODkwNjIgMC43MDMxMjUgLTQuNTMxMjUgMC43MDMxMjUgLTQuODEyNSBDIDAuNzAz%0D%0AMTI1IC01LjA1MDc4MSAwLjc1IC01LjI4MTI1IDAuODQzNzUgLTUuNSBDIDAuOTM3NSAtNS43MTg3%0D%0ANSAxLjA3ODEyNSAtNS45MTAxNTYgMS4yNjU2MjUgLTYuMDc4MTI1IEMgMS40NTMxMjUgLTYuMjQy%0D%0AMTg4IDEuNjg3NSAtNi4zNzUgMS45Njg3NSAtNi40Njg3NSBDIDIuMjUgLTYuNTcwMzEyIDIuNTc4%0D%0AMTI1IC02LjYyNSAyLjk1MzEyNSAtNi42MjUgQyAzLjEwOTM3NSAtNi42MjUgMy4yNjk1MzEgLTYu%0D%0ANjA5Mzc1IDMuNDM3NSAtNi41NzgxMjUgQyAzLjYwMTU2MiAtNi41NTQ2ODggMy43NSAtNi41MjM0%0D%0AMzggMy44NzUgLTYuNDg0Mzc1IEMgNC4wMDc4MTIgLTYuNDQxNDA2IDQuMTI1IC02LjM5NDUzMSA0%0D%0ALjIxODc1IC02LjM0Mzc1IEMgNC4zMTI1IC02LjMwMDc4MSA0LjM3ODkwNiAtNi4yNTc4MTIgNC40%0D%0AMjE4NzUgLTYuMjE4NzUgQyA0LjQ3MjY1NiAtNi4xODc1IDQuNTAzOTA2IC02LjE1NjI1IDQuNTE1%0D%0ANjI1IC02LjEyNSBDIDQuNTM1MTU2IC02LjEwMTU2MiA0LjU1MDc4MSAtNi4wNzAzMTIgNC41NjI1%0D%0AIC02LjAzMTI1IEMgNC41NzAzMTIgLTYgNC41NzgxMjUgLTUuOTU3MDMxIDQuNTc4MTI1IC01Ljkw%0D%0ANjI1IEMgNC41ODU5MzggLTUuODYzMjgxIDQuNTkzNzUgLTUuODA0Njg4IDQuNTkzNzUgLTUuNzM0%0D%0AMzc1IEMgNC41OTM3NSAtNS42NDg0MzggNC41ODU5MzggLTUuNTc4MTI1IDQuNTc4MTI1IC01LjUx%0D%0ANTYyNSBDIDQuNTc4MTI1IC01LjQ2MDkzOCA0LjU2NjQwNiAtNS40MTQwNjIgNC41NDY4NzUgLTUu%0D%0AMzc1IEMgNC41MzUxNTYgLTUuMzQzNzUgNC41MTU2MjUgLTUuMzE2NDA2IDQuNDg0Mzc1IC01LjI5%0D%0ANjg3NSBDIDQuNDYwOTM4IC01LjI4NTE1NiA0LjQ0MTQwNiAtNS4yODEyNSA0LjQyMTg3NSAtNS4y%0D%0AODEyNSBDIDQuMzc4OTA2IC01LjI4MTI1IDQuMzE2NDA2IC01LjMwNDY4OCA0LjIzNDM3NSAtNS4z%0D%0ANTkzNzUgQyA0LjE2MDE1NiAtNS40MTAxNTYgNC4wNjI1IC01LjQ2MDkzOCAzLjkzNzUgLTUuNTE1%0D%0ANjI1IEMgMy44MTI1IC01LjU3ODEyNSAzLjY2NDA2MiAtNS42Mjg5MDYgMy41IC01LjY3MTg3NSBD%0D%0AIDMuMzMyMDMxIC01LjcyMjY1NiAzLjE0NDUzMSAtNS43NSAyLjkzNzUgLTUuNzUgQyAyLjczODI4%0D%0AMSAtNS43NSAyLjU2NjQwNiAtNS43MjY1NjIgMi40MjE4NzUgLTUuNjg3NSBDIDIuMjczNDM4IC01%0D%0ALjY0NDUzMSAyLjE1NjI1IC01LjU4MjAzMSAyLjA2MjUgLTUuNSBDIDEuOTY4NzUgLTUuNDI1Nzgx%0D%0AIDEuODk0NTMxIC01LjMzMjAzMSAxLjg0Mzc1IC01LjIxODc1IEMgMS44MDA3ODEgLTUuMTEzMjgx%0D%0AIDEuNzgxMjUgLTUgMS43ODEyNSAtNC44NzUgQyAxLjc4MTI1IC00LjY4NzUgMS44MjgxMjUgLTQu%0D%0ANTMxMjUgMS45MjE4NzUgLTQuNDA2MjUgQyAyLjAxNTYyNSAtNC4yODkwNjIgMi4xMzI4MTIgLTQu%0D%0AMTg3NSAyLjI4MTI1IC00LjA5Mzc1IEMgMi40MjU3ODEgLTQuMDA3ODEyIDIuNTkzNzUgLTMuOTI5%0D%0ANjg4IDIuNzgxMjUgLTMuODU5Mzc1IEMgMi45Njg3NSAtMy43ODUxNTYgMy4xNTYyNSAtMy43MDcw%0D%0AMzEgMy4zNDM3NSAtMy42MjUgQyAzLjUzOTA2MiAtMy41NTA3ODEgMy43MzQzNzUgLTMuNDY4NzUg%0D%0AMy45MjE4NzUgLTMuMzc1IEMgNC4xMDkzNzUgLTMuMjgxMjUgNC4yNzM0MzggLTMuMTYwMTU2IDQu%0D%0ANDIxODc1IC0zLjAxNTYyNSBDIDQuNTY2NDA2IC0yLjg3ODkwNiA0LjY3OTY4OCAtMi43MTg3NSA0%0D%0ALjc2NTYyNSAtMi41MzEyNSBDIDQuODU5Mzc1IC0yLjM0Mzc1IDQuOTA2MjUgLTIuMTEzMjgxIDQu%0D%0AOTA2MjUgLTEuODQzNzUgWiBNIDQuOTA2MjUgLTEuODQzNzUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9s%0D%0AIG92ZXJmbG93PSJ2aXNpYmxlIiBpZD0iZ2x5cGgyLTQiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5v%0D%0AbmU7IiBkPSIiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlw%0D%0AaDItNSI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0gMy4zOTA2MjUgLTkuNDIxODc1%0D%0AIEMgMi45NTMxMjUgLTguNTAzOTA2IDIuNjI1IC03LjU2MjUgMi40MDYyNSAtNi41OTM3NSBDIDIu%0D%0AMTk1MzEyIC01LjYzMjgxMiAyLjA5Mzc1IC00LjY2NDA2MiAyLjA5Mzc1IC0zLjY4NzUgQyAyLjA5%0D%0AMzc1IC0yLjcwNzAzMSAyLjIwMzEyNSAtMS43MzgyODEgMi40MjE4NzUgLTAuNzgxMjUgQyAyLjY0%0D%0AMDYyNSAwLjE3NTc4MSAyLjk1NzAzMSAxLjExNzE4OCAzLjM3NSAyLjA0Njg3NSBDIDMuMzk0NTMx%0D%0AIDIuMTA5Mzc1IDMuNDA2MjUgMi4xNTYyNSAzLjQwNjI1IDIuMTg3NSBDIDMuNDA2MjUgMi4yMjY1%0D%0ANjIgMy4zOTA2MjUgMi4yNTc4MTIgMy4zNTkzNzUgMi4yODEyNSBDIDMuMzI4MTI1IDIuMzEyNSAz%0D%0ALjI4MTI1IDIuMzMyMDMxIDMuMjE4NzUgMi4zNDM3NSBDIDMuMTU2MjUgMi4zNTE1NjIgMy4wNjY0%0D%0AMDYgMi4zNTkzNzUgMi45NTMxMjUgMi4zNTkzNzUgQyAyLjg2NzE4OCAyLjM1OTM3NSAyLjc5Njg3%0D%0ANSAyLjM1MTU2MiAyLjczNDM3NSAyLjM0Mzc1IEMgMi42NzE4NzUgMi4zNDM3NSAyLjYxNzE4OCAy%0D%0ALjMzMjAzMSAyLjU3ODEyNSAyLjMxMjUgQyAyLjUzNTE1NiAyLjMwMDc4MSAyLjUgMi4yODUxNTYg%0D%0AMi40Njg3NSAyLjI2NTYyNSBDIDIuNDQ1MzEyIDIuMjUzOTA2IDIuNDI1NzgxIDIuMjM4MjgxIDIu%0D%0ANDA2MjUgMi4yMTg3NSBDIDIuMTY0MDYyIDEuNzUgMS45NTMxMjUgMS4yNzM0MzggMS43NjU2MjUg%0D%0AMC43OTY4NzUgQyAxLjU3ODEyNSAwLjMyODEyNSAxLjQxNDA2MiAtMC4xNDg0MzggMS4yODEyNSAt%0D%0AMC42NDA2MjUgQyAxLjE1NjI1IC0xLjE0MDYyNSAxLjA2MjUgLTEuNjQwNjI1IDEgLTIuMTQwNjI1%0D%0AIEMgMC45Mzc1IC0yLjY0ODQzOCAwLjkwNjI1IC0zLjE2MDE1NiAwLjkwNjI1IC0zLjY3MTg3NSBD%0D%0AIDAuOTA2MjUgLTQuMTc5Njg4IDAuOTQxNDA2IC00LjY4NzUgMS4wMTU2MjUgLTUuMTg3NSBDIDEu%0D%0AMDg1OTM4IC01LjY5NTMxMiAxLjE4NzUgLTYuMTk1MzEyIDEuMzEyNSAtNi42ODc1IEMgMS40NDUz%0D%0AMTIgLTcuMTg3NSAxLjYwOTM3NSAtNy42NzE4NzUgMS43OTY4NzUgLTguMTQwNjI1IEMgMS45ODQz%0D%0ANzUgLTguNjE3MTg4IDIuMTkxNDA2IC05LjA5NzY1NiAyLjQyMTg3NSAtOS41NzgxMjUgQyAyLjQy%0D%0AMTg3NSAtOS41ODU5MzggMi40Mjk2ODggLTkuNTk3NjU2IDIuNDUzMTI1IC05LjYwOTM3NSBDIDIu%0D%0ANDcyNjU2IC05LjYyODkwNiAyLjUgLTkuNjQ0NTMxIDIuNTMxMjUgLTkuNjU2MjUgQyAyLjU3MDMx%0D%0AMiAtOS42NzU3ODEgMi42MjUgLTkuNjg3NSAyLjY4NzUgLTkuNjg3NSBDIDIuNzU3ODEyIC05LjY5%0D%0ANTMxMiAyLjg0Mzc1IC05LjcwMzEyNSAyLjkzNzUgLTkuNzAzMTI1IEMgMy4wMzkwNjIgLTkuNzAz%0D%0AMTI1IDMuMTI1IC05LjY5MTQwNiAzLjE4NzUgLTkuNjcxODc1IEMgMy4yNSAtOS42NjAxNTYgMy4y%0D%0AOTY4NzUgLTkuNjQ0NTMxIDMuMzI4MTI1IC05LjYyNSBDIDMuMzY3MTg4IC05LjYwMTU2MiAzLjM5%0D%0AMDYyNSAtOS41NzAzMTIgMy4zOTA2MjUgLTkuNTMxMjUgQyAzLjM5ODQzOCAtOS41IDMuMzk4NDM4%0D%0AIC05LjQ2MDkzOCAzLjM5MDYyNSAtOS40MjE4NzUgWiBNIDMuMzkwNjI1IC05LjQyMTg3NSAiLz4K%0D%0APC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlwaDItNiI+CjxwYXRo%0D%0AIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9Ik0gNi4yOTY4NzUgLTMuNTE1NjI1IEMgNi4yOTY4NzUg%0D%0ALTMuMzM1OTM4IDYuMjUgLTMuMjA3MDMxIDYuMTU2MjUgLTMuMTI1IEMgNi4wNzAzMTIgLTMuMDUw%0D%0ANzgxIDUuOTcyNjU2IC0zLjAxNTYyNSA1Ljg1OTM3NSAtMy4wMTU2MjUgTCAxLjgyODEyNSAtMy4w%0D%0AMTU2MjUgQyAxLjgyODEyNSAtMi42Nzk2ODggMS44NTkzNzUgLTIuMzc1IDEuOTIxODc1IC0yLjA5%0D%0AMzc1IEMgMS45OTIxODggLTEuODIwMzEyIDIuMTA5Mzc1IC0xLjU4NTkzOCAyLjI2NTYyNSAtMS4z%0D%0AOTA2MjUgQyAyLjQyOTY4OCAtMS4xOTE0MDYgMi42NDA2MjUgLTEuMDM5MDYyIDIuODkwNjI1IC0w%0D%0ALjkzNzUgQyAzLjE0ODQzOCAtMC44MzIwMzEgMy40NjA5MzggLTAuNzgxMjUgMy44MjgxMjUgLTAu%0D%0ANzgxMjUgQyA0LjExNzE4OCAtMC43ODEyNSA0LjM3ODkwNiAtMC44MDQ2ODggNC42MDkzNzUgLTAu%0D%0AODU5Mzc1IEMgNC44MzU5MzggLTAuOTEwMTU2IDUuMDM1MTU2IC0wLjk2MDkzOCA1LjIwMzEyNSAt%0D%0AMS4wMTU2MjUgQyA1LjM2NzE4OCAtMS4wNzgxMjUgNS41MDM5MDYgLTEuMTI4OTA2IDUuNjA5Mzc1%0D%0AIC0xLjE3MTg3NSBDIDUuNzEwOTM4IC0xLjIyMjY1NiA1Ljc5Njg3NSAtMS4yNSA1Ljg1OTM3NSAt%0D%0AMS4yNSBDIDUuODkwNjI1IC0xLjI1IDUuOTE0MDYyIC0xLjIzODI4MSA1LjkzNzUgLTEuMjE4NzUg%0D%0AQyA1Ljk1NzAzMSAtMS4yMDcwMzEgNS45NzI2NTYgLTEuMTg3NSA1Ljk4NDM3NSAtMS4xNTYyNSBD%0D%0AIDYuMDAzOTA2IC0xLjEyNSA2LjAxNTYyNSAtMS4wNzgxMjUgNi4wMTU2MjUgLTEuMDE1NjI1IEMg%0D%0ANi4wMjM0MzggLTAuOTYwOTM4IDYuMDMxMjUgLTAuODk0NTMxIDYuMDMxMjUgLTAuODEyNSBDIDYu%0D%0AMDMxMjUgLTAuNzUgNi4wMjM0MzggLTAuNjk1MzEyIDYuMDE1NjI1IC0wLjY1NjI1IEMgNi4wMTU2%0D%0AMjUgLTAuNjEzMjgxIDYuMDA3ODEyIC0wLjU3ODEyNSA2IC0wLjU0Njg3NSBDIDYgLTAuNTE1NjI1%0D%0AIDUuOTg4MjgxIC0wLjQ4NDM3NSA1Ljk2ODc1IC0wLjQ1MzEyNSBDIDUuOTU3MDMxIC0wLjQyMTg3%0D%0ANSA1LjkzNzUgLTAuMzkwNjI1IDUuOTA2MjUgLTAuMzU5Mzc1IEMgNS44ODI4MTIgLTAuMzM1OTM4%0D%0AIDUuODEyNSAtMC4zMDA3ODEgNS42ODc1IC0wLjI1IEMgNS41NzAzMTIgLTAuMTk1MzEyIDUuNDE0%0D%0AMDYyIC0wLjE0MDYyNSA1LjIxODc1IC0wLjA3ODEyNSBDIDUuMDE5NTMxIC0wLjAyMzQzNzUgNC43%0D%0AODkwNjIgMC4wMTk1MzEyIDQuNTMxMjUgMC4wNjI1IEMgNC4yODEyNSAwLjEwMTU2MiA0LjAwNzgx%0D%0AMiAwLjEyNSAzLjcxODc1IDAuMTI1IEMgMy4yMDcwMzEgMC4xMjUgMi43NTc4MTIgMC4wNTQ2ODc1%0D%0AIDIuMzc1IC0wLjA3ODEyNSBDIDIgLTAuMjIyNjU2IDEuNjc5Njg4IC0wLjQyOTY4OCAxLjQyMTg3%0D%0ANSAtMC43MDMxMjUgQyAxLjE2MDE1NiAtMC45ODQzNzUgMC45NjA5MzggLTEuMzMyMDMxIDAuODI4%0D%0AMTI1IC0xLjc1IEMgMC43MDMxMjUgLTIuMTc1NzgxIDAuNjQwNjI1IC0yLjY2NDA2MiAwLjY0MDYy%0D%0ANSAtMy4yMTg3NSBDIDAuNjQwNjI1IC0zLjc1IDAuNzA3MDMxIC00LjIyMjY1NiAwLjg0Mzc1IC00%0D%0ALjY0MDYyNSBDIDAuOTc2NTYyIC01LjA2NjQwNiAxLjE3MTg3NSAtNS40MjU3ODEgMS40MjE4NzUg%0D%0ALTUuNzE4NzUgQyAxLjY3OTY4OCAtNi4wMDc4MTIgMS45OTIxODggLTYuMjM0Mzc1IDIuMzU5Mzc1%0D%0AIC02LjM5MDYyNSBDIDIuNzIyNjU2IC02LjU0Njg3NSAzLjEzMjgxMiAtNi42MjUgMy41OTM3NSAt%0D%0ANi42MjUgQyA0LjA3MDMxMiAtNi42MjUgNC40NzY1NjIgLTYuNTQ2ODc1IDQuODEyNSAtNi4zOTA2%0D%0AMjUgQyA1LjE1NjI1IC02LjI0MjE4OCA1LjQzNzUgLTYuMDM1MTU2IDUuNjU2MjUgLTUuNzY1NjI1%0D%0AIEMgNS44NzUgLTUuNTAzOTA2IDYuMDM1MTU2IC01LjE5NTMxMiA2LjE0MDYyNSAtNC44NDM3NSBD%0D%0AIDYuMjQyMTg4IC00LjUgNi4yOTY4NzUgLTQuMTI1IDYuMjk2ODc1IC0zLjcxODc1IFogTSA1LjE1%0D%0ANjI1IC0zLjg0Mzc1IEMgNS4xNzU3ODEgLTQuNDM3NSA1LjA0Njg3NSAtNC44OTg0MzggNC43NjU2%0D%0AMjUgLTUuMjM0Mzc1IEMgNC40OTIxODggLTUuNTc4MTI1IDQuMDg1OTM4IC01Ljc1IDMuNTQ2ODc1%0D%0AIC01Ljc1IEMgMy4yNjU2MjUgLTUuNzUgMy4wMTk1MzEgLTUuNjk1MzEyIDIuODEyNSAtNS41OTM3%0D%0ANSBDIDIuNjAxNTYyIC01LjQ4ODI4MSAyLjQyNTc4MSAtNS4zNDc2NTYgMi4yODEyNSAtNS4xNzE4%0D%0ANzUgQyAyLjE0NDUzMSAtNS4wMDM5MDYgMi4wMzUxNTYgLTQuODAwNzgxIDEuOTUzMTI1IC00LjU2%0D%0AMjUgQyAxLjg3ODkwNiAtNC4zMzIwMzEgMS44MzU5MzggLTQuMDkzNzUgMS44MjgxMjUgLTMuODQz%0D%0ANzUgWiBNIDUuMTU2MjUgLTMuODQzNzUgIi8+Cjwvc3ltYm9sPgo8c3ltYm9sIG92ZXJmbG93PSJ2%0D%0AaXNpYmxlIiBpZD0iZ2x5cGgyLTciPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDYu%0D%0ANzAzMTI1IC0zLjM0Mzc1IEMgNi43MDMxMjUgLTIuODAwNzgxIDYuNjQ0NTMxIC0yLjMxNjQwNiA2%0D%0ALjUzMTI1IC0xLjg5MDYyNSBDIDYuNDE0MDYyIC0xLjQ2MDkzOCA2LjI0MjE4OCAtMS4wOTc2NTYg%0D%0ANi4wMTU2MjUgLTAuNzk2ODc1IEMgNS43OTY4NzUgLTAuNTAzOTA2IDUuNTE5NTMxIC0wLjI3MzQz%0D%0AOCA1LjE4NzUgLTAuMTA5Mzc1IEMgNC44NTE1NjIgMC4wNDY4NzUgNC40NzI2NTYgMC4xMjUgNC4w%0D%0ANDY4NzUgMC4xMjUgQyAzLjg1OTM3NSAwLjEyNSAzLjY4NzUgMC4xMDE1NjIgMy41MzEyNSAwLjA2%0D%0AMjUgQyAzLjM3NSAwLjAzMTI1IDMuMjIyNjU2IC0wLjAxOTUzMTIgMy4wNzgxMjUgLTAuMDkzNzUg%0D%0AQyAyLjkyOTY4OCAtMC4xNzU3ODEgMi43ODUxNTYgLTAuMjczNDM4IDIuNjQwNjI1IC0wLjM5MDYy%0D%0ANSBDIDIuNDkyMTg4IC0wLjUwMzkwNiAyLjMzNTkzOCAtMC42NDQ1MzEgMi4xNzE4NzUgLTAuODEy%0D%0ANSBMIDIuMTcxODc1IDIuMjY1NjI1IEMgMi4xNzE4NzUgMi4zMDQ2ODggMi4xNjAxNTYgMi4zMzU5%0D%0AMzggMi4xNDA2MjUgMi4zNTkzNzUgQyAyLjEyODkwNiAyLjM5MDYyNSAyLjEwMTU2MiAyLjQxNDA2%0D%0AMiAyLjA2MjUgMi40Mzc1IEMgMi4wMTk1MzEgMi40NTcwMzEgMS45NjA5MzggMi40Njg3NSAxLjg5%0D%0AMDYyNSAyLjQ2ODc1IEMgMS44MTY0MDYgMi40NzY1NjIgMS43MjI2NTYgMi40ODQzNzUgMS42MDkz%0D%0ANzUgMi40ODQzNzUgQyAxLjQ5MjE4OCAyLjQ4NDM3NSAxLjM5ODQzOCAyLjQ3NjU2MiAxLjMyODEy%0D%0ANSAyLjQ2ODc1IEMgMS4yNTM5MDYgMi40Njg3NSAxLjE5NTMxMiAyLjQ1NzAzMSAxLjE1NjI1IDIu%0D%0ANDM3NSBDIDEuMTEzMjgxIDIuNDE0MDYyIDEuMDgyMDMxIDIuMzkwNjI1IDEuMDYyNSAyLjM1OTM3%0D%0ANSBDIDEuMDUwNzgxIDIuMzM1OTM4IDEuMDQ2ODc1IDIuMzA0Njg4IDEuMDQ2ODc1IDIuMjY1NjI1%0D%0AIEwgMS4wNDY4NzUgLTYuMzI4MTI1IEMgMS4wNDY4NzUgLTYuMzY3MTg4IDEuMDUwNzgxIC02LjM5%0D%0AODQzOCAxLjA2MjUgLTYuNDIxODc1IEMgMS4wODIwMzEgLTYuNDQxNDA2IDEuMTA5Mzc1IC02LjQ2%0D%0AMDkzOCAxLjE0MDYyNSAtNi40ODQzNzUgQyAxLjE3OTY4OCAtNi41MDM5MDYgMS4yMzQzNzUgLTYu%0D%0ANTE1NjI1IDEuMjk2ODc1IC02LjUxNTYyNSBDIDEuMzY3MTg4IC02LjUyMzQzOCAxLjQ0NTMxMiAt%0D%0ANi41MzEyNSAxLjUzMTI1IC02LjUzMTI1IEMgMS42MzI4MTIgLTYuNTMxMjUgMS43MTA5MzggLTYu%0D%0ANTIzNDM4IDEuNzY1NjI1IC02LjUxNTYyNSBDIDEuODI4MTI1IC02LjUxNTYyNSAxLjg3ODkwNiAt%0D%0ANi41MDM5MDYgMS45MjE4NzUgLTYuNDg0Mzc1IEMgMS45NjA5MzggLTYuNDYwOTM4IDEuOTg4Mjgx%0D%0AIC02LjQ0MTQwNiAyIC02LjQyMTg3NSBDIDIuMDE5NTMxIC02LjM5ODQzOCAyLjAzMTI1IC02LjM2%0D%0ANzE4OCAyLjAzMTI1IC02LjMyODEyNSBMIDIuMDMxMjUgLTUuNSBDIDIuMjE4NzUgLTUuNjg3NSAy%0D%0ALjM5ODQzOCAtNS44NDc2NTYgMi41NzgxMjUgLTUuOTg0Mzc1IEMgMi43NTM5MDYgLTYuMTI4OTA2%0D%0AIDIuOTI1NzgxIC02LjI1IDMuMDkzNzUgLTYuMzQzNzUgQyAzLjI2OTUzMSAtNi40Mzc1IDMuNDUz%0D%0AMTI1IC02LjUwMzkwNiAzLjY0MDYyNSAtNi41NDY4NzUgQyAzLjgyODEyNSAtNi41OTc2NTYgNC4w%0D%0AMTk1MzEgLTYuNjI1IDQuMjE4NzUgLTYuNjI1IEMgNC42NjQwNjIgLTYuNjI1IDUuMDQ2ODc1IC02%0D%0ALjUzNTE1NiA1LjM1OTM3NSAtNi4zNTkzNzUgQyA1LjY3OTY4OCAtNi4xOTE0MDYgNS45NDE0MDYg%0D%0ALTUuOTU3MDMxIDYuMTQwNjI1IC01LjY1NjI1IEMgNi4zMzU5MzggLTUuMzUxNTYyIDYuNDc2NTYy%0D%0AIC01LjAwMzkwNiA2LjU2MjUgLTQuNjA5Mzc1IEMgNi42NTYyNSAtNC4yMTA5MzggNi43MDMxMjUg%0D%0ALTMuNzg5MDYyIDYuNzAzMTI1IC0zLjM0Mzc1IFogTSA1LjUxNTYyNSAtMy4yMTg3NSBDIDUuNTE1%0D%0ANjI1IC0zLjUzMTI1IDUuNDg4MjgxIC0zLjgzMjAzMSA1LjQzNzUgLTQuMTI1IEMgNS4zOTQ1MzEg%0D%0ALTQuNDE0MDYyIDUuMzE2NDA2IC00LjY3NTc4MSA1LjIwMzEyNSAtNC45MDYyNSBDIDUuMDg1OTM4%0D%0AIC01LjEzMjgxMiA0LjkyOTY4OCAtNS4zMTY0MDYgNC43MzQzNzUgLTUuNDUzMTI1IEMgNC41MzUx%0D%0ANTYgLTUuNTg1OTM4IDQuMjg5MDYyIC01LjY1NjI1IDQgLTUuNjU2MjUgQyAzLjg1MTU2MiAtNS42%0D%0ANTYyNSAzLjcwNzAzMSAtNS42MzI4MTIgMy41NjI1IC01LjU5Mzc1IEMgMy40MjU3ODEgLTUuNTUw%0D%0ANzgxIDMuMjg1MTU2IC01LjQ4NDM3NSAzLjE0MDYyNSAtNS4zOTA2MjUgQyAyLjk5MjE4OCAtNS4y%0D%0AOTY4NzUgMi44MzU5MzggLTUuMTcxODc1IDIuNjcxODc1IC01LjAxNTYyNSBDIDIuNTE1NjI1IC00%0D%0ALjg2NzE4OCAyLjM0NzY1NiAtNC42Nzk2ODggMi4xNzE4NzUgLTQuNDUzMTI1IEwgMi4xNzE4NzUg%0D%0ALTIuMDE1NjI1IEMgMi40ODQzNzUgLTEuNjI4OTA2IDIuNzczNDM4IC0xLjMzNTkzOCAzLjA0Njg3%0D%0ANSAtMS4xNDA2MjUgQyAzLjMyODEyNSAtMC45NDE0MDYgMy42MjUgLTAuODQzNzUgMy45Mzc1IC0w%0D%0ALjg0Mzc1IEMgNC4yMTg3NSAtMC44NDM3NSA0LjQ1NzAzMSAtMC45MTAxNTYgNC42NTYyNSAtMS4w%0D%0ANDY4NzUgQyA0Ljg2MzI4MSAtMS4xNzk2ODggNS4wMzEyNSAtMS4zNjMyODEgNS4xNTYyNSAtMS41%0D%0AOTM3NSBDIDUuMjgxMjUgLTEuODIwMzEyIDUuMzY3MTg4IC0yLjA3ODEyNSA1LjQyMTg3NSAtMi4z%0D%0ANTkzNzUgQyA1LjQ4NDM3NSAtMi42NDA2MjUgNS41MTU2MjUgLTIuOTI1NzgxIDUuNTE1NjI1IC0z%0D%0ALjIxODc1IFogTSA1LjUxNTYyNSAtMy4yMTg3NSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZs%0D%0Ab3c9InZpc2libGUiIGlkPSJnbHlwaDItOCI+CjxwYXRoIHN0eWxlPSJzdHJva2U6bm9uZTsiIGQ9%0D%0AIk0gNS40Njg3NSAtMS4xMDkzNzUgQyA1LjQ2ODc1IC0xLjAyMzQzOCA1LjQ2MDkzOCAtMC45NTcw%0D%0AMzEgNS40NTMxMjUgLTAuOTA2MjUgQyA1LjQ1MzEyNSAtMC44NTE1NjIgNS40NDUzMTIgLTAuODA0%0D%0ANjg4IDUuNDM3NSAtMC43NjU2MjUgQyA1LjQyNTc4MSAtMC43MjI2NTYgNS40MTAxNTYgLTAuNjg3%0D%0ANSA1LjM5MDYyNSAtMC42NTYyNSBDIDUuMzc4OTA2IC0wLjYyNSA1LjM0Mzc1IC0wLjU3ODEyNSA1%0D%0ALjI4MTI1IC0wLjUxNTYyNSBDIDUuMjI2NTYyIC0wLjQ2MDkzOCA1LjEzMjgxMiAtMC4zOTQ1MzEg%0D%0ANSAtMC4zMTI1IEMgNC44NjMyODEgLTAuMjI2NTYyIDQuNzA3MDMxIC0wLjE1NjI1IDQuNTMxMjUg%0D%0ALTAuMDkzNzUgQyA0LjM2MzI4MSAtMC4wMzEyNSA0LjE3NTc4MSAwLjAxOTUzMTIgMy45Njg3NSAw%0D%0ALjA2MjUgQyAzLjc2OTUzMSAwLjEwMTU2MiAzLjU2NjQwNiAwLjEyNSAzLjM1OTM3NSAwLjEyNSBD%0D%0AIDIuOTEwMTU2IDAuMTI1IDIuNTE1NjI1IDAuMDUwNzgxMiAyLjE3MTg3NSAtMC4wOTM3NSBDIDEu%0D%0AODM1OTM4IC0wLjIzODI4MSAxLjU1NDY4OCAtMC40NTMxMjUgMS4zMjgxMjUgLTAuNzM0Mzc1IEMg%0D%0AMS4wOTc2NTYgLTEuMDE1NjI1IDAuOTIxODc1IC0xLjM1OTM3NSAwLjc5Njg3NSAtMS43NjU2MjUg%0D%0AQyAwLjY3MTg3NSAtMi4xNzE4NzUgMC42MDkzNzUgLTIuNjQwNjI1IDAuNjA5Mzc1IC0zLjE3MTg3%0D%0ANSBDIDAuNjA5Mzc1IC0zLjc3MzQzOCAwLjY3OTY4OCAtNC4yOTY4NzUgMC44MjgxMjUgLTQuNzM0%0D%0AMzc1IEMgMC45ODQzNzUgLTUuMTcxODc1IDEuMTg3NSAtNS41MjM0MzggMS40Mzc1IC01Ljc5Njg3%0D%0ANSBDIDEuNjk1MzEyIC02LjA3ODEyNSAyIC02LjI4NTE1NiAyLjM0Mzc1IC02LjQyMTg3NSBDIDIu%0D%0ANjk1MzEyIC02LjU1NDY4OCAzLjA3ODEyNSAtNi42MjUgMy40ODQzNzUgLTYuNjI1IEMgMy42Nzk2%0D%0AODggLTYuNjI1IDMuODY3MTg4IC02LjYwMTU2MiA0LjA0Njg3NSAtNi41NjI1IEMgNC4yMzQzNzUg%0D%0ALTYuNTMxMjUgNC40MDYyNSAtNi40ODQzNzUgNC41NjI1IC02LjQyMTg3NSBDIDQuNzE4NzUgLTYu%0D%0AMzY3MTg4IDQuODUxNTYyIC02LjMwMDc4MSA0Ljk2ODc1IC02LjIxODc1IEMgNS4wOTM3NSAtNi4x%0D%0ANDQ1MzEgNS4xNzk2ODggLTYuMDc4MTI1IDUuMjM0Mzc1IC02LjAxNTYyNSBDIDUuMjk2ODc1IC01%0D%0ALjk2MDkzOCA1LjMzNTkzOCAtNS45MjE4NzUgNS4zNTkzNzUgLTUuODkwNjI1IEMgNS4zNzg5MDYg%0D%0ALTUuODU5Mzc1IDUuMzk0NTMxIC01LjgyMDMxMiA1LjQwNjI1IC01Ljc4MTI1IEMgNS40MTQwNjIg%0D%0ALTUuNzM4MjgxIDUuNDIxODc1IC01LjY4NzUgNS40MjE4NzUgLTUuNjI1IEMgNS40Mjk2ODggLTUu%0D%0ANTcwMzEyIDUuNDM3NSAtNS41MDM5MDYgNS40Mzc1IC01LjQyMTg3NSBDIDUuNDM3NSAtNS4yNDIx%0D%0AODggNS40MTQwNjIgLTUuMTE3MTg4IDUuMzc1IC01LjA0Njg3NSBDIDUuMzMyMDMxIC00Ljk4NDM3%0D%0ANSA1LjI4NTE1NiAtNC45NTMxMjUgNS4yMzQzNzUgLTQuOTUzMTI1IEMgNS4xNjAxNTYgLTQuOTUz%0D%0AMTI1IDUuMDc4MTI1IC00Ljk4ODI4MSA0Ljk4NDM3NSAtNS4wNjI1IEMgNC44OTg0MzggLTUuMTMy%0D%0AODEyIDQuNzg5MDYyIC01LjIxODc1IDQuNjU2MjUgLTUuMzEyNSBDIDQuNTE5NTMxIC01LjQwNjI1%0D%0AIDQuMzUxNTYyIC01LjQ4ODI4MSA0LjE1NjI1IC01LjU2MjUgQyAzLjk2ODc1IC01LjYzMjgxMiAz%0D%0ALjczODI4MSAtNS42NzE4NzUgMy40Njg3NSAtNS42NzE4NzUgQyAyLjkyNTc4MSAtNS42NzE4NzUg%0D%0AMi41MDc4MTIgLTUuNDYwOTM4IDIuMjE4NzUgLTUuMDQ2ODc1IEMgMS45Mzc1IC00LjYyODkwNiAx%0D%0ALjc5Njg3NSAtNC4wMjM0MzggMS43OTY4NzUgLTMuMjM0Mzc1IEMgMS43OTY4NzUgLTIuODM1OTM4%0D%0AIDEuODMyMDMxIC0yLjQ4ODI4MSAxLjkwNjI1IC0yLjE4NzUgQyAxLjk3NjU2MiAtMS44OTQ1MzEg%0D%0AMi4wODU5MzggLTEuNjQ0NTMxIDIuMjM0Mzc1IC0xLjQzNzUgQyAyLjM3ODkwNiAtMS4yMzgyODEg%0D%0AMi41NTQ2ODggLTEuMDg1OTM4IDIuNzY1NjI1IC0wLjk4NDM3NSBDIDIuOTg0Mzc1IC0wLjg5MDYy%0D%0ANSAzLjIyNjU2MiAtMC44NDM3NSAzLjUgLTAuODQzNzUgQyAzLjc1NzgxMiAtMC44NDM3NSAzLjk4%0D%0AODI4MSAtMC44ODI4MTIgNC4xODc1IC0wLjk2ODc1IEMgNC4zODI4MTIgLTEuMDUwNzgxIDQuNTUw%0D%0ANzgxIC0xLjE0MDYyNSA0LjY4NzUgLTEuMjM0Mzc1IEMgNC44MzIwMzEgLTEuMzM1OTM4IDQuOTUz%0D%0AMTI1IC0xLjQyNTc4MSA1LjA0Njg3NSAtMS41IEMgNS4xNDg0MzggLTEuNTgyMDMxIDUuMjI2NTYy%0D%0AIC0xLjYyNSA1LjI4MTI1IC0xLjYyNSBDIDUuMzEyNSAtMS42MjUgNS4zMzU5MzggLTEuNjEzMjgx%0D%0AIDUuMzU5Mzc1IC0xLjU5Mzc1IEMgNS4zOTA2MjUgLTEuNTgyMDMxIDUuNDEwMTU2IC0xLjU1MDc4%0D%0AMSA1LjQyMTg3NSAtMS41IEMgNS40NDE0MDYgLTEuNDU3MDMxIDUuNDUzMTI1IC0xLjQwNjI1IDUu%0D%0ANDUzMTI1IC0xLjM0Mzc1IEMgNS40NjA5MzggLTEuMjgxMjUgNS40Njg3NSAtMS4yMDMxMjUgNS40%0D%0ANjg3NSAtMS4xMDkzNzUgWiBNIDUuNDY4NzUgLTEuMTA5Mzc1ICIvPgo8L3N5bWJvbD4KPHN5bWJv%0D%0AbCBvdmVyZmxvdz0idmlzaWJsZSIgaWQ9ImdseXBoMi05Ij4KPHBhdGggc3R5bGU9InN0cm9rZTpu%0D%0Ab25lOyIgZD0iTSA2LjM0Mzc1IC0wLjE3MTg3NSBDIDYuMzQzNzUgLTAuMTI4OTA2IDYuMzMyMDMx%0D%0AIC0wLjA5NzY1NjIgNi4zMTI1IC0wLjA3ODEyNSBDIDYuMzAwNzgxIC0wLjA1NDY4NzUgNi4yNzM0%0D%0AMzggLTAuMDM1MTU2MiA2LjIzNDM3NSAtMC4wMTU2MjUgQyA2LjE5MTQwNiAwLjAwMzkwNjI1IDYu%0D%0AMTMyODEyIDAuMDE5NTMxMiA2LjA2MjUgMC4wMzEyNSBDIDUuOTg4MjgxIDAuMDM5MDYyNSA1Ljg5%0D%0ANDUzMSAwLjA0Njg3NSA1Ljc4MTI1IDAuMDQ2ODc1IEMgNS42NjQwNjIgMC4wNDY4NzUgNS41NzAz%0D%0AMTIgMC4wMzkwNjI1IDUuNSAwLjAzMTI1IEMgNS40MjU3ODEgMC4wMTk1MzEyIDUuMzY3MTg4IDAu%0D%0AMDAzOTA2MjUgNS4zMjgxMjUgLTAuMDE1NjI1IEMgNS4yODUxNTYgLTAuMDM1MTU2MiA1LjI1Mzkw%0D%0ANiAtMC4wNTQ2ODc1IDUuMjM0Mzc1IC0wLjA3ODEyNSBDIDUuMjIyNjU2IC0wLjA5NzY1NjIgNS4y%0D%0AMTg3NSAtMC4xMjg5MDYgNS4yMTg3NSAtMC4xNzE4NzUgTCA1LjIxODc1IC0zLjc4MTI1IEMgNS4y%0D%0AMTg3NSAtNC4xMjUgNS4xODc1IC00LjM5ODQzOCA1LjEyNSAtNC42MDkzNzUgQyA1LjA3MDMxMiAt%0D%0ANC44MjgxMjUgNC45OTIxODggLTUuMDE1NjI1IDQuODkwNjI1IC01LjE3MTg3NSBDIDQuNzg1MTU2%0D%0AIC01LjMyODEyNSA0LjY0ODQzOCAtNS40NDUzMTIgNC40ODQzNzUgLTUuNTMxMjUgQyA0LjMxNjQw%0D%0ANiAtNS42MTMyODEgNC4xMjUgLTUuNjU2MjUgMy45MDYyNSAtNS42NTYyNSBDIDMuNjI1IC01LjY1%0D%0ANjI1IDMuMzQzNzUgLTUuNTUwNzgxIDMuMDYyNSAtNS4zNDM3NSBDIDIuNzgxMjUgLTUuMTQ0NTMx%0D%0AIDIuNDg0Mzc1IC00Ljg1MTU2MiAyLjE3MTg3NSAtNC40Njg3NSBMIDIuMTcxODc1IC0wLjE3MTg3%0D%0ANSBDIDIuMTcxODc1IC0wLjEyODkwNiAyLjE2MDE1NiAtMC4wOTc2NTYyIDIuMTQwNjI1IC0wLjA3%0D%0AODEyNSBDIDIuMTI4OTA2IC0wLjA1NDY4NzUgMi4xMDE1NjIgLTAuMDM1MTU2MiAyLjA2MjUgLTAu%0D%0AMDE1NjI1IEMgMi4wMTk1MzEgMC4wMDM5MDYyNSAxLjk2MDkzOCAwLjAxOTUzMTIgMS44OTA2MjUg%0D%0AMC4wMzEyNSBDIDEuODE2NDA2IDAuMDM5MDYyNSAxLjcyMjY1NiAwLjA0Njg3NSAxLjYwOTM3NSAw%0D%0ALjA0Njg3NSBDIDEuNDkyMTg4IDAuMDQ2ODc1IDEuMzk4NDM4IDAuMDM5MDYyNSAxLjMyODEyNSAw%0D%0ALjAzMTI1IEMgMS4yNTM5MDYgMC4wMTk1MzEyIDEuMTk1MzEyIDAuMDAzOTA2MjUgMS4xNTYyNSAt%0D%0AMC4wMTU2MjUgQyAxLjExMzI4MSAtMC4wMzUxNTYyIDEuMDgyMDMxIC0wLjA1NDY4NzUgMS4wNjI1%0D%0AIC0wLjA3ODEyNSBDIDEuMDUwNzgxIC0wLjA5NzY1NjIgMS4wNDY4NzUgLTAuMTI4OTA2IDEuMDQ2%0D%0AODc1IC0wLjE3MTg3NSBMIDEuMDQ2ODc1IC05LjI5Njg3NSBDIDEuMDQ2ODc1IC05LjMzNTkzOCAx%0D%0ALjA1MDc4MSAtOS4zNjcxODggMS4wNjI1IC05LjM5MDYyNSBDIDEuMDgyMDMxIC05LjQyMTg3NSAx%0D%0ALjExMzI4MSAtOS40NDUzMTIgMS4xNTYyNSAtOS40Njg3NSBDIDEuMTk1MzEyIC05LjQ4ODI4MSAx%0D%0ALjI1MzkwNiAtOS41IDEuMzI4MTI1IC05LjUgQyAxLjM5ODQzOCAtOS41MDc4MTIgMS40OTIxODgg%0D%0ALTkuNTE1NjI1IDEuNjA5Mzc1IC05LjUxNTYyNSBDIDEuNzIyNjU2IC05LjUxNTYyNSAxLjgxNjQw%0D%0ANiAtOS41MDc4MTIgMS44OTA2MjUgLTkuNSBDIDEuOTYwOTM4IC05LjUgMi4wMTk1MzEgLTkuNDg4%0D%0AMjgxIDIuMDYyNSAtOS40Njg3NSBDIDIuMTAxNTYyIC05LjQ0NTMxMiAyLjEyODkwNiAtOS40MjE4%0D%0ANzUgMi4xNDA2MjUgLTkuMzkwNjI1IEMgMi4xNjAxNTYgLTkuMzY3MTg4IDIuMTcxODc1IC05LjMz%0D%0ANTkzOCAyLjE3MTg3NSAtOS4yOTY4NzUgTCAyLjE3MTg3NSAtNS42MjUgQyAyLjQ5MjE4OCAtNS45%0D%0ANTcwMzEgMi44MTY0MDYgLTYuMjA3MDMxIDMuMTQwNjI1IC02LjM3NSBDIDMuNDcyNjU2IC02LjUz%0D%0AOTA2MiAzLjgwNDY4OCAtNi42MjUgNC4xNDA2MjUgLTYuNjI1IEMgNC41NTQ2ODggLTYuNjI1IDQu%0D%0AODk4NDM4IC02LjU1NDY4OCA1LjE3MTg3NSAtNi40MjE4NzUgQyA1LjQ1MzEyNSAtNi4yODUxNTYg%0D%0ANS42Nzk2ODggLTYuMDk3NjU2IDUuODU5Mzc1IC01Ljg1OTM3NSBDIDYuMDM1MTU2IC01LjYyODkw%0D%0ANiA2LjE2MDE1NiAtNS4zNTkzNzUgNi4yMzQzNzUgLTUuMDQ2ODc1IEMgNi4zMDQ2ODggLTQuNzM0%0D%0AMzc1IDYuMzQzNzUgLTQuMzU5Mzc1IDYuMzQzNzUgLTMuOTIxODc1IFogTSA2LjM0Mzc1IC0wLjE3%0D%0AMTg3NSAiLz4KPC9zeW1ib2w+CjxzeW1ib2wgb3ZlcmZsb3c9InZpc2libGUiIGlkPSJnbHlwaDIt%0D%0AMTAiPgo8cGF0aCBzdHlsZT0ic3Ryb2tlOm5vbmU7IiBkPSJNIDMuMzI4MTI1IC0zLjY3MTg3NSBD%0D%0AIDMuMzI4MTI1IC0zLjE2MDE1NiAzLjI5Njg3NSAtMi42NDg0MzggMy4yMzQzNzUgLTIuMTQwNjI1%0D%0AIEMgMy4xNzE4NzUgLTEuNjQwNjI1IDMuMDcwMzEyIC0xLjE0MDYyNSAyLjkzNzUgLTAuNjQwNjI1%0D%0AIEMgMi44MTI1IC0wLjE0ODQzOCAyLjY1NjI1IDAuMzI4MTI1IDIuNDY4NzUgMC43OTY4NzUgQyAy%0D%0ALjI4OTA2MiAxLjI3MzQzOCAyLjA3ODEyNSAxLjc1IDEuODI4MTI1IDIuMjE4NzUgQyAxLjgxNjQw%0D%0ANiAyLjIzODI4MSAxLjc5Njg3NSAyLjI1MzkwNiAxLjc2NTYyNSAyLjI2NTYyNSBDIDEuNzQyMTg4%0D%0AIDIuMjg1MTU2IDEuNzA3MDMxIDIuMzAwNzgxIDEuNjU2MjUgMi4zMTI1IEMgMS42MTMyODEgMi4z%0D%0AMzIwMzEgMS41NjI1IDIuMzQzNzUgMS41IDIuMzQzNzUgQyAxLjQ0NTMxMiAyLjM1MTU2MiAxLjM3%0D%0AODkwNiAyLjM1OTM3NSAxLjI5Njg3NSAyLjM1OTM3NSBDIDEuMTc5Njg4IDIuMzU5Mzc1IDEuMDg1%0D%0AOTM4IDIuMzUxNTYyIDEuMDE1NjI1IDIuMzQzNzUgQyAwLjk1MzEyNSAyLjMzMjAzMSAwLjkwNjI1%0D%0AIDIuMzEyNSAwLjg3NSAyLjI4MTI1IEMgMC44NDM3NSAyLjI1NzgxMiAwLjgyODEyNSAyLjIyNjU2%0D%0AMiAwLjgyODEyNSAyLjE4NzUgQyAwLjgzNTkzOCAyLjE1NjI1IDAuODUxNTYyIDIuMTA5Mzc1IDAu%0D%0AODc1IDIuMDQ2ODc1IEMgMS4yODkwNjIgMS4xMjg5MDYgMS42MDE1NjIgMC4xOTE0MDYgMS44MTI1%0D%0AIC0wLjc2NTYyNSBDIDIuMDMxMjUgLTEuNzM0Mzc1IDIuMTQwNjI1IC0yLjcwNzAzMSAyLjE0MDYy%0D%0ANSAtMy42ODc1IEMgMi4xNDA2MjUgLTQuNjY0MDYyIDIuMDMxMjUgLTUuNjMyODEyIDEuODEyNSAt%0D%0ANi41OTM3NSBDIDEuNjAxNTYyIC03LjU2MjUgMS4yODUxNTYgLTguNTAzOTA2IDAuODU5Mzc1IC05%0D%0ALjQyMTg3NSBDIDAuODM1OTM4IC05LjQ2MDkzOCAwLjgyODEyNSAtOS41IDAuODI4MTI1IC05LjUz%0D%0AMTI1IEMgMC44MzU5MzggLTkuNTcwMzEyIDAuODU5Mzc1IC05LjYwMTU2MiAwLjg5MDYyNSAtOS42%0D%0AMjUgQyAwLjkyOTY4OCAtOS42NDQ1MzEgMC45ODQzNzUgLTkuNjYwMTU2IDEuMDQ2ODc1IC05LjY3%0D%0AMTg3NSBDIDEuMTE3MTg4IC05LjY5MTQwNiAxLjIwNzAzMSAtOS43MDMxMjUgMS4zMTI1IC05Ljcw%0D%0AMzEyNSBDIDEuNDA2MjUgLTkuNzAzMTI1IDEuNDg0Mzc1IC05LjY5NTMxMiAxLjU0Njg3NSAtOS42%0D%0AODc1IEMgMS42MDkzNzUgLTkuNjg3NSAxLjY1NjI1IC05LjY3NTc4MSAxLjY4NzUgLTkuNjU2MjUg%0D%0AQyAxLjcyNjU2MiAtOS42NDQ1MzEgMS43NTc4MTIgLTkuNjI4OTA2IDEuNzgxMjUgLTkuNjA5Mzc1%0D%0AIEMgMS44MDA3ODEgLTkuNTk3NjU2IDEuODE2NDA2IC05LjU4NTkzOCAxLjgyODEyNSAtOS41Nzgx%0D%0AMjUgQyAyLjI5Njg3NSAtOC42NDA2MjUgMi42NjAxNTYgLTcuNjc1NzgxIDIuOTIxODc1IC02LjY4%0D%0ANzUgQyAzLjE5MTQwNiAtNS42OTUzMTIgMy4zMjgxMjUgLTQuNjkxNDA2IDMuMzI4MTI1IC0zLjY3%0D%0AMTg3NSBaIE0gMy4zMjgxMjUgLTMuNjcxODc1ICIvPgo8L3N5bWJvbD4KPC9nPgo8Y2xpcFBhdGgg%0D%0AaWQ9ImNsaXAxIj4KICA8cGF0aCBkPSJNIDUwLjM5ODQzOCA1NCBMIDc4MyA1NCBMIDc4MyA1MzQg%0D%0ATCA1MC4zOTg0MzggNTM0IFogTSA1MC4zOTg0MzggNTQgIi8+CjwvY2xpcFBhdGg+CjxjbGlwUGF0%0D%0AaCBpZD0iY2xpcDIiPgogIDxwYXRoIGQ9Ik0gNTAuMzk4NDM4IDU0IEwgNzg0IDU0IEwgNzg0IDUz%0D%0ANCBMIDUwLjM5ODQzOCA1MzQgWiBNIDUwLjM5ODQzOCA1NCAiLz4KPC9jbGlwUGF0aD4KPC9kZWZz%0D%0APgo8ZyBpZD0ic3VyZmFjZTEiPgo8ZyBjbGlwLXBhdGg9InVybCgjY2xpcDEpIiBjbGlwLXJ1bGU9%0D%0AIm5vbnplcm8iPgo8cGF0aCBzdHlsZT0iIHN0cm9rZTpub25lO2ZpbGwtcnVsZTpldmVub2RkO2Zp%0D%0AbGw6cmdiKDEwMCUsMTAwJSwxMDAlKTtmaWxsLW9wYWNpdHk6MTsiIGQ9Ik0gNTAuMzk4NDM4IDUz%0D%0AMy4wNDY4NzUgTCA3ODIuNzgxMjUgNTMzLjA0Njg3NSBMIDc4Mi43ODEyNSA1My45OTYwOTQgTCA1%0D%0AMC4zOTg0MzggNTMuOTk2MDk0IFogTSA1MC4zOTg0MzggNTMzLjA0Njg3NSAiLz4KPC9nPgo8cGF0%0D%0AaCBzdHlsZT0iZmlsbDpub25lO3N0cm9rZS13aWR0aDowLjc1O3N0cm9rZS1saW5lY2FwOmJ1dHQ7%0D%0Ac3Ryb2tlLWxpbmVqb2luOnJvdW5kO3N0cm9rZTpyZ2IoODUuMDk5NzkyJSw4NS4wOTk3OTIlLDg1%0D%0ALjA5OTc5MiUpO3N0cm9rZS1vcGFjaXR5OjE7c3Ryb2tlLW1pdGVybGltaXQ6MTA7IiBkPSJNIDc0%0D%0ALjM2NzE4OCAxMzcuNzU4NTk0IEwgNzcxLjc4MTI1IDEzNy43NTg1OTQgTSA3NC4zNjcxODggMTkw%0D%0ALjA3ODkwNiBMIDc3MS43ODEyNSAxOTAuMDc4OTA2IE0gNzQuMzY3MTg4IDI0Mi4zOTkyMTkgTCA3%0D%0ANzEuNzgxMjUgMjQyLjM5OTIxOSBNIDc0LjM2NzE4OCAyOTQuNzE5NTMxIEwgNzcxLjc4MTI1IDI5%0D%0ANC43MTk1MzEgTSA3NC4zNjcxODggMzQ3LjAzOTg0NCBMIDc3MS43ODEyNSAzNDcuMDM5ODQ0IE0g%0D%0ANzQuMzY3MTg4IDM5OS4zNjAxNTYgTCA3NzEuNzgxMjUgMzk5LjM2MDE1NiBNIDc0LjM2NzE4OCA0%0D%0ANTEuNjgwNDY5IEwgNzcxLjc4MTI1IDQ1MS42ODA0NjkgTSA3NC4zNjcxODggNTA0LjExMDE1NiBM%0D%0AIDc3MS43ODEyNSA1MDQuMTEwMTU2ICIgdHJhbnNmb3JtPSJtYXRyaXgoMSwwLDAsLTEsMCw1OTUu%0D%0AMikiLz4KPHBhdGggc3R5bGU9ImZpbGw6bm9uZTtzdHJva2Utd2lkdGg6MC43NTtzdHJva2UtbGlu%0D%0AZWNhcDpidXR0O3N0cm9rZS1saW5lam9pbjpyb3VuZDtzdHJva2U6cmdiKDg1LjA5OTc5MiUsODUu%0D%0AMDk5NzkyJSw4NS4wOTk3OTIlKTtzdHJva2Utb3BhY2l0eToxO3N0cm9rZS1taXRlcmxpbWl0OjEw%0D%0AOyIgZD0iTSA3NC4zNjcxODggODUuMzUyMzQ0IEwgNzcxLjc4MTI1IDg1LjM1MjM0NCAiIHRyYW5z%0D%0AZm9ybT0ibWF0cml4KDEsMCwwLC0xLDAsNTk1LjIpIi8+CjxwYXRoIHN0eWxlPSJmaWxsOm5vbmU7%0D%0Ac3Ryb2tlLXdpZHRoOjIuMjU7c3Ryb2tlLWxpbmVjYXA6cm91bmQ7c3Ryb2tlLWxpbmVqb2luOnJv%0D%0AdW5kO3N0cm9rZTpyZ2IoMjYuNjk5ODI5JSw0NC42OTkwOTclLDc2Ljg5OTcxOSUpO3N0cm9rZS1v%0D%0AcGFjaXR5OjE7c3Ryb2tlLW1pdGVybGltaXQ6MTA7IiBkPSJNIDc5LjM1MTU2MiA0NDguNDEwOTM4%0D%0AIEwgODkuMjgxMjUgNDIzLjU5ODQzOCBMIDk5LjM1OTM3NSAzNzYuODAxNTYzIEwgMTA5LjE5OTIx%0D%0AOSAzNjAuNzE5NTMxIEwgMTE5LjI4MTI1IDM1Mi44MDE1NjMgTCAxMjkuMTIxMDk0IDM0Mi40ODEy%0D%0ANSBMIDEzOS4xOTkyMTkgMzM5LjM2MDE1NiBMIDE0OS4wMzkwNjIgMzMzLjEyMTg3NSBMIDE1OS4x%0D%0AMjEwOTQgMzI4LjA3ODkwNiBMIDE2OC45NjA5MzggMzIzLjAzOTg0NCBMIDE3OS4wMzkwNjIgMzIw%0D%0ALjM5OTIxOSBMIDE4OC44Nzg5MDYgMzE1LjEyMTg3NSBMIDE5OC45NjA5MzggMzE4LjAwMDc4MSBM%0D%0AIDIwOC44MDA3ODEgMzE0LjM5OTIxOSBMIDIxOC44Nzg5MDYgMzExLjc1ODU5NCBMIDIyOC43MTg3%0D%0ANSAzMDQuMzIxMDk0IEwgMjM4LjgwMDc4MSAzMDAuNzE5NTMxIEwgMjQ4LjY0MDYyNSAzMDAuMjM5%0D%0AMDYzIEwgMjU4LjcxODc1IDMwMC40ODEyNSBMIDI2OC41NTg1OTQgMjk2LjM5OTIxOSBMIDI3OC42%0D%0ANDA2MjUgMjkzLjAzOTg0NCBMIDI4OC40ODA0NjkgMjg5LjQzODI4MSBMIDI5OC41NTg1OTQgMjkw%0D%0ALjE2MDkzOCBMIDMwOC4zOTg0MzggMjkwLjg3OTY4OCBMIDMxOC40ODA0NjkgMjg5LjkxODc1IEwg%0D%0AMzI4LjMyMDMxMiAyODguNzE5NTMxIEwgMzM4LjM5ODQzOCAyODYuMzIxMDk0IEwgMzQ4LjIzODI4%0D%0AMSAyODMuNjgwNDY5IEwgMzU4LjMyMDMxMiAyODUuNTk4NDM4IEwgMzY4LjE2MDE1NiAyODIuNDgx%0D%0AMjUgTCAzNzguMjM4MjgxIDI4NC4xNjA5MzggTCAzODguMzIwMzEyIDI4MS4wMzk4NDQgTCAzOTgu%0D%0AMTYwMTU2IDI4MC41NTkzNzUgTCA0MDguMjM4MjgxIDI3OC4zOTkyMTkgTCA0MTguMDc4MTI1IDI3%0D%0AOS4xMjE4NzUgTCA0MjguMTYwMTU2IDI4MS4wMzk4NDQgTCA0MzggMjc1LjUyMDMxMyBMIDQ0OC4w%0D%0ANzgxMjUgMjc3LjY4MDQ2OSBMIDQ1Ny45MjE4NzUgMjc1Ljc1ODU5NCBMIDQ2OCAyNzcuNjgwNDY5%0D%0AIEwgNDc3LjgzOTg0NCAyNzUuNTIwMzEzIEwgNDg3LjkyMTg3NSAyNzEuMiBMIDQ5Ny43NjE3MTkg%0D%0AMjc0LjgwMTU2MyBMIDUwNy44Mzk4NDQgMjczLjg0MDYyNSBMIDUxNy42Nzk2ODggMjc2LjIzOTA2%0D%0AMyBMIDUyNy43NjE3MTkgMjY5LjQ4MTI1IEwgNTM3LjYwMTU2MiAyNzQuMDc4OTA2IEwgNTQ3LjY3%0D%0AOTY4OCAyNzAuNzE5NTMxIEwgNTU3LjUxOTUzMSAyNzguMTYwOTM4IEwgNTY3LjYwMTU2MiAyNzMu%0D%0AODQwNjI1IEwgNTc3LjQ0MTQwNiAyNzMuODQwNjI1IEwgNTg3LjUxOTUzMSAyNzcuMiBMIDU5Ny4z%0D%0ANTkzNzUgMjc3LjIgTCA2MDcuNDQxNDA2IDI3MS42ODA0NjkgTCA2MTcuMjgxMjUgMjc5LjM2MDE1%0D%0ANiBMIDYyNy4zNTkzNzUgMjgwLjMyMTA5NCBMIDYzNy4xOTkyMTkgMjc2LjcxOTUzMSBMIDY0Ny4y%0D%0AODEyNSAyNzQuODAxNTYzIEwgNjU3LjEyMTA5NCAyNzIuODc5Njg4IEwgNjY3LjE5OTIxOSAyNzUu%0D%0ANTIwMzEzIEwgNjc3LjAzOTA2MiAyNzAuNzE5NTMxIEwgNjg3LjEyMTA5NCAyODMuOTE4NzUgTCA2%0D%0AOTYuOTYwOTM4IDI3Ny42ODA0NjkgTCA3MDcuMDM5MDYyIDI3NS4yNzgxMjUgTCA3MTYuODc4OTA2%0D%0AIDI3NS4yNzgxMjUgTCA3MjYuOTYwOTM4IDI3Ni43MTk1MzEgTCA3MzYuODAwNzgxIDI3My41OTg0%0D%0AMzggTCA3NDYuODc4OTA2IDI3Ni45NjE3MTkgTCA3NTYuNzE4NzUgMjc5LjU5ODQzOCBMIDc2Ni44%0D%0AMDA3ODEgMjc4LjY0MTQwNiAiIHRyYW5zZm9ybT0ibWF0cml4KDEsMCwwLC0xLDAsNTk1LjIpIi8+%0D%0ACjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7Zmls%0D%0AbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDAtMSIgeD0iNjEuNDg4IiB5%0D%0APSI1MTIuNzEyIi8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAy%0D%0AJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBo%0D%0AMC0yIiB4PSI2MS40ODgiIHk9IjQ2MC4zNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44%0D%0AOTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhs%0D%0AaW5rOmhyZWY9IiNnbHlwaDAtMyIgeD0iNTYuOTI4IiB5PSI0MDcuOTgiLz4KICA8dXNlIHhsaW5r%0D%0AOmhyZWY9IiNnbHlwaDAtMSIgeD0iNjEuNDg4MDQiIHk9IjQwNy45OCIvPgo8L2c+CjxnIHN0eWxl%0D%0APSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5%0D%0AOjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDAtMyIgeD0iNTYuOTI4IiB5PSIzNTUuNjEi%0D%0ALz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDAtMiIgeD0iNjEuNDg4MDQiIHk9IjM1NS42MSIv%0D%0APgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAy%0D%0AJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDAtNCIgeD0iNTYu%0D%0AOTI4IiB5PSIzMDMuMjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDAtMSIgeD0iNjEuNDg4%0D%0AMDQiIHk9IjMwMy4yNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5%0D%0AOTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNn%0D%0AbHlwaDAtNCIgeD0iNTYuOTI4IiB5PSIyNTAuODciLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlw%0D%0AaDAtMiIgeD0iNjEuNDg4MDQiIHk9IjI1MC44NyIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigz%0D%0ANC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNl%0D%0AIHhsaW5rOmhyZWY9IiNnbHlwaDAtNSIgeD0iNTYuOTI4IiB5PSIxOTguNSIvPgogIDx1c2UgeGxp%0D%0Abms6aHJlZj0iI2dseXBoMC0xIiB4PSI2MS40ODgwNCIgeT0iMTk4LjUiLz4KPC9nPgo8ZyBzdHls%0D%0AZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0%0D%0AeToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgwLTUiIHg9IjU2LjkyOCIgeT0iMTQ2LjE0%0D%0AIi8+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgwLTIiIHg9IjYxLjQ4ODA0IiB5PSIxNDYuMTQi%0D%0ALz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkw%0D%0AMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgwLTYiIHg9IjU2%0D%0ALjkyOCIgeT0iOTMuNzciLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDAtMSIgeD0iNjEuNDg4%0D%0AMDQiIHk9IjkzLjc3Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5%0D%0AOTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2ds%0D%0AeXBoMS0xIiB4PSI3Ny4xMTIiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0%0D%0ALjg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2Ug%0D%0AeGxpbms6aHJlZj0iI2dseXBoMS0yIiB4PSI4Ny4wNzIiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5%0D%0AbGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNp%0D%0AdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0zIiB4PSI5Ny4wMzIiIHk9IjUyNC40%0D%0AIi8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5%0D%0AMDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS00IiB4PSIx%0D%0AMDYuOTkiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQu%0D%0AODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0i%0D%0AI2dseXBoMS01IiB4PSIxMTYuOTgiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdi%0D%0AKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1%0D%0Ac2UgeGxpbms6aHJlZj0iI2dseXBoMS02IiB4PSIxMjYuOTQiIHk9IjUyNC40Ii8+CjwvZz4KPGcg%0D%0Ac3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9w%0D%0AYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS03IiB4PSIxMzYuOSIgeT0iNTI0%0D%0ALjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5%0D%0AOTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTgiIHg9%0D%0AIjE0Ni44NiIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwz%0D%0ANC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVm%0D%0APSIjZ2x5cGgxLTkiIHg9IjE1Ni44NCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpy%0D%0AZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAg%0D%0APHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTEiIHg9IjE2NC41MiIgeT0iNTI0LjQiLz4KICA8dXNl%0D%0AIHhsaW5rOmhyZWY9IiNnbHlwaDEtMTAiIHg9IjE2OS4wODAwMDgiIHk9IjUyNC40Ii8+CjwvZz4K%0D%0APGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxs%0D%0ALW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0xIiB4PSIxNzQuNDgiIHk9%0D%0AIjUyNC40Ii8+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTEiIHg9IjE3OS4wNDAwMDgiIHk9%0D%0AIjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwz%0D%0ANC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0x%0D%0AIiB4PSIxODQuNDQiIHk9IjUyNC40Ii8+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTIiIHg9%0D%0AIjE4OS4wMDAwMDgiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkw%0D%0AMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6%0D%0AaHJlZj0iI2dseXBoMS0xIiB4PSIxOTQuNCIgeT0iNTI0LjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9%0D%0AIiNnbHlwaDEtMyIgeD0iMTk4Ljk4NDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmls%0D%0AbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+%0D%0ACiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTEiIHg9IjIwNC4zOCIgeT0iNTI0LjQiLz4KICA8%0D%0AdXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iMjA4Ljk0MDAwOCIgeT0iNTI0LjQiLz4KPC9n%0D%0APgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2Zp%0D%0AbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTEiIHg9IjIxNC4zNCIg%0D%0AeT0iNTI0LjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNSIgeD0iMjE4LjkwMDAwOCIg%0D%0AeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIl%0D%0ALDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgx%0D%0ALTEiIHg9IjIyNC4zIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS02IiB4%0D%0APSIyMjguODYwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5%0D%0AMDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5r%0D%0AOmhyZWY9IiNnbHlwaDEtMSIgeD0iMjM0LjI2IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJl%0D%0AZj0iI2dseXBoMS03IiB4PSIyMzguODIwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJm%0D%0AaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7%0D%0AIj4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMSIgeD0iMjQ0LjI1IiB5PSI1MjQuNCIvPgog%0D%0AIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS04IiB4PSIyNDguODEwMDA4IiB5PSI1MjQuNCIvPgo8%0D%0AL2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7%0D%0AZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMSIgeD0iMjU0LjIx%0D%0AIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS05IiB4PSIyNTguNzcwMDA4%0D%0AIiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkw%0D%0AMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlw%0D%0AaDEtMiIgeD0iMjY0LjE3IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0x%0D%0AMCIgeD0iMjY4LjczMDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQu%0D%0AODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4%0D%0AbGluazpocmVmPSIjZ2x5cGgxLTIiIHg9IjI3NC4xMyIgeT0iNTI0LjQiLz4KICA8dXNlIHhsaW5r%0D%0AOmhyZWY9IiNnbHlwaDEtMSIgeD0iMjc4LjY5MDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHls%0D%0AZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0%0D%0AeToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTIiIHg9IjI4NC4xMSIgeT0iNTI0LjQi%0D%0ALz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMiIgeD0iMjg4LjY3MDAwOCIgeT0iNTI0LjQi%0D%0ALz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkw%0D%0AMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTIiIHg9IjI5%0D%0ANC4wNyIgeT0iNTI0LjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMyIgeD0iMjk4LjYz%0D%0AMDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44%0D%0AOTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIj%0D%0AZ2x5cGgxLTIiIHg9IjMwNC4wMyIgeT0iNTI0LjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlw%0D%0AaDEtNCIgeD0iMzA4LjU5MDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2Io%0D%0AMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVz%0D%0AZSB4bGluazpocmVmPSIjZ2x5cGgxLTIiIHg9IjMxMy45OSIgeT0iNTI0LjQiLz4KICA8dXNlIHhs%0D%0AaW5rOmhyZWY9IiNnbHlwaDEtNSIgeD0iMzE4LjU1MDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBz%0D%0AdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3Bh%0D%0AY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTIiIHg9IjMyMy45NSIgeT0iNTI0%0D%0ALjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iMzI4LjUzNDAwOCIgeT0iNTI0%0D%0ALjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5%0D%0AOTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTIiIHg9%0D%0AIjMzMy45NCIgeT0iNTI0LjQiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNyIgeD0iMzM4%0D%0ALjUwMDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwz%0D%0ANC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVm%0D%0APSIjZ2x5cGgxLTIiIHg9IjM0My45IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2ds%0D%0AeXBoMS04IiB4PSIzNDguNDYwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJn%0D%0AYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8%0D%0AdXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMiIgeD0iMzUzLjg2IiB5PSI1MjQuNCIvPgogIDx1c2Ug%0D%0AeGxpbms6aHJlZj0iI2dseXBoMS05IiB4PSIzNTguNDIwMDA4IiB5PSI1MjQuNCIvPgo8L2c+Cjxn%0D%0AIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1v%0D%0AcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMyIgeD0iMzYzLjgyIiB5PSI1%0D%0AMjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0xMCIgeD0iMzY4LjM4MDAwOCIgeT0i%0D%0ANTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0%0D%0ALjg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTMi%0D%0AIHg9IjM3My44IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0xIiB4PSIz%0D%0ANzguMzYwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIl%0D%0ALDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhy%0D%0AZWY9IiNnbHlwaDEtMyIgeD0iMzgzLjc2IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0i%0D%0AI2dseXBoMS0yIiB4PSIzODguMzIwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxs%0D%0AOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4K%0D%0AICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMyIgeD0iMzkzLjcyIiB5PSI1MjQuNCIvPgogIDx1%0D%0Ac2UgeGxpbms6aHJlZj0iI2dseXBoMS0zIiB4PSIzOTguMjgwMDA4IiB5PSI1MjQuNCIvPgo8L2c+%0D%0ACjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7Zmls%0D%0AbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMyIgeD0iNDAzLjY4IiB5%0D%0APSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS00IiB4PSI0MDguMjQwMDA4IiB5%0D%0APSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUs%0D%0AMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEt%0D%0AMyIgeD0iNDEzLjY2IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS01IiB4%0D%0APSI0MTguMjIwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5%0D%0AMDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5r%0D%0AOmhyZWY9IiNnbHlwaDEtMyIgeD0iNDIzLjYyIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJl%0D%0AZj0iI2dseXBoMS02IiB4PSI0MjguMTgwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJm%0D%0AaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7%0D%0AIj4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMyIgeD0iNDMzLjU4IiB5PSI1MjQuNCIvPgog%0D%0AIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS03IiB4PSI0MzguMTQwMDA4IiB5PSI1MjQuNCIvPgo8%0D%0AL2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7%0D%0AZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtMyIgeD0iNDQzLjU0%0D%0AIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS04IiB4PSI0NDguMTAwMDA4%0D%0AIiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkw%0D%0AMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlw%0D%0AaDEtMyIgeD0iNDUzLjUiIHk9IjUyNC40Ii8+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTki%0D%0AIHg9IjQ1OC4wODQwMDgiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5%0D%0AOTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxp%0D%0Abms6aHJlZj0iI2dseXBoMS00IiB4PSI0NjMuNDkiIHk9IjUyNC40Ii8+CiAgPHVzZSB4bGluazpo%0D%0AcmVmPSIjZ2x5cGgxLTEwIiB4PSI0NjguMDUwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxl%0D%0APSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5%0D%0AOjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iNDczLjQ1IiB5PSI1MjQuNCIv%0D%0APgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0xIiB4PSI0NzguMDEwMDA4IiB5PSI1MjQuNCIv%0D%0APgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAy%0D%0AJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iNDgz%0D%0ALjQxIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0yIiB4PSI0ODcuOTcw%0D%0AMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5%0D%0AOTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNn%0D%0AbHlwaDEtNCIgeD0iNDkzLjM3IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBo%0D%0AMS0zIiB4PSI0OTcuOTMwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigz%0D%0ANC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNl%0D%0AIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iNTAzLjM1IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxp%0D%0Abms6aHJlZj0iI2dseXBoMS00IiB4PSI1MDcuOTEwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0%0D%0AeWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFj%0D%0AaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iNTEzLjMxIiB5PSI1MjQu%0D%0ANCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS01IiB4PSI1MTcuODcwMDA4IiB5PSI1MjQu%0D%0ANCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5%0D%0AOTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0i%0D%0ANTIzLjI3IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS02IiB4PSI1Mjcu%0D%0AODMwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0%0D%0ALjg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9%0D%0AIiNnbHlwaDEtNCIgeD0iNTMzLjIzIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2ds%0D%0AeXBoMS03IiB4PSI1MzcuNzkwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJn%0D%0AYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8%0D%0AdXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iNTQzLjIyIiB5PSI1MjQuNCIvPgogIDx1c2Ug%0D%0AeGxpbms6aHJlZj0iI2dseXBoMS04IiB4PSI1NDcuNzgwMDA4IiB5PSI1MjQuNCIvPgo8L2c+Cjxn%0D%0AIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1v%0D%0AcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNCIgeD0iNTUzLjE4IiB5PSI1%0D%0AMjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS05IiB4PSI1NTcuNzQwMDA4IiB5PSI1%0D%0AMjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQu%0D%0AODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNSIg%0D%0AeD0iNTYzLjE0IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0xMCIgeD0i%0D%0ANTY3LjcwMDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAy%0D%0AJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpo%0D%0AcmVmPSIjZ2x5cGgxLTUiIHg9IjU3My4xIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0i%0D%0AI2dseXBoMS0xIiB4PSI1NzcuNjYwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxs%0D%0AOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4K%0D%0AICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNSIgeD0iNTgzLjA4IiB5PSI1MjQuNCIvPgogIDx1%0D%0Ac2UgeGxpbms6aHJlZj0iI2dseXBoMS0yIiB4PSI1ODcuNjQwMDA4IiB5PSI1MjQuNCIvPgo8L2c+%0D%0ACjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7Zmls%0D%0AbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNSIgeD0iNTkzLjA0IiB5%0D%0APSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0zIiB4PSI1OTcuNjAwMDA4IiB5%0D%0APSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUs%0D%0AMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEt%0D%0ANSIgeD0iNjAzIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS00IiB4PSI2%0D%0AMDcuNTYwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIl%0D%0ALDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhy%0D%0AZWY9IiNnbHlwaDEtNSIgeD0iNjEyLjk2IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0i%0D%0AI2dseXBoMS01IiB4PSI2MTcuNTIwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxs%0D%0AOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4K%0D%0AICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNSIgeD0iNjIyLjkyIiB5PSI1MjQuNCIvPgogIDx1%0D%0Ac2UgeGxpbms6aHJlZj0iI2dseXBoMS02IiB4PSI2MjcuNDgwMDA4IiB5PSI1MjQuNCIvPgo8L2c+%0D%0ACjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7Zmls%0D%0AbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNSIgeD0iNjMyLjkiIHk9%0D%0AIjUyNC40Ii8+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTciIHg9IjYzNy40NjAwMDgiIHk9%0D%0AIjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwz%0D%0ANC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS01%0D%0AIiB4PSI2NDIuODYiIHk9IjUyNC40Ii8+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTgiIHg9%0D%0AIjY0Ny40MjAwMDgiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkw%0D%0AMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6%0D%0AaHJlZj0iI2dseXBoMS01IiB4PSI2NTIuODIiIHk9IjUyNC40Ii8+CiAgPHVzZSB4bGluazpocmVm%0D%0APSIjZ2x5cGgxLTkiIHg9IjY1Ny4zODAwMDgiIHk9IjUyNC40Ii8+CjwvZz4KPGcgc3R5bGU9ImZp%0D%0AbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsi%0D%0APgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS02IiB4PSI2NjIuNzgiIHk9IjUyNC40Ii8+CiAg%0D%0APHVzZSB4bGluazpocmVmPSIjZ2x5cGgxLTEwIiB4PSI2NjcuMzQwMDA4IiB5PSI1MjQuNCIvPgo8%0D%0AL2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7%0D%0AZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iNjcyLjc3%0D%0AIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0xIiB4PSI2NzcuMzMwMDA4%0D%0AIiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkw%0D%0AMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlw%0D%0AaDEtNiIgeD0iNjgyLjczIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS0y%0D%0AIiB4PSI2ODcuMjkwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44%0D%0AOTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhs%0D%0AaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iNjkyLjY5IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6%0D%0AaHJlZj0iI2dseXBoMS0zIiB4PSI2OTcuMjUwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxl%0D%0APSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5%0D%0AOjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iNzAyLjY1IiB5PSI1MjQuNCIv%0D%0APgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS00IiB4PSI3MDcuMjEwMDA4IiB5PSI1MjQuNCIv%0D%0APgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAy%0D%0AJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iNzEy%0D%0ALjYzIiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS01IiB4PSI3MTcuMTkw%0D%0AMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5%0D%0AOTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNn%0D%0AbHlwaDEtNiIgeD0iNzIyLjU5IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBo%0D%0AMS02IiB4PSI3MjcuMTUwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigz%0D%0ANC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNl%0D%0AIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iNzMyLjU1IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxp%0D%0Abms6aHJlZj0iI2dseXBoMS03IiB4PSI3MzcuMTEwMDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0%0D%0AeWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFj%0D%0AaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0iNzQyLjUxIiB5PSI1MjQu%0D%0ANCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS04IiB4PSI3NDcuMDcwMDA4IiB5PSI1MjQu%0D%0ANCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5%0D%0AOTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDEtNiIgeD0i%0D%0ANzUyLjQ3IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMS05IiB4PSI3NTcu%0D%0AMDU0MDA4IiB5PSI1MjQuNCIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0%0D%0ALjg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9%0D%0AIiNnbHlwaDEtNyIgeD0iNzYyLjQ2IiB5PSI1MjQuNCIvPgogIDx1c2UgeGxpbms6aHJlZj0iI2ds%0D%0AeXBoMS0xMCIgeD0iNzY3LjAyMDAwOCIgeT0iNTI0LjQiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpy%0D%0AZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAg%0D%0APHVzZSB4bGluazpocmVmPSIjZ2x5cGgyLTEiIHg9IjM3OS43IiB5PSI3NC45NSIvPgo8L2c+Cjxn%0D%0AIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1v%0D%0AcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDItMiIgeD0iMzgyLjgyMzQ1NiIg%0D%0AeT0iNzQuOTUiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIl%0D%0ALDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgy%0D%0ALTMiIHg9IjM5MC4yNTU2MDgiIHk9Ijc0Ljk1Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0%0D%0ALjg5OTkwMiUsMzQuODk5OTAyJSwzNC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2Ug%0D%0AeGxpbms6aHJlZj0iI2dseXBoMi0zIiB4PSIzOTUuNzYzNDg4IiB5PSI3NC45NSIvPgo8L2c+Cjxn%0D%0AIHN0eWxlPSJmaWxsOnJnYigzNC44OTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1v%0D%0AcGFjaXR5OjE7Ij4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDItNCIgeD0iNDAxLjI3MTM2OCIg%0D%0AeT0iNzQuOTUiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDItNSIgeD0iNDA0LjQyMjcxMiIg%0D%0AeT0iNzQuOTUiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIl%0D%0ALDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgy%0D%0ALTYiIHg9IjQwOC43MDM1MiIgeT0iNzQuOTUiLz4KICA8dXNlIHhsaW5rOmhyZWY9IiNnbHlwaDIt%0D%0ANyIgeD0iNDE1LjY0NzYzMiIgeT0iNzQuOTUiLz4KPC9nPgo8ZyBzdHlsZT0iZmlsbDpyZ2IoMzQu%0D%0AODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3BhY2l0eToxOyI+CiAgPHVzZSB4%0D%0AbGluazpocmVmPSIjZ2x5cGgyLTIiIHg9IjQyMi44NTY2OCIgeT0iNzQuOTUiLz4KPC9nPgo8ZyBz%0D%0AdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3Bh%0D%0AY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgyLTgiIHg9IjQzMC4yODg4MzIiIHk9%0D%0AIjc0Ljk1Ii8+CjwvZz4KPGcgc3R5bGU9ImZpbGw6cmdiKDM0Ljg5OTkwMiUsMzQuODk5OTAyJSwz%0D%0ANC44OTk5MDIlKTtmaWxsLW9wYWNpdHk6MTsiPgogIDx1c2UgeGxpbms6aHJlZj0iI2dseXBoMi05%0D%0AIiB4PSI0MzYuMjcwODA4IiB5PSI3NC45NSIvPgo8L2c+CjxnIHN0eWxlPSJmaWxsOnJnYigzNC44%0D%0AOTk5MDIlLDM0Ljg5OTkwMiUsMzQuODk5OTAyJSk7ZmlsbC1vcGFjaXR5OjE7Ij4KICA8dXNlIHhs%0D%0AaW5rOmhyZWY9IiNnbHlwaDItMyIgeD0iNDQzLjQ2NTkxMiIgeT0iNzQuOTUiLz4KPC9nPgo8ZyBz%0D%0AdHlsZT0iZmlsbDpyZ2IoMzQuODk5OTAyJSwzNC44OTk5MDIlLDM0Ljg5OTkwMiUpO2ZpbGwtb3Bh%0D%0AY2l0eToxOyI+CiAgPHVzZSB4bGluazpocmVmPSIjZ2x5cGgyLTEwIiB4PSI0NDguOTczNzkyIiB5%0D%0APSI3NC45NSIvPgo8L2c+CjxnIGNsaXAtcGF0aD0idXJsKCNjbGlwMikiIGNsaXAtcnVsZT0ibm9u%0D%0AemVybyI+CjxwYXRoIHN0eWxlPSJmaWxsOm5vbmU7c3Ryb2tlLXdpZHRoOjAuNzU7c3Ryb2tlLWxp%0D%0AbmVjYXA6YnV0dDtzdHJva2UtbGluZWpvaW46cm91bmQ7c3Ryb2tlOnJnYig4NS4wOTk3OTIlLDg1%0D%0ALjA5OTc5MiUsODUuMDk5NzkyJSk7c3Ryb2tlLW9wYWNpdHk6MTtzdHJva2UtbWl0ZXJsaW1pdDox%0D%0AMDsiIGQ9Ik0gNTAuMzk4NDM4IDYyLjE1MzEyNSBMIDc4Mi43ODEyNSA2Mi4xNTMxMjUgTCA3ODIu%0D%0ANzgxMjUgNTQxLjIwMzkwNiBMIDUwLjM5ODQzOCA1NDEuMjAzOTA2IFogTSA1MC4zOTg0MzggNjIu%0D%0AMTUzMTI1ICIgdHJhbnNmb3JtPSJtYXRyaXgoMSwwLDAsLTEsMCw1OTUuMikiLz4KPC9nPgo8L2c+%0D%0ACjwvc3ZnPgo=)

In [3]:
BATCH_SIZE = 32

indexesFromSentence(lang, sentence) = append!(get.(Ref(lang.word2index), split(lowercase(sentence), " "), 3), 2)

function batch(data, batch_size, voc_size; gpu=true)
    chunks = Iterators.partition(data, batch_size)
    batches = []
    for chunk in chunks
        max_length = maximum(length.(chunk))
        chunk = map(sentence->append!(sentence, fill(4, max_length-length(sentence))), chunk)
        chunk = hcat(reshape.(chunk, :, 1)...)
        batch = []
        for i in 1:size(chunk, 1)
            if gpu
                push!(batch, cu(Flux.onehotbatch(chunk[i, :], [1:voc_size...])))
            else
                push!(batch, Flux.onehotbatch(chunk[i, :], [1:voc_size...]))
            end
        end
        push!(batches, batch)
    end
    return(batches)
end

x, y = batch.([indexesFromSentence.([eng], xs), indexesFromSentence.([fr], ys)], [BATCH_SIZE], [eng.n_words, fr.n_words]; gpu=true);

# The Model

For the model, we're using a **encoder-decoder** architecture.
![encoder-decoder](https://smerity.com/media/images/articles/2016/gnmt_arch_attn.svg)
*image source: https://smerity.com/articles/2016/google_nmt_arch.html*

### High level overview
The **encoder** takes the OneHot-encoded words and uses the embedding layer to get their embedding, a multidimensional-representation of that word. Next, the words get passed through a RNN (in our case a GRU). For each word, the RNN spits out a state-vector (encoder-outputs).

The job of the **decoder** is to take the output of the encoder and mold it into a correct translation of the original sentence. The **attention** layer acts as a guide for the decoder. Every timestep (every time the decoder is to predict a word), it takes all the encoder-outputs and creates **one** state vector (the context vector) with the most relevant information for that particular timestep.

In [4]:
# some constants to be used for the model
HIDDEN = 128
LEARNING_RATE = 0.1
DROPOUT = 0.1;

For the encoder, we're using a bidirectional GRU, the input is read from front to back as well as from back to front. This should help for a more robust `encoder_output`.  
The `Flux.@treelike` macro makes sure all the parameters are recognized by the optimizer to optimise the values.

In [5]:
struct Encoder
    embedding
    dropout
    forward
    backward
    output
end
Encoder(voc_size::Integer; h_size::Integer=HIDDEN, dropout::Number=DROPOUT) = Encoder(
    param(Flux.glorot_uniform(h_size, voc_size)),
    Dropout(dropout),
    GRU(h_size, h_size),
    GRU(h_size, h_size),
    Dense(2*h_size, h_size))
function (e::Encoder)(x)
    x = map(x->e.embedding*x, x)
    return e.output.(vcat.(e.forward.(x), Flux.flip(e.backward, x)))
end
Flux.@treelike Encoder

The decoder takes the word it predicted in the previous timestep as well the `encoder_outputs`. The context vector gets created by passing these `encoder_outputs` as well as the current state of the decoder's RNN to the attention layer. Finally, the context vector is concatenated with the word of the previous timestep to predict the word of the current timestep.

*During the first timestep, the decoder doesn't have acces to a previously predicted word. To combat this, a `SOS`-token is provided*

In [6]:
struct Decoder
    embedding
    attention
    rnn
    output
end
Decoder(h_size, voc_size) = Decoder(
    param(Flux.glorot_uniform(h_size, voc_size)),
    Attention(h_size),
    GRU(h_size*2, h_size),
    Dense(h_size, voc_size, relu))
function (d::Decoder)(x, encoder_outputs; dropout=0)
    x = d.embedding * x
    x = Dropout(dropout)(x)
    decoder_state = d.rnn.state
    context = d.attention(encoder_outputs, decoder_state)
    x = d.rnn([x; context])
    x = softmax(d.output(x))
    return(x)
end
Flux.@treelike Decoder

Each state from the `encoder_outputs` gets concatenated with the decoder state one by one to get a weight, a measure of importance for that state. Each state gets multiplied by it's corresponding weight and all the states get summed together.

In [7]:
struct Attention
    linear
end
Attention(h_size::Int) = Attention(Dense(2*h_size, 1, tanh))
function (a::Attention)(encoder_outputs, decoder_state)
    weights = []
    results = []
    for state in encoder_outputs
        weight = a.linear([state; decoder_state])
        push!(weights, weight)
    end
    weights = softmax(vcat(weights...))
    return sum([encoder_outputs[i].*weights[i, :]' for i in 1:size(weights, 1)])
end
Flux.@treelike Attention

In [8]:
testEncoder = Encoder(eng.n_words)|>gpu
testDecoder = Decoder(HIDDEN, fr.n_words)|>gpu;

The model function is made to return the loss when the input and the target are provided.  
The hidden states of the RNN from both the encoder as well as the decoder are reset, by doing this you make sure no information of previous sentences is remembered.

The encoder_ouputs are made by passing the input through the encoder, the initial decoder input is made and the decoder's rnn state is initialized with the last encoder output.  
The decoder has to predict `max_length` words with `max_length` being the length of the longes sentence.

First off, the model decides whether teacher forcing will be used this timestep. Teacher forcing means instead of using the decoder output as the next timestep's decoder input, the correct input is used. Teacher forcing is especially useful in the beginning of training since decoder outputs won't make sense.

Every timestep, the decoder's prediction as well as the correct target are passed to a loss function. All the losses of all timesteps are summed up and returned.

In [9]:
function model(encoder::Encoder, decoder::Decoder, x, y; teacher_forcing = 0.5, dropout=DROPOUT, voc_size=fr.n_words)
    total_loss = 0
    max_length = length(y)
    batch_size = size(x[1], 2)
    Flux.reset!.([encoder, decoder])
    encoder_outputs = encoder(x)
    decoder_input = Flux.onehotbatch(ones(batch_size), [1:voc_size...])
    decoder.rnn.state = encoder_outputs[end]
    for i in 1:max_length
        use_teacher_forcing = rand() < teacher_forcing
        decoder_output = decoder(decoder_input, encoder_outputs; dropout=dropout)
        total_loss += loss(decoder_output, y[i])
        if use_teacher_forcing
            decoder_input = y[i]
        else
            decoder_input = Flux.onehotbatch(Flux.onecold(decoder_output.data), [1:voc_size...])
        end
    end
    return(total_loss)
end

model(x, y) = model(testEncoder, testDecoder, x, y; dropout = 0.05)

model (generic function with 2 methods)

When the target is not provided to the `model` function, the model returns a prediction instead of a loss value.


*Note that, when the model is trained, the loop could be set to run indefinitely because the loop will break when an `EOS`-token is predicted.
I've set the loop to run for an arbitrary amount of timesteps (in this case 12) because the model doesn't seem to be able to learn to predict an `EOS token`*

In [10]:
function model(encoder::Encoder, decoder::Decoder, x; reset=true, voc_size=fr.n_words)
    result = []
    if reset Flux.reset!.([encoder, decoder]) end
    encoder_outputs = encoder(x)
    decoder_input = Flux.onehot(1, [1:voc_size...])
    decoder.rnn.state = encoder_outputs[end]
    for i in 1:12
        decoder_output = Flux.onecold(decoder(decoder_input, encoder_outputs))
        if decoder_output[1] == 2 break end
        push!(result, decoder_output...)
    end
    return(result)
end

model (generic function with 3 methods)

The `loss` function expects a probability distribution over all possible words in the vocabulary, this gets accounted for by the softmax layer in the decoder. The loss function itself is crossentropy (a.k.a. negative-log-likelihood).  
We pass an vector of ones, except for the `PAD`-index (4) as weight to the loss function. This way the model will disregard any predictions that should have been PAD, since padding only occurs after the sentence has ended.


For the optimizer, we use (stochastic) gradient descent.

In [11]:
lossmask = ones(fr.n_words)|>gpu
lossmask[4] = 0

loss(logits, target) = Flux.crossentropy(logits, target; weight=lossmask)

opt = SGD(params(testEncoder, testDecoder), LEARNING_RATE)

#43 (generic function with 1 method)

The data (`x` and `y`) gets passed to `partitionTrainTest` to split the data in a train and a test set.

Finally the model is trained for a number of epochs. Every epoch, the loss on the test set gets printed.

In [12]:
function partitionTrainTest(x, y, at)
    n = length(x)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    train_x, test_x = x[train_idx,:], x[test_idx,:]
    train_y, test_y = y[train_idx,:], y[test_idx,:]
    return(train_x, train_y, test_x, test_y)
end

train_x, train_y, test_x, test_y = partitionTrainTest(x, y, 0.90);

In [ ]:
EPOCHS = 15

for i in 1:EPOCHS
    Flux.train!(model, zip(train_x, train_y), opt)
    println("loss: ", mean(model.(test_x, test_y)).data)
end

loss: 

The `predict` function takes an encoder, decoder and an english sentence. It converts the sentence into it's OneHot representation and passes it to the `model` function. The output gets converted back to a string and returned.

In [ ]:
function predict(encoder, decoder, sentence::String)
    sentence = normalizeString(sentence)
    input = append!(get.(Ref(eng.word2index), split(lowercase(sentence), " "), 3), 2)
    input = [Flux.onehot(word, [1:eng.n_words...]) for word in input]
    output = model(encoder, decoder, input)
    output = get.(Ref(fr.index2word), output, "UNK")
    println(output)
end

In [ ]:
predict(testEncoder, testDecoder, "she's doing her thing")
predict(testEncoder, testDecoder, "you're too skinny")
predict(testEncoder, testDecoder, "He is singing")

As you can see, when I run the model for 70 epochs, the results are quite underwhelming... Even though sentence subjects are mostly correct, most part of the translation does not make sense.

If you have a look at the loss on the test set during these 70 epochs, you can clearly see the model seems to hit a barrier around 18.

I'm not sure why this is happening and I'd love to know! If you've got an idea on how to improve/fix this model, definitely let me know.

Thanks
![encoder-decoder](C:/users/Jules/desktop/grafiek.svg)